# **Rental property listings in Berlin**

# Import packages

In [ ]:
import pandas as pd
import numpy as np
import re
from pickle import TRUE
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from google.colab import files
uploaded = files.upload()

Saving merged_listings.csv to merged_listings.csv


# Import the dataset containing web-scraped rental property listings

In [ ]:
df = pd.read_csv("merged_listings.csv")

# Removing duplicate listings

Remove duplicate listings by checking for duplicate values in the 'URL' column, as each property should have a unique URL.

In [ ]:
# Check for duplicates in the 'URL' column
duplicate_rows = df[df.duplicated(subset=['URL'], keep=False)]  # Keep=False shows all duplicates
# Print the count of duplicates
print(f"Number of duplicate rows based on 'URL': {duplicate_rows.shape}")

# Remove duplicates, keeping only the first occurrence
df = df.drop_duplicates(subset=['URL'], keep='first')
# Verify that duplicates are removed
print(f"New dataset size: {df.shape}")

Number of duplicate rows based on 'URL': (0, 38)
New dataset size: (3808, 38)


# Filtering out swap listings

This project focuses only on rental property listings. During an initial review of the data, I noticed that all listings from the real estate agency 'Tauschwohnung GmbH' were related to property swapping rather than rentals, so I decided to exclude them.

In [ ]:
df[['Real Estate 1', 'Real Estate 2', 'Real Estate 3', 'Real Estate 4', 'Real Estate 5 Subtitle', 'Real Estate 5 Title', 'URL']]

,Real Estate 1,Real Estate 2,Real Estate 3,Real Estate 4,Real Estate 5 Subtitle,Real Estate 5 Title,URL
0,NaN,NaN,NaN,Privater Anbieter,NaN,Privater Anbieter,https://www.immowelt.de/expose/83a070dd-c407-4...
1,NaN,NaN,Ambius Immobilien GmbH,Milena Hyer,Milena Hyer,Ambius Immobilien GmbH,https://www.immowelt.de/expose/f8942fb4-1007-4...
2,NaN,NaN,Blueground Germany GmbH,Team Blueground,Team Blueground,Blueground Germany GmbH,https://www.immowelt.de/expose/fd9e5847-f017-4...
3,NaN,NaN,NaN,Privater Anbieter,NaN,Privater Anbieter,https://www.immowelt.de/expose/599eb8d0-b846-4...
4,NaN,NaN,HOMELIKE INTERNET GmbH,Team Homelike,Team Homelike,HOMELIKE INTERNET GmbH,https://www.immowelt.de/expose/53457c89-871f-4...
...,...,...,...,...,...,...,...
3803,NaN,NaN,Tauschwohnung GmbH,Herr John Weinert,Herr John Weinert,Tauschwohnung GmbH,https://www.immowelt.de/expose/72baf2c6-1b41-4...
3804,NaN,NaN,Tauschwohnung GmbH,Herr John Weinert,Herr John Weinert,Tauschwohnung GmbH,https://www.immowelt.de/expose/8591f50d-b1d9-4...
3805,NaN,NaN,HOMELIKE INTERNET GmbH,Team Homelike,Team Homelike,HOMELIKE INTERNET GmbH,https://www.immowelt.de/expose/56e439f3-4463-4...
3806,NaN,NaN,SCHNORR & PARTNER IMMOBILIEN,Albrecht Schnorr,Albrecht Schnorr,SCHNORR & PARTNER IMMOBILIEN,https://www.immowelt.de/expose/195f8fcc-6324-4...


In [ ]:
df["Real Estate 3"].value_counts(dropna=False)

,count
Real Estate 3,
HOMELIKE INTERNET GmbH,1552
Tauschwohnung GmbH,790
Engel & Völkers Berlin Mitte GmbH,107
Blueground Germany GmbH,107
NaN,96
...,...
Hans-Joachim Braun Hausverwaltung,1
Heese und Acar Immobilien GmbH,1
NAMOLA GmbH,1


In [ ]:
df = df[df["Real Estate 3"] != "Tauschwohnung GmbH"]

# Redefining column headers

Redefining column headers to include units for better clarity.

In [ ]:
df.columns

Index(['Title', 'Kaltmiete zzgl. Nebenkosten', 'Warmmiete', 'Nebenkosten',
       'Heizkosten_1', 'Heizkosten_2', 'Miete pro Stellplatz', 'Rooms_number',
       'Surface Area', 'Floor', 'availability', 'Adress', 'Zip Code',
       'Additional Price Info', 'pics_number', 'energy_efficiency',
       'Year of Construction', 'Condition', 'Heating type', 'Heating system',
       'Price per sqm', 'Caution 1', 'Caution 2', 'Real Estate 1',
       'Real Estate 2', 'Real Estate 3', 'Real Estate 4',
       'Real Estate 5 Subtitle', 'Real Estate 5 Title', 'URL', 'Merkmale_1',
       'Merkmale_2', 'Merkmale_3', 'Merkmale_4', 'Merkmale_5', 'Merkmale_6',
       'Merkmale_7', 'Merkmale_8'],
      dtype='object')

In [ ]:
df.columns = ["Title", "Kaltmiete zzgl. Nebenkosten (€/month)", "Warmmiete (€/month)", "Nebenkosten (€/month)",
              "Heizkosten_1 (€/month)", "Heizkosten_2 (€/month)", "Miete pro Stellplatz (€/month)", "Rooms_number",
              "Surface Area (m^2)", "Floor", "availability", "Address", "Zip Code", "Additional Price Info",
              "pics_number", "energy_efficiency", "Year of Construction", "Condition", "Heating type", "Heating system",
              "Price per sqm", "Deposit 1", "Deposit 2", "Real Estate 1", "Real Estate 2", "Real Estate 3", "Real Estate 4",
              "Real Estate 5 Subtitle", "Real Estate 5 Title", "URL", "Feature_1", "Feature_2", "Feature_3", "Feature_4",
              "Feature_5", "Feature_6", "Feature_7", "Feature_8"]  # Replace with actual names

# Preparing a new column "Bezirk"

To add a 'Bezirk' column later, I created a mapping of zip codes to Berlin district names using information from an online source (e.g., https://www.in-berlin-brandenburg.com/Berliner_Bezirke/plz-berlin.html).

In [ ]:
# Define the mapping of zip codes to Bezirke
bezirk_mapping = {
    "Charlottenburg-Wilmersdorf": [
        10585, 10587, 10589, 10623, 10625, 10627, 10629, 10707, 10709, 10711,
        10719, 10787, 10789, 14050, 14055, 14057, 14059, 13353, 13627, 13629,
        14193, 14195, 14199, 10713, 14197, 10715, 10717, 14052
    ],
    "Friedrichshain-Kreuzberg": [
        10243, 10179, 10245, 10317, 10247, 10249, 10178, 10961, 10969, 10963,
        10785, 10965, 10967, 10997, 10999
    ],
    "Lichtenberg-Hohenschönhausen": [
        12681, 13051, 13053, 13055, 13057, 10367, 10369, 10315, 10317, 10319,
        10365, 10318, 13059
    ],
    "Marzahn-Hellersdorf": [
        12683, 12685, 12555, 12619, 12621, 12623, 12627, 12629, 12679, 12681,
        12687, 12689
    ],
    "Mitte": [
        13347, 13353, 13355, 13357, 13359, 13409, 10555, 10557, 10115, 10117,
        10119, 10178, 10179, 10435, 10551, 10553, 10559, 13349, 13351, 13405,
        13407
    ],
    "Neukölln": [
        12051, 12057, 12099, 12347, 12349, 12351, 12359, 12107, 12305, 12353,
        12357, 10965, 10967, 12043, 12045, 12047, 12049, 12053, 12055, 12059,
        12355
    ],
    "Pankow": [
        13051, 13125, 13129, 13127, 13158, 13159, 13086, 13088, 13089, 10439,
        13187, 13189, 10119, 10247, 10249, 10369, 10405, 10407, 10409, 10435,
        10437, 10439, 13156
    ],
    "Reinickendorf": [
        13403, 13509, 13465, 13503, 13505, 13467, 13435, 13469, 13439, 13407,
        13409, 13437, 13507, 13599, 13629
    ],
    "Schoenefeld-Waltersdorf": [
        12529
    ],
    "Spandau": [
        13583, 13585, 13589, 13591, 14089, 13587, 13597, 13599, 13627, 13629,
        13581, 13593, 13595
    ],
    "Steglitz-Zehlendorf": [
        12203, 14169, 14193, 14195, 14199, 12167, 12209, 12247, 12249, 12277,
        12165, 12205, 12207, 12279, 14167, 14109, 14129, 14163, 12157, 12161,
        12163, 12169, 14165
    ],
    "Tempelhof-Schöneberg": [
        10827, 12159, 12161, 12163, 14197, 12107, 12277, 12305, 12307, 12309,
        12099, 12105, 12109, 12279, 10777, 10779, 10781, 10783, 10785, 10787,
        10789, 10823, 10825, 10829, 12101, 12103, 12157
    ],
    "Treptow-Köpenick": [
        12439, 12487, 12489, 12435, 12524, 12526, 12437, 12587, 12527, 12555,
        12557, 12559, 12589, 10318, 12459
    ]
}

# Converting relevant columns to a numeric data type

## Converting 'Rooms_number' to a numeric data type

While reviewing the collected values, I noticed that the web scraping process did not go as smoothly as expected. Cleaning the scraped data will require significant effort to ensure accuracy while minimizing data loss.

In [ ]:
df[["Rooms_number", "URL"]]

,Rooms_number,URL
0,2,https://www.immowelt.de/expose/83a070dd-c407-4...
1,1,https://www.immowelt.de/expose/f8942fb4-1007-4...
2,2,https://www.immowelt.de/expose/fd9e5847-f017-4...
3,3,https://www.immowelt.de/expose/599eb8d0-b846-4...
4,3,https://www.immowelt.de/expose/53457c89-871f-4...
...,...,...
3797,1,https://www.immowelt.de/expose/e0e4544e-226f-4...
3799,2,https://www.immowelt.de/expose/3bc83d6c-d821-4...
3801,2,https://www.immowelt.de/expose/0f2c7cbd-9153-4...
3805,1,https://www.immowelt.de/expose/56e439f3-4463-4...


In [ ]:
df["Rooms_number"].value_counts(dropna=False)

,count
Rooms_number,
1,949
2,838
3,574
4,393
5,116
6,33
"1,5",24
"2,5",15
"3,5",12


A comma (',') was often used as a decimal separator, so I replaced it with a period ('.') to ensure proper numerical formatting.

In [ ]:
df["Rooms_number"] = df["Rooms_number"].str.replace(",", ".")

After that, I attempted to convert the 'Rooms_number' column (which represents the number of rooms in the property) to a numeric data type. Since I wanted to preserve as many data points as possible, I made sure that the code returned errors instead of automatically converting them to NaN values.

This approach allowed me to manually check each error, open the corresponding listing URLs, and correct the values myself.

For some error values, I noticed that they were not complete mistakes. Instead of representing the number of rooms, some values actually referred to the surface area of the property. In such cases, I reassigned these values to their proper columns.

In [ ]:
try:
    df["Rooms_number"] = pd.to_numeric(df["Rooms_number"], errors='raise')
except ValueError as e:
    print("Error:", e)

Error: Unable to parse string "19 m²" at position 161


In [ ]:
df.loc[3642, ["availability", "Surface Area (m^2)", "Rooms_number", "URL"]]

,3642
availability,01.03.2025
Surface Area (m^2),EG
Rooms_number,15 m²
URL,https://www.immowelt.de/expose/df43c1a5-65fa-4...


For example, in the listing corresponding to row 3642, the 'Rooms_number' column contained the surface area (m²) instead of the number of rooms. Additionally, the 'Surface Area (m²)' column contained useful information about the property's floor level (e.g., 'EG' for ground floor). This value should be reassigned to the 'floor' column, but only if that column does not already contain the correct information.

In [ ]:
df.loc[3642, ["Floor", "URL"]]

,3642
Floor,NaN
URL,https://www.immowelt.de/expose/df43c1a5-65fa-4...


After checking, whether that listing contained that information, we can see that it was missing it, therefore I also manually reassigned this value.

Hence the following line of code:
```
df.loc[3642, ["Surface Area (m^2)", "Floor", "Rooms_number"]] = [15, "EG",np.nan]
```

Here, the surface area is correctly set to 15 m², the floor level is assigned as 'EG' (ground floor), and the incorrect room number entry is replaced with NaN.



In [ ]:
df.loc[2160, ["availability", "Surface Area (m^2)", "Rooms_number"]] = ["01.03.2025", 12, np.nan]
df.loc[2190, ["availability", "Surface Area (m^2)", "Rooms_number"]] = ["01.03.2025", 12, np.nan]
df.loc[2169, ["availability", "Surface Area (m^2)", "Rooms_number"]] = ["01.03.2025", 9, np.nan]
df.loc[227, ["Surface Area (m^2)", "Rooms_number"]] = [19, np.nan]
df.loc[912, ["Surface Area (m^2)", "Rooms_number"]] = [20, np.nan]
df.loc[1037, ["Surface Area (m^2)", "Rooms_number"]] = [16, np.nan]
df.loc[1040, ["Surface Area (m^2)", "Rooms_number"]] = [19, np.nan]
df.loc[1041, ["Surface Area (m^2)", "Rooms_number"]] = [185, np.nan]
df.loc[1063, ["Surface Area (m^2)", "Rooms_number"]] = [21, np.nan]
df.loc[1079, ["Surface Area (m^2)", "Rooms_number"]] = [26, np.nan]
df.loc[1081, ["Surface Area (m^2)", "Rooms_number"]] = [21, np.nan]
df.loc[1083, ["Surface Area (m^2)", "Rooms_number"]] = [31, np.nan]
df.loc[1086, ["Surface Area (m^2)", "Rooms_number"]] = [38, np.nan]
df.loc[1097, ["Surface Area (m^2)", "Rooms_number"]] = [21, np.nan]
df.loc[1101, ["Surface Area (m^2)", "Rooms_number"]] = [55, np.nan]
df.loc[1111, ["Surface Area (m^2)", "Rooms_number"]] = [26, np.nan]
df.loc[1266, ["Surface Area (m^2)", "Rooms_number"]] = [10, np.nan]
df.loc[1897, ["Surface Area (m^2)", "Rooms_number"]] = [22, np.nan]
df.loc[1912, ["Surface Area (m^2)", "Rooms_number"]] = [16, np.nan]
df.loc[1989, ["Surface Area (m^2)", "Rooms_number"]] = [42, np.nan]
df.loc[2016, ["Surface Area (m^2)", "Rooms_number"]] = [20, np.nan]
df.loc[2098, ["Surface Area (m^2)", "Rooms_number"]] = [10, np.nan]
df.loc[2110, ["Surface Area (m^2)", "Rooms_number"]] = [12, np.nan]
df.loc[2177, ["Surface Area (m^2)", "Rooms_number"]] = [62, np.nan]
df.loc[2185, ["Surface Area (m^2)", "Rooms_number"]] = [21, np.nan]
df.loc[2191, ["Surface Area (m^2)", "Rooms_number"]] = [21, np.nan]
df.loc[2196, ["Surface Area (m^2)", "Rooms_number"]] = [40, np.nan]
df.loc[2842, ["Surface Area (m^2)", "Rooms_number"]] = [30, np.nan]
df.loc[3024, ["Surface Area (m^2)", "Rooms_number"]] = [20, np.nan]
df.loc[3227, ["Surface Area (m^2)", "Rooms_number"]] = [10, np.nan]
df.loc[3361, ["Surface Area (m^2)", "Rooms_number"]] = [20, np.nan]
df.loc[3471, ["Surface Area (m^2)", "Rooms_number"]] = [23.2, np.nan]
df.loc[3481, ["Surface Area (m^2)", "Rooms_number"]] = [31, np.nan]
df.loc[3527, ["Surface Area (m^2)", "Rooms_number"]] = [25.1, np.nan]
df.loc[3578, ["Surface Area (m^2)", "Rooms_number"]] = [22, np.nan]
df.loc[3592, ["Surface Area (m^2)", "Rooms_number"]] = [16, np.nan]
df.loc[3596, ["Surface Area (m^2)", "Rooms_number"]] = [22, np.nan]
df.loc[3604, ["Surface Area (m^2)", "Rooms_number"]] = [17, np.nan]
df.loc[3642, ["Surface Area (m^2)", "Floor", "Rooms_number"]] = [15, "EG",np.nan]

Here, I verified that the intended changes were successfully applied:

In [ ]:
df.loc[3642, ["availability", "Surface Area (m^2)", "Rooms_number", "Floor", "URL"]]

,3642
availability,01.03.2025
Surface Area (m^2),15
Rooms_number,NaN
Floor,EG
URL,https://www.immowelt.de/expose/df43c1a5-65fa-4...


After making these corrections, I attempted to convert the 'Rooms_number' column to a numeric data type. I continued fixing any remaining issues until no more errors were returned for this column.

In [ ]:
try:
    df["Rooms_number"] = pd.to_numeric(df["Rooms_number"], errors='raise')
except ValueError as e:
    print("Error:", e)

## Converting the columns 'Miete pro Stellplatz (€/month)', 'Price per sqm', and 'Heizkosten_2 (€/month)' to numeric data types

I followed the same approach to convert the values in the columns 'Miete pro Stellplatz (€/month)', 'Price per sqm', and 'Heizkosten_2 (€/month)' to numeric data types. Instead of automatically replacing problematic values with 'np.nan' (as would happen with errors='raise'), I intentionally raised errors to manually resolve as many issues as possible.

To do this, I first examined the unique values in each column using the following command:
```
.value_counts(dropna=False)
```

This allowed me to identify and correct inconsistencies before proceeding with the conversion.

In [ ]:
df["Miete pro Stellplatz (€/month)"].value_counts(dropna=False)

,count
Miete pro Stellplatz (€/month),
NaN,2824
110 €110 €,38
100 €100 €,38
120 €120 €,32
150 €150 €,22
80 €80 €,8
50 €50 €,8
"236.81 €236,81 €",6
250 €250 €,5


In [ ]:
df["Miete pro Stellplatz (€/month)"] = pd.to_numeric(
    df["Miete pro Stellplatz (€/month)"].str.extract(r'(\d+)')[0],
    errors='raise'
)

Then, I double-checked to ensure that the conversions were applied correctly.

In [ ]:
df["Miete pro Stellplatz (€/month)"].value_counts(dropna=False)

,count
Miete pro Stellplatz (€/month),
NaN,2824
110.0,38
100.0,38
120.0,32
150.0,22
80.0,8
50.0,8
236.0,6
250.0,5


Similar steps were followed for "Heizkosten_2 (€/month)" and "Price per sqm".

In [ ]:
df["Heizkosten_2 (€/month)"] = pd.to_numeric(
    df["Heizkosten_2 (€/month)"].str.extract(r'(\d+)')[0],
    errors='raise'
)
df["Heizkosten_2 (€/month)"] = pd.to_numeric(df["Heizkosten_2 (€/month)"], errors='raise')

df["Price per sqm"] = df["Price per sqm"].str.extract(r'([\d,]+)', expand=False)
df["Price per sqm"] = df["Price per sqm"].str.replace(",", ".")
df["Price per sqm"] = pd.to_numeric(df["Price per sqm"], errors='raise')

## Converting "Year of Construction" and "Floor" to numeric data types

I followed similar steps for the columns 'Year of Construction' and 'Floor'.

During this process, I noticed that some values in the 'Year of Construction' column actually belonged to the 'Condition' column. I manually corrected these misplacements.

Additionally, I manually modified some values in the 'Year of Construction' column after verifying their corresponding property listings using their URLs.

In [ ]:
df["Condition"].value_counts(dropna=False)

,count
Condition,
NaN,2103
neuwertig,269
renoviert / saniert,180
Erstbezug,132
Gepflegt,98
"Altbau (bis 1945), renoviert / saniert",81
Neubau,48
"Neubau, Erstbezug",29
"Neubau, neuwertig",23


In [ ]:
df["Year of Construction"].value_counts(dropna=False)

,count
Year of Construction,
NaN,1846
2024.0,153
2023.0,106
1900.0,60
2020.0,53
...,...
1881.0,1
2011.0,1
1935.0,1


In [ ]:
df.loc[1072, ["Condition", "Year of Construction"]] = ["neuwertig", np.nan]
df.loc[1483, ["Year of Construction"]] = [np.nan]
df.loc[2670, ["Year of Construction"]] = [1910]
df.loc[2963, ["Year of Construction"]] = [np.nan]
df.loc[3503, ["Year of Construction"]] = [2012]
df.loc[3524, ["Condition", "Year of Construction"]] = ["neuwertig", np.nan]
df.loc[3551, ["Year of Construction"]] = [2012]

In [ ]:
df["Year of Construction"] = pd.to_numeric(df["Year of Construction"], errors='raise')

Here a code to clean and convert the values in the "Floor" column.

- new_floor_values is a list to store the clean floor values
- error_rows is a list to store rows with conversion errors

The function loops through each row of the Floor column:
- If val is NaN (missing value), add np.nan to new_floor_values.
- If the value is not missing:
It converts it to a string, removes extra spaces (strip()) and non-breaking spaces (\xa0).

The function handles different floor format:
- If the retrieved string s is "EG" (Erdgeschoss, meaning "ground floor" in German), store it as 0.
- If the string s contains a "/", it represents a flat or a house taking several floors, for instance from the 2nd to the 4rth floor. (e.g., "2/4"). In that case, the function splits s at /, converts both parts to float, and stores the average. Example: "2/4" → (2 + 4) / 2 = 3.0. It also adds a "1" to the "Multiple floors" column.
- Normal Floor Numbers (e.g., "5."). The function converts the string s to a float, removing any periods ".".

In [ ]:
df.loc[1102, ["Floor"]] = [np.nan]
df.loc[2149, ["Floor"]] = [np.nan]
df.loc[1266, ["Floor"]] = [np.nan]
df.loc[2090, ["Floor"]] = [np.nan]
df.loc[2541, ["Floor"]] = [np.nan]

In [ ]:
df['Multiple floors'] = np.nan # Initialize column with NaN

new_floor_values = [] #store the clean floor values
error_rows = []  # List to store rows with conversion errors

#Looping through each row of the Floor column
for idx, val in df["Floor"].items():
    """
   1. The function loops through each row of the Floor column:
   If val is NaN (missing value), add np.nan to new_floor_values.
   If the value is not missing: It converts it to a string, removes extra spaces (strip()) and non-breaking spaces (\xa0).

    2. The function handles different floor format:
    If the retrieved string s is "EG" (Erdgeschoss, meaning "ground floor" in German), store it as 0.
    If the string s contains a "/", it represents a flat or a house taking several floors, for instance from the 2nd to the 4rth floor. (e.g., "2/4").
    In that case, the function splits s at /, converts both parts to float, and stores the average. Example: "2/4" → (2 + 4) / 2 = 3.0.
    It also adds a "1" to the "Multiple floors" column.
    Normal Floor Numbers (e.g., "5."). The function converts the string s to a float, removing any periods ".".
    """
    try:
        if pd.isna(val):
            new_floor_values.append(np.nan)
        else:
            s = str(val).strip().replace("\xa0", "")
            if s == 'EG':
                new_floor_values.append(0)
            elif "/" in s:
                parts = s.split("/")
                new_floor_values.append((float(parts[0]) + float(parts[1])) / 2)
                df.loc[idx, 'Multiple floors'] = 1  # Mark as multiple floors
            else:
                new_floor_values.append(float(s.rstrip(".")))
    except ValueError as e:
        error_rows.append(idx)  # Store row index in the list
        new_floor_values.append(np.nan)

## Converting "Deposit 2" to a numeric data type

First, I examined the values in the 'Deposit 2' column using the following function to understand its content and identify any inconsistencies:

```
.value_counts(dropna=False)
```
This helped me detect patterns, errors, and missing values before proceeding with data cleaning.



In [ ]:
df["Deposit 2"].value_counts(dropna=False)

,count
Deposit 2,
NaN,2388
950 €,161
2.000 €,73
900 €,44
3.000 €,18
...,...
"2.469,06 €",1
9.000 €,1
"946,50 €",1


Cleaning and formatting the "Deposit 2" column in the df DataFrame by converting it into a numeric-friendly format.

- converts values to a string
- removes the currency symbol "€" and non-breaking spaces (\xa0).
- removes dots ".", which are often used as thousands separators
- Replaces commas "," with dots ".", making the numbers compatible with decimal notation in Python

In [ ]:
df["Deposit 2"] = df["Deposit 2"].astype(str).str.replace("\xa0€", "", regex=True).str.replace(".", "", regex=False).str.replace(",", ".", regex=False)

After performing initial cleaning and formatting of the 'Deposit 2' column to facilitate conversion into a numeric data type, I wanted to check if any remaining values might still cause issues.

To do this:
1. I tried converting all of the values to a numeric type.
2. I stored the row indices of problematic values.
3. After manually checking problematic values, I decided to transform them to np.nan, because althought it was indicated "nan", those were strings and not real missing values.
4. I double-checked that no problematic values were remaining.

In [ ]:
# I tried converting all of the values to a numeric type.
problematic_values = df.loc[pd.to_numeric(df["Deposit 2"], errors="coerce").isna(), "Deposit 2"]

# I stored the row indices of problematic values.
problematic_rows = problematic_values.index.tolist()

In [ ]:
df.loc[problematic_rows, ["Deposit 2", "URL"]]

,Deposit 2,URL
2,nan,https://www.immowelt.de/expose/fd9e5847-f017-4...
4,nan,https://www.immowelt.de/expose/53457c89-871f-4...
6,nan,https://www.immowelt.de/expose/c9184088-278f-4...
9,nan,https://www.immowelt.de/expose/7760e8c6-ce79-4...
12,nan,https://www.immowelt.de/expose/0e187981-2b42-4...
...,...,...
3792,nan,https://www.immowelt.de/expose/814b26ea-4739-4...
3795,nan,https://www.immowelt.de/expose/c1cd0abe-d59b-4...
3797,nan,https://www.immowelt.de/expose/e0e4544e-226f-4...
3801,nan,https://www.immowelt.de/expose/0f2c7cbd-9153-4...


In [ ]:
problematic_values.value_counts(dropna=False)

,count
Deposit 2,
nan,2388


After manually reviewing the problematic values, I decided to convert them to np.nan. Although they were labeled as 'nan', they were actually strings rather than true missing values. Converting them ensures proper handling of missing data during analysis.

In [ ]:
df.loc[problematic_rows, ["Deposit 2"]] = np.nan

I double-checked to ensure that no problematic values remained after the cleaning process:

In [ ]:
df["Deposit 2"] = pd.to_numeric(df["Deposit 2"], errors="raise")

## Converting 'energy_efficiency' to a numeric data type

In [ ]:
df["energy_efficiency"].value_counts(dropna=False)

,count
energy_efficiency,
NaN,2321
B,231
A,145
C,127
D,80
E,49
A+,34
F,22
G,6


I defined a mapping for energy efficiency ratings to convert the column values into a numeric data type. After applying the mapping, I double-checked to ensure that no errors occurred.

In [ ]:
# Define the mapping for Energy efficiency
efficiency_mapping = {
    "A+": 0,
    "A": 1,
    "B": 2,
    "C": 3,
    "D": 4,
    "E": 5,
    "F": 6,
    "G": 7,
    "H": 8
}

# Apply the mapping to the 'energy_efficiency' column
df["energy_efficiency"] = df["energy_efficiency"].replace(efficiency_mapping)

df["energy_efficiency"] = pd.to_numeric(df["energy_efficiency"], errors="raise")

<ipython-input-37-5b5c582d4434>:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["energy_efficiency"] = df["energy_efficiency"].replace(efficiency_mapping)


I double-checked to ensure that the changes were correctly applied:

In [ ]:
df["energy_efficiency"].value_counts(dropna=False)

,count
energy_efficiency,
NaN,2321
2.0,231
1.0,145
3.0,127
4.0,80
5.0,49
0.0,34
6.0,22
7.0,6


## Converting the 'Zip Code' column into an additional 'Bezirk' column,
replacing zip codes with district names.

1. convert the "Zip Code" column to numeric datatype checking that no errors occured.
2. For this, the mapping "bezirk_mapping" created ealier was used
3. Create a function to map zip codes to Bezirke using the postal codes to berlin's district mapping created above.
4. Apply the mapping function, returning a new column called "Bezirk", meaning district in German.

In [ ]:
# Convert the "Zip Code" column to numeric, print errors
try:
    df["Zip Code"] = pd.to_numeric(df['Zip Code'], errors='raise')
except ValueError as e:
    print("Error:", e)

# Create a function to map zip codes to Bezirke
def get_bezirk(zip_code):
    for bezirk, zip_codes in bezirk_mapping.items():
        if zip_code in zip_codes:
            return bezirk
    return None  # If no match is found

# Apply the mapping function
df["Bezirk"] = df["Zip Code"].apply(get_bezirk)

We can now examine the values in the newly created 'Bezirk' column. This allows us to see how many listings our dataset contains for each district (Bezirk) in Berlin.

In [ ]:
df["Bezirk"].value_counts(dropna=False)

,count
Bezirk,
Friedrichshain-Kreuzberg,696
Mitte,679
Charlottenburg-Wilmersdorf,446
Pankow,364
Treptow-Köpenick,184
Tempelhof-Schöneberg,156
Neukölln,148
Steglitz-Zehlendorf,91
Lichtenberg-Hohenschönhausen,74


When reviewing the values in the 'Bezirk' column, I noticed that 10 listings were not assigned to any district (i.e., they were labeled as 'None').

To investigate further, I used Google Colab's interactive table, sorted the 'Bezirk' column in ascending order, and examined the missing values first. By checking the corresponding URLs for these listings, I discovered that the web scraper had captured properties located just outside Berlin, in neighboring cities/towns such as Ahrensfelde (16356), Glienicke/Nordbahn (16548), and Teltow (14513).

Since this project focuses specifically on rental properties within Berlin, I later removed these listings from the dataset.

In [ ]:
df[['Bezirk', "URL"]]

,Bezirk,URL
0,Treptow-Köpenick,https://www.immowelt.de/expose/83a070dd-c407-4...
1,Mitte,https://www.immowelt.de/expose/f8942fb4-1007-4...
2,Friedrichshain-Kreuzberg,https://www.immowelt.de/expose/fd9e5847-f017-4...
3,Reinickendorf,https://www.immowelt.de/expose/599eb8d0-b846-4...
4,Pankow,https://www.immowelt.de/expose/53457c89-871f-4...
...,...,...
3797,Friedrichshain-Kreuzberg,https://www.immowelt.de/expose/e0e4544e-226f-4...
3799,Mitte,https://www.immowelt.de/expose/3bc83d6c-d821-4...
3801,Mitte,https://www.immowelt.de/expose/0f2c7cbd-9153-4...
3805,Charlottenburg-Wilmersdorf,https://www.immowelt.de/expose/56e439f3-4463-4...


## Handling and converting "pics_number" to a data column type

In [ ]:
df["pics_number"].value_counts(dropna=False)

,count
pics_number,
1 / 6,238
1 / 10,236
1 / 11,207
1 / 9,195
1 / 12,188
1 / 8,170
1 / 13,164
1 / 7,155
1 / 14,135


The values in the 'pics_number' column follow the format "1/*", where the "1/" indicates that the user is currently viewing the first picture out of a total number of images for the listing. However, I was only interested in extracting the total number of pictures available for each property.

To achieve this, I took the following steps:
1. Extracting the Total Number of Pictures:
- I extracted the part of the string after the / symbol, as it contains the total number of pictures for the listing.
2. Converting to Numeric Values:
- I converted the extracted values to a numeric data type.
- Instead of automatically replacing errors with np.nan, I raised them to manually review and correct any problematic entries.

In [ ]:
df["pics_number"] = df["pics_number"].str.split(" / ").str[-1]
try:
    df["pics_number"] = pd.to_numeric(df["pics_number"], errors='raise')
except ValueError as e:
    print("Error:", e)

Let's verify that all values in the 'pics_number' column were correctly converted:

In [ ]:
df["pics_number"].value_counts(dropna=False)

,count
pics_number,
6.0,238
10.0,236
11.0,207
9.0,195
12.0,188
8.0,170
13.0,164
7.0,155
14.0,135


## Converting 'Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)' and  'Nebenkosten (€/month)' to numeric data types

Several columns in my dataset had formatting issues that prevented direct conversion to a numeric data type. Specifically:

- The period (".") was used as a thousands separator.
- The comma (",") was used as a decimal separator.
- Non-numeric symbols, such as the euro sign ("€"), were also present.

These inconsistencies needed to be addressed before converting the affected columns to a numeric format.

In [ ]:
df['Kaltmiete zzgl. Nebenkosten (€/month)'].value_counts(dropna=False)

,count
Kaltmiete zzgl. Nebenkosten (€/month),
NaN,2411
1.200 €,12
1.600 €,12
2.500 €,12
1.500 €,10
...,...
2.840 €,1
1.925 €,1
2.945 €,1


This function cleans a column by:

- Removing non-numeric characters (except for "," and ".").
- Fixing incorrect thousands separators by ensuring that a dot (".") is only considered a thousands separator if it is followed by exactly three digits.
- Converting comma decimal separators (",") to dots (".") to standardize numeric formatting.
- Converting to float

In [ ]:
def clean_2_numerical_values(df, column_name):
    """
    Cleans a rent-related column by:
    1. Removing non-numeric characters except ',' and '.'
    2. Removing incorrect thousands separators
    3. Converting comma decimal separators to dots
    4. Converting to float

    Parameters:
        df (pd.DataFrame): The DataFrame containing the column
        column_name (str): The name of the column to clean

    Returns:
        pd.DataFrame: The DataFrame with the cleaned column
    """
    # Remove non-numeric characters except ',' and '.'
    df.loc[:, column_name] = df[column_name].str.replace(r"[^\d,\.]", "", regex=True)

    # Remove incorrect thousands separators (e.g., '1.401.48' → '1401.48')
    df.loc[:, column_name] = df[column_name].str.replace(r"\.(?=\d{3})", "", regex=True)

    # Replace commas with dots (for decimal compatibility) and convert to float
    df.loc[:, column_name] = df[column_name].str.replace(",", ".").astype(float)

    return df

In [ ]:
df = clean_2_numerical_values(df, 'Kaltmiete zzgl. Nebenkosten (€/month)')
df = clean_2_numerical_values(df, 'Warmmiete (€/month)')
df = clean_2_numerical_values(df, 'Nebenkosten (€/month)')

Let's check whether the function successfully cleaned one of these columns:

In [ ]:
df['Kaltmiete zzgl. Nebenkosten (€/month)'].value_counts(dropna=False)

,count
Kaltmiete zzgl. Nebenkosten (€/month),
NaN,2411
1600.0,14
1200.0,12
2500.0,12
1500.0,10
...,...
848.0,1
1341.52,1
1087.52,1


## Converting 'Surface Area (m^2)' to a numeric data type

Since the 'Surface Area (m²)' column was not fully cleaned earlier, I decided to complete the cleaning process at this stage.

In [ ]:
df["Surface Area (m^2)"].value_counts(dropna=False)

,count
Surface Area (m^2),
9 m²,58
10 m²,57
60 m²,56
15 m²,50
50 m²,50
...,...
210 m²,1
165 m²,1
143 m²,1


1. I created a temporary series "cleaned_series" and remove any non-numeric characters apart from dots "." and replaced commas "," with dots "." to match decimal format used in Python.

2. I converted cleaned_series into numeric values replacing errors by np.nan.

3. I identified problematic rows finding where numeric_series contains NaN and printed the original problematic values from "Surface Area (m²)".

4. Opening the URLs of each of these rows, I manually checked and corrected the problematic values to have as few data loss as possible.

In [ ]:
# Create a temporary Series with the same cleaning steps as in your function
cleaned_series = df['Surface Area (m^2)'].str.replace(r"[^\d,\.]", "", regex=True)
cleaned_series = cleaned_series.str.replace(",", ".", regex=True)

# Convert the cleaned series to numeric, coercing errors to NaN.
numeric_series = pd.to_numeric(cleaned_series, errors='coerce')

# Identify rows where conversion to numeric failed.
problematic_rows = df.loc[numeric_series.isna(), 'Surface Area (m^2)']

print(f"Problematic values in '{'Surface Area (m^2)'}':")
df.loc[problematic_rows.index, ['Surface Area (m^2)', "Floor", "availability", "Real Estate 1", "Real Estate 2", "Real Estate 3", "Real Estate 4", "Real Estate 5 Subtitle", "Real Estate 5 Title", "URL"]]

Problematic values in 'Surface Area (m^2)':


,Surface Area (m^2),Floor,availability,Real Estate 1,Real Estate 2,Real Estate 3,Real Estate 4,Real Estate 5 Subtitle,Real Estate 5 Title,URL
227,19,NaN,Fahrtzeitberechnung,NaN,NaN,Bricc Gmbh,Herr dr.c di.Aler,Herr dr.c di.Aler,Bricc Gmbh,https://www.immowelt.de/expose/43a40897-fd29-4...
622,EG,NaN,Fahrtzeitberechnung,NaN,NaN,Ambius Immobilien GmbH,Stina Moepert,Stina Moepert,Ambius Immobilien GmbH,https://www.immowelt.de/expose/5eca010f-3f3c-4...
912,20,NaN,Heizungsart,NaN,NaN,NaN,Privater Anbieter,NaN,Privater Anbieter,https://www.immowelt.de/expose/849898c1-113f-4...
1037,16,NaN,ab sofort,NaN,NaN,NaN,Privater Anbieter,NaN,Privater Anbieter,https://www.immowelt.de/expose/4aa7e306-a978-4...
1040,19,NaN,ab sofort,NaN,NaN,NaN,Privater Anbieter,NaN,Privater Anbieter,https://www.immowelt.de/expose/00147078-c7aa-4...
1041,185,NaN,Baujahr,NaN,NaN,Financial Workout Concepts GmbH,Frau Maria Struss,Frau Maria Struss,Financial Workout Concepts GmbH,https://www.immowelt.de/expose/cf01de6e-605b-4...
1063,21,NaN,ab sofort,NaN,NaN,Acopio Facility GmbH & Co. KG,NaN,Gewerblicher Anbieter,Acopio Facility GmbH & Co. KG,https://www.immowelt.de/expose/46f2d6b4-fa2c-4...
1079,26,NaN,Zustand der Immobilie,NaN,NaN,VISIONAPARTMENTS BERLIN GmbH,Team Global VISIONAPARTMENTS,Team Global VISIONAPARTMENTS,VISIONAPARTMENTS BERLIN GmbH,https://www.immowelt.de/expose/30f6e626-b576-4...
1081,21,NaN,Zustand der Immobilie,NaN,NaN,VISIONAPARTMENTS BERLIN GmbH,Team Global VISIONAPARTMENTS,Team Global VISIONAPARTMENTS,VISIONAPARTMENTS BERLIN GmbH,https://www.immowelt.de/expose/370a410c-c446-4...
1083,31,NaN,Baujahr,NaN,NaN,VISIONAPARTMENTS BERLIN GmbH,Team Global VISIONAPARTMENTS,Team Global VISIONAPARTMENTS,VISIONAPARTMENTS BERLIN GmbH,https://www.immowelt.de/expose/20cf9117-24e3-4...


Examples of problematic values: "EG", "ab sorfort".
For the "EG" value:
1. Creates a mask (mask_EG) to find rows where "Surface Area (m²)" contains "EG".
2. Assigns "EG" to the "Floor" column.
3. Removes "EG" from "Surface Area (m²)" by setting it to NaN.

Similarly steps were run for the "ab sofort" value, mostly likely belonging to the "availability" column.

If other problematic values were found, such as "Privater Anbieter" and "Zustand der Immobilie" but that they already had the same value registered in the appropriate column "Surface Area (m²)" was simply set to np.nan for the corresponding rows.

Lasty, there were some problematic strings contaning a mix of numbers and letters such as "120 m²". These were extracted and converted to float.

In [ ]:
# (a) For rows with 'EG' – assign them to the 'Floor' column.
mask_EG = df['Surface Area (m^2)'].astype(str).str.strip() == 'EG'
df.loc[mask_EG, 'Floor'] = 'EG'
df.loc[mask_EG, 'Surface Area (m^2)'] = np.nan  # set area to NaN

# (b) For rows with 'ab sofort' – assign them to the 'availability' column.
mask_ab_sofort = df['Surface Area (m^2)'].astype(str).str.strip() == 'ab sofort'
df.loc[mask_ab_sofort, 'availability'] = 'ab sofort'
df.loc[mask_ab_sofort, 'Surface Area (m^2)'] = np.nan  # set area to NaN

# (c) For the other cases (e.g. "Privater Anbieter", "Zustand der Immobilie"),
# simply leave 'Surface Area (m^2)' as NaN.
mask_other = ~mask_EG & ~mask_ab_sofort & numeric_series.isna()
df.loc[mask_other, 'Surface Area (m^2)'] = np.nan

df["Surface Area (m^2)"] = df["Surface Area (m^2)"].str.extract(r"(\d+)").astype(float)

In [ ]:
df["Surface Area (m^2)"].value_counts(dropna=False)

,count
Surface Area (m^2),
10.0,130
12.0,101
20.0,89
15.0,77
9.0,73
...,...
139.0,1
290.0,1
143.0,1


## Converting 'Additional Price Info' to a numeric data type

In [ ]:
df['Additional Price Info'].value_counts(dropna=False)

,count
Additional Price Info,
NaN,2577
"Nettokaltmiete: 1.350,00 EUR",11
"Nettokaltmiete: 1.600,00 EUR",7
"Nettokaltmiete: 2.500,00 EUR",6
"Nettokaltmiete: 1.000,00 EUR",5
...,...
"Nettokaltmiete: 1.262,88 EUR",1
"Nettokaltmiete: 2.595,00 EUR",1
"Nettokaltmiete: 995,00 EUR",1


I applied the same cleaning steps to the 'Additional Price Info' column, including:

- Extracting numerical values,
- Removing the thousands separator ("."),
- Replacing the decimal separator ("," with "."),
- Converting the values to float for proper numerical analysis.

In [ ]:
# Extract numbers, remove thousands separator (.), replace decimal separator (,), and convert to float
df["Additional Price Info"] = (
    df["Additional Price Info"]
    .str.extract(r'([\d.,]+)')[0]  # Extract numeric part
    .str.replace('.', '', regex=False)  # Remove thousands separator
    .str.replace(',', '.', regex=False)  # Replace decimal separator
    .astype(float)  # Convert to float
)

In [ ]:
df['Additional Price Info'].value_counts(dropna=False)

,count
Additional Price Info,
NaN,2577
1350.00,11
1600.00,7
2500.00,6
1000.00,5
...,...
1262.88,1
2595.00,1
995.00,1


## Merging "Additional Price Info" and "Kaltmiete zzgl. Nebenkosten (€/month)" columns

The values from the 'Additional Price Info' column were transferred to 'Kaltmiete zzgl. Nebenkosten (€/month)' using the following steps:

1. Filtering Non-NaN Values:
- Only non-NaN values from the 'Additional Price Info' column were considered for transfer.
2. Ensuring Consistency Before Replacement:
- The filtered values were compared to the existing values in 'Kaltmiete zzgl. Nebenkosten (€/month)'.
- The replacement was allowed only if:
  - The target value in 'Kaltmiete zzgl. Nebenkosten (€/month)' was missing (NaN).
  - The values in both columns were the same.
3. Handling Conflicting Values:
- If a listing contained different values in both columns (e.g., row 200 had 'Kaltmiete zzgl. Nebenkosten (€/month)' = 2100 and 'Additional Price Info' = 3200), I manually reviewed the corresponding listing by opening its URL.
- Based on the actual listing details, I decided which value to keep.

In [ ]:
# Filter and print rows where 'Additional Price Info' is not NaN
df.loc[df["Additional Price Info"].notna(), ["Kaltmiete zzgl. Nebenkosten (€/month)", "Warmmiete (€/month)", "Additional Price Info"]]

,Kaltmiete zzgl. Nebenkosten (€/month),Warmmiete (€/month),Additional Price Info
12,NaN,NaN,1480.00
16,NaN,1507.44,1232.44
28,NaN,9658.03,8073.03
32,NaN,1859.75,1609.75
34,NaN,1785.0,1584.00
...,...,...,...
3762,NaN,2020.0,1819.00
3773,NaN,1946.0,1645.00
3781,NaN,1531.0,1295.00
3786,NaN,858.9,579.90


In [ ]:
non_nan_rows = df[df["Additional Price Info"].notna()]
df.loc[df["Additional Price Info"].notna(), "Kaltmiete zzgl. Nebenkosten (€/month)"] = df["Additional Price Info"]
df.loc[df["Additional Price Info"].notna(), ["Kaltmiete zzgl. Nebenkosten (€/month)", "Warmmiete (€/month)", "Additional Price Info"]]

,Kaltmiete zzgl. Nebenkosten (€/month),Warmmiete (€/month),Additional Price Info
12,1480.0,NaN,1480.00
16,1232.44,1507.44,1232.44
28,8073.03,9658.03,8073.03
32,1609.75,1859.75,1609.75
34,1584.0,1785.0,1584.00
...,...,...,...
3762,1819.0,2020.0,1819.00
3773,1645.0,1946.0,1645.00
3781,1295.0,1531.0,1295.00
3786,579.9,858.9,579.90


Now that the values from the 'Additional Price Info' column have been successfully transferred to 'Kaltmiete zzgl. Nebenkosten (€/month)', I deleted the 'Additional Price Info' column as it was no longer needed.

In [ ]:
df.drop(columns=["Additional Price Info"], inplace=True)

## Converting "Floor" to a numeric data type

In [ ]:
df["Floor"].value_counts(dropna=False)

,count
Floor,
NaN,2086
1.,169
2.,167
3.,137
4.,134
EG,106
5.,100
6.,45
7.,18


I returned to the 'Floor' column to continue cleaning it.

First, I updated the 'Multiple floors' column to retain information about whether a property spans multiple floors in a building.

In [ ]:
# Update the "Multiple floors" column:
df['Multiple floors'] = df['Floor'].apply(lambda x: 1 if pd.notnull(x) and isinstance(x, str) and '/' in x else np.nan)

Now that the information about multi-floor properties has been saved, I completed the cleaning process for this column. The cleaning steps include:

1. If the value is NaN, it remains unchanged.
Processing Multi-Floor Properties:
2. If a property spans multiple floors, the average of all the floors it spans is calculated and stored as its floor value.
Standardizing Ground Floor Values:
3. The German abbreviation "EG" (Erdgeschoss, meaning "ground floor") is converted to 0.
Cleaning Floor Number Formatting:
4. If a floor number ends with a dot ("."), the dot is removed, and the value is converted to an integer.
Final Conversion:
5. The 'Floor' column is converted to a numeric data type for consistency in analysis.

In [ ]:
# Modify the "Floor" column using an inline lambda:
df['Floor'] = df['Floor'].apply(
    lambda x: np.nan if pd.isnull(x) else
              (sum(map(float, x.split('/'))) / 2 if '/' in x else
               (0 if x.strip().upper() == 'EG' else
                (int(x.rstrip('.')) if x.endswith('.') else float(x))))
)

# Convert "Floor" column to numerical datatype
df['Floor'] = pd.to_numeric(df['Floor'])

Let's verify the cleaned 'Floor' column to ensure the transformations were applied correctly:

In [ ]:
df["Floor"].value_counts(dropna=False)

,count
Floor,
NaN,2086
1.0,172
2.0,171
3.0,145
4.0,137
0.0,106
5.0,104
6.0,47
7.0,18


## Converting "Deposit 1" to a numeric data type

In [ ]:
df["Deposit 1"].value_counts(dropna=False)

,count
Deposit 1,
NaN,630
keine Angabe,340
2 Monatsmieten,113
"1.000,00",104
"2.000,00",82
...,...
"3.510,00",1
"2.028,00",1
"1.616,00",1


This function cleans and converts deposit values into a numeric format.

- It ensures that only strings are processed
- If the value is already a number, the function returns it unchanged
- it checks if the string contains at least one digit, and skip the conversion if no digits are found.
- it attempts to clean and convert the value removing "€" and "." when used as thousand separators and spaces. It replaces commas "," with dots "." to ensure correct decimal formatting.
- If conversion fails, the function returns the original value instead of raising an error. This helps preserve non-numeric deposit values like "2 Monatsmieten" (meaning "2 months' rent") which should not be replaced by nan nor converted to "2", but can still be converted later to a number.

In [ ]:
# Define a function to clean and convert the deposit values
def clean_deposit_value(value):
    if isinstance(value, str):
        # If it contains digits, attempt to clean and convert
        if any(char.isdigit() for char in value):
            try:
                # Remove currency symbols, commas, and spaces, then replace commas with periods for decimal conversion
                cleaned_value = value.replace(".", "").replace(",", ".").replace("€", "").strip()
                # Convert to float if possible
                return float(cleaned_value)
            except ValueError:
                # In case of any error, return the original string (like '2 Monatsmieten')
                return value
    # Return NaN or the original non-string value
    return value

In [ ]:
# Apply the cleaning function to the 'Deposit 1' column
df["Deposit 1"] = df["Deposit 1"].apply(clean_deposit_value)

In [ ]:
df["Deposit 1"].value_counts(dropna=False)

,count
Deposit 1,
NaN,630
keine Angabe,340
2 Monatsmieten,113
1000.0,104
2000.0,85
...,...
3162.0,1
2196.0,1
10440.0,1


I applied the function and manually reviewed and corrected some data by checking the corresponding listing URLs.

To do this, I used Google Colab's interactive table and applied column filtering functions to efficiently identify and correct inconsistencies.

In [ ]:
df[["Kaltmiete zzgl. Nebenkosten (€/month)", "Warmmiete (€/month)", "Deposit 1", "Deposit 2", "URL"]]

,Kaltmiete zzgl. Nebenkosten (€/month),Warmmiete (€/month),Deposit 1,Deposit 2,URL
0,NaN,1355.0,NaN,3330.0,https://www.immowelt.de/expose/83a070dd-c407-4...
1,900.0,1100.0,NaN,1400.0,https://www.immowelt.de/expose/f8942fb4-1007-4...
2,2260.0,2620.0,2 Monatsmieten,NaN,https://www.immowelt.de/expose/fd9e5847-f017-4...
3,1550.0,1850.0,NaN,3600.0,https://www.immowelt.de/expose/599eb8d0-b846-4...
4,NaN,2380.0,7000.0,NaN,https://www.immowelt.de/expose/53457c89-871f-4...
...,...,...,...,...,...
3797,NaN,1275.0,1912.5,NaN,https://www.immowelt.de/expose/e0e4544e-226f-4...
3799,NaN,658.0,NaN,987.0,https://www.immowelt.de/expose/3bc83d6c-d821-4...
3801,NaN,746.0,1119.0,NaN,https://www.immowelt.de/expose/0f2c7cbd-9153-4...
3805,NaN,700.0,NaN,950.0,https://www.immowelt.de/expose/56e439f3-4463-4...


In [ ]:
df = df.drop([88, 2086]) #No Kalt- nor Warmmiete data. Useless listings because my goal is to use the rent-related columns as independat variables for my analysis later.

# Replace "keine Angabe" with np.nan in the "Deposit 1" column
df["Deposit 1"] = df["Deposit 1"].replace("keine Angabe", np.nan)

df.loc[50, "Kaltmiete zzgl. Nebenkosten (€/month)"] = 1300
df.loc[2435, "Kaltmiete zzgl. Nebenkosten (€/month)"] = 318.58
df.loc[3241, "Kaltmiete zzgl. Nebenkosten (€/month)"] = 2000
df.loc[3418, "Kaltmiete zzgl. Nebenkosten (€/month)"] = 2750
df.loc[955, "Deposit 1"] = np.nan
df.loc[2121, "Deposit 1"] = np.nan
df.loc[3022, "Deposit 1"] = np.nan
df.loc[64, "Deposit 1"] = 5700
df.loc[83, "Deposit 1"] = 4200
df.loc[866, "Deposit 1"] = 1500
df.loc[1099, "Deposit 1"] = 3000
df.loc[1106, "Deposit 1"] = 3000
df.loc[1880, "Deposit 1"] = 4400
df.loc[2247, "Deposit 1"] = 1750
df.loc[2269, "Deposit 1"] = 1100
df.loc[2288, "Deposit 1"] = 1400
df.loc[3032, "Deposit 1"] = 2200
df.loc[3490, "Deposit 1"] = 1200
df.loc[3503, "Deposit 1"] = 1500
df.loc[3512, "Deposit 1"] = 1250
df.loc[3522, "Deposit 1"] = 1350
df.loc[3418, "Deposit 1"] = 5500
df.loc[3531, "Deposit 1"] = 1600
df.loc[3535, "Deposit 1"] = 1700
df.loc[3551, "Deposit 1"] = 1800
df.loc[3563, "Deposit 1"] = 1200
df.loc[3565, "Deposit 1"] = 1600
df.loc[3697, "Deposit 1"] = 2850
df.loc[216, "Kaltmiete zzgl. Nebenkosten (€/month)"] = 1600
df.loc[3024, "Kaltmiete zzgl. Nebenkosten (€/month)"] = 630.58
df.loc[3328, "Kaltmiete zzgl. Nebenkosten (€/month)"] = 420.70
df.loc[555, "Kaltmiete zzgl. Nebenkosten (€/month)"] = 95.0*18.25
df.loc[2730, "Kaltmiete zzgl. Nebenkosten (€/month)"] = 87.0*15.90
df.loc[2731, "Kaltmiete zzgl. Nebenkosten (€/month)"] = 48.0*6.70
df.loc[3333, "Kaltmiete zzgl. Nebenkosten (€/month)"] = 66.0*23.02

# Define a mapping of row indexes to (multiplier, source column)
updates = {
    45: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    132: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    213: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    256: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    614: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    643: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    663: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    710: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    717: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    871: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    902: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    932: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    946: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1965: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3024: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3328: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3241: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2435: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    50: (1, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3527: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3544: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3547: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3558: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3606: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3646: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3704: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3791: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    292: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    727: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2310: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3376: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3443: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3471: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3475: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3480: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3481: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3482: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3483: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3487: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3505: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3518: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3526: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2708: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2842: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2918: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2947: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2952: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3053: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3062: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3065: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3114: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3200: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3236: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3255: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3261: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3271: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3318: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3354: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3373: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    218: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    275: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    323: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    661: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    704: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    709: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    726: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    748: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    777: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    793: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    828: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    837: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    838: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2048: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2505: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2512: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2523: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2544: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2574: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2695: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    13: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    32: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    33: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    34: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    37: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    46: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    61: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    72: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    269: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    311: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    316: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    423: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    465: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    473: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    546: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    565: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    568: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    633: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    676: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    685: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    699: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    718: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    770: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    826: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    861: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    896: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    898: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    930: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    990: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1002: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1024: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1027: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1044: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1056: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1062: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1064: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1092: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1116: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1126: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1287: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1476: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1486: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1566: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1906: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1914: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1942: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1967: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1971: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1985: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2006: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2008: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2014: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2019: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2021: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2030: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2033: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2073: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2079: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2084: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2141: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2151: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2172: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2178: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2230: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2266: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2344: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2357: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2363: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2366: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2380: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2407: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2427: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2447: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2475: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2480: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2488: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2494: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2500: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2502: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2641: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2699: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2709: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    216: (2, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    1997: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2067: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2331: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2374: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    2963: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3455: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3637: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3651: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    3705: (3, "Kaltmiete zzgl. Nebenkosten (€/month)"),
    599: (2, "Warmmiete (€/month)"),
    1131: (2, "Warmmiete (€/month)"),
    2177: (3, "Warmmiete (€/month)"),
    2240: (2, "Warmmiete (€/month)"),
    3366: (2, "Warmmiete (€/month)"),
    134: (2, "Warmmiete (€/month)"),
    384: (2, "Warmmiete (€/month)"),
    420: (2, "Warmmiete (€/month)"),
    477: (2, "Warmmiete (€/month)"),
    749: (2, "Warmmiete (€/month)"),
    846: (2, "Warmmiete (€/month)"),
    850: (2, "Warmmiete (€/month)"),
    852: (2, "Warmmiete (€/month)"),
    859: (2, "Warmmiete (€/month)"),
    863: (2, "Warmmiete (€/month)"),
    865: (2, "Warmmiete (€/month)"),
    869: (2, "Warmmiete (€/month)"),
    882: (2, "Warmmiete (€/month)"),
    884: (2, "Warmmiete (€/month)"),
    887: (2, "Warmmiete (€/month)"),
    888: (2, "Warmmiete (€/month)"),
    889: (2, "Warmmiete (€/month)"),
    893: (2, "Warmmiete (€/month)"),
    900: (2, "Warmmiete (€/month)"),
    906: (2, "Warmmiete (€/month)"),
    915: (2, "Warmmiete (€/month)"),
    917: (2, "Warmmiete (€/month)"),
    921: (2, "Warmmiete (€/month)"),
    926: (2, "Warmmiete (€/month)"),
    927: (2, "Warmmiete (€/month)"),
    935: (2, "Warmmiete (€/month)"),
    939: (2, "Warmmiete (€/month)"),
    940: (2, "Warmmiete (€/month)"),
    945: (2, "Warmmiete (€/month)"),
    957: (2, "Warmmiete (€/month)"),
    975: (2, "Warmmiete (€/month)"),
    997: (2, "Warmmiete (€/month)"),
    1040: (3, "Warmmiete (€/month)"),
    1043: (2, "Warmmiete (€/month)"),
    1072: (2, "Warmmiete (€/month)"),
    1073: (2, "Warmmiete (€/month)"),
    1077: (2, "Warmmiete (€/month)"),
    1078: (2, "Warmmiete (€/month)"),
    1087: (2, "Warmmiete (€/month)"),
    2: (2, "Warmmiete (€/month)"),
    19: (2, "Warmmiete (€/month)"),
    94: (2, "Warmmiete (€/month)"),
    378: (2, "Warmmiete (€/month)"),
    534: (2, "Warmmiete (€/month)"),
    641: (2, "Warmmiete (€/month)"),
    2225: (2, "Warmmiete (€/month)"),
    1088: (2, "Warmmiete (€/month)"),
    1089: (2, "Warmmiete (€/month)"),
    1091: (2, "Warmmiete (€/month)"),
    1120: (2, "Warmmiete (€/month)"),
    1121: (2, "Warmmiete (€/month)"),
    1122: (2, "Warmmiete (€/month)"),
    1128: (2, "Warmmiete (€/month)"),
    1135: (2, "Warmmiete (€/month)"),
    1426: (2, "Warmmiete (€/month)"),
    1464: (2, "Warmmiete (€/month)"),
    1859: (2, "Warmmiete (€/month)"),
    1918: (2, "Warmmiete (€/month)"),
    1974: (2, "Warmmiete (€/month)"),
    1981: (2, "Warmmiete (€/month)"),
    1988: (2, "Warmmiete (€/month)"),
    1991: (2, "Warmmiete (€/month)"),
    1992: (2, "Warmmiete (€/month)"),
    2002: (2, "Warmmiete (€/month)"),
    2011: (2, "Warmmiete (€/month)"),
    2012: (2, "Warmmiete (€/month)"),
    2013: (2, "Warmmiete (€/month)"),
    2020: (2, "Warmmiete (€/month)"),
    2022: (2, "Warmmiete (€/month)"),
    2024: (2, "Warmmiete (€/month)"),
    2042: (2, "Warmmiete (€/month)"),
    2045: (2, "Warmmiete (€/month)"),
    2047: (2, "Warmmiete (€/month)"),
    2049: (2, "Warmmiete (€/month)"),
    2052: (2, "Warmmiete (€/month)"),
    2053: (2, "Warmmiete (€/month)"),
    2063: (2, "Warmmiete (€/month)"),
    2065: (2, "Warmmiete (€/month)"),
    2066: (2, "Warmmiete (€/month)"),
    2081: (2, "Warmmiete (€/month)"),
    2089: (2, "Warmmiete (€/month)"),
    2104: (2, "Warmmiete (€/month)"),
    2139: (2, "Warmmiete (€/month)"),
    2154: (2, "Warmmiete (€/month)"),
    2181: (2, "Warmmiete (€/month)"),
    2182: (2, "Warmmiete (€/month)"),
    2183: (2, "Warmmiete (€/month)"),
    2196: (2, "Warmmiete (€/month)"),
    2742: (2, "Warmmiete (€/month)"),
    2832: (2, "Warmmiete (€/month)"),
    3041: (2, "Warmmiete (€/month)"),
    3044: (2, "Warmmiete (€/month)"),
    3078: (2, "Warmmiete (€/month)"),
    3100: (2, "Warmmiete (€/month)"),
    3283: (2, "Warmmiete (€/month)"),
    3451: (2, "Warmmiete (€/month)"),
    3458: (2, "Warmmiete (€/month)"),
    3462: (2, "Warmmiete (€/month)"),
    3466: (2, "Warmmiete (€/month)"),
    3467: (2, "Warmmiete (€/month)"),
    3474: (2, "Warmmiete (€/month)"),
    3484: (2, "Warmmiete (€/month)"),
    3485: (2, "Warmmiete (€/month)"),
    3486: (2, "Warmmiete (€/month)"),
    3492: (2, "Warmmiete (€/month)"),
    3493: (2, "Warmmiete (€/month)"),
    3497: (2, "Warmmiete (€/month)"),
    3498: (2, "Warmmiete (€/month)"),
    3499: (2, "Warmmiete (€/month)"),
    3500: (2, "Warmmiete (€/month)"),
    3506: (2, "Warmmiete (€/month)"),
    3508: (2, "Warmmiete (€/month)"),
    3509: (2, "Warmmiete (€/month)"),
    3521: (2, "Warmmiete (€/month)"),
    3523: (2, "Warmmiete (€/month)"),
    3524: (2, "Warmmiete (€/month)"),
    3525: (2, "Warmmiete (€/month)"),
    3528: (2, "Warmmiete (€/month)"),
    3541: (2, "Warmmiete (€/month)"),
    3545: (2, "Warmmiete (€/month)"),
    3546: (2, "Warmmiete (€/month)"),
    3555: (2, "Warmmiete (€/month)"),
    3556: (2, "Warmmiete (€/month)"),
    3561: (2, "Warmmiete (€/month)"),
    3566: (2, "Warmmiete (€/month)"),
    3567: (2, "Warmmiete (€/month)"),
    3620: (2, "Warmmiete (€/month)"),
    3671: (2, "Warmmiete (€/month)"),
    3769: (2, "Warmmiete (€/month)")
}

# Update "Deposit 1" for the specified row indexes
for idx, (multiplier, source_col) in updates.items():
    if idx in df.index:  # check that the row exists
        # Calculate the new deposit value
        new_value = multiplier * df.loc[idx, source_col]
        # Assign the computed value to the "Deposit 1" column
        df.loc[idx, "Deposit 1"] = new_value

In [ ]:
df["Deposit 1"].value_counts(dropna=False)

,count
Deposit 1,
NaN,980
1000.0,104
2000.0,85
1200.0,45
1425.0,42
...,...
4174.5,1
9300.0,1
4150.0,1


# Cleaning "Kaltmiete zzgl. Nebenkosten (€/month)", "Nebenkosten (€/month)" and "Heizkosten_2 (€/month)"

In [ ]:
df[["Kaltmiete zzgl. Nebenkosten (€/month)", "Nebenkosten (€/month)", "Heizkosten_2 (€/month)", "URL"]]

,Kaltmiete zzgl. Nebenkosten (€/month),Nebenkosten (€/month),Heizkosten_2 (€/month),URL
0,NaN,NaN,NaN,https://www.immowelt.de/expose/83a070dd-c407-4...
1,900.0,300.0,NaN,https://www.immowelt.de/expose/f8942fb4-1007-4...
2,2260.0,360.0,NaN,https://www.immowelt.de/expose/fd9e5847-f017-4...
3,1550.0,300.0,NaN,https://www.immowelt.de/expose/599eb8d0-b846-4...
4,NaN,NaN,NaN,https://www.immowelt.de/expose/53457c89-871f-4...
...,...,...,...,...
3797,NaN,NaN,NaN,https://www.immowelt.de/expose/e0e4544e-226f-4...
3799,NaN,NaN,NaN,https://www.immowelt.de/expose/3bc83d6c-d821-4...
3801,NaN,NaN,NaN,https://www.immowelt.de/expose/0f2c7cbd-9153-4...
3805,NaN,NaN,NaN,https://www.immowelt.de/expose/56e439f3-4463-4...


I defined a dictionary to correct specific erroneous or problematic values that I identified during the previous data exploration. These corrections were applied to the following columns:

- 'Kaltmiete zzgl. Nebenkosten (€/month)'
- 'Nebenkosten (€/month)'
- 'Heizkosten_2 (€/month)'

In [ ]:
# Define updates as a dictionary: for each index, only include the columns that should be updated.
updates = {
    20: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1550,
         "Nebenkosten (€/month)": 340},
    30: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1257.55,
         "Nebenkosten (€/month)": 130,
         "Heizkosten_2 (€/month)": 97.50},
    43: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1080,
         "Nebenkosten (€/month)": 98},
    53: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2850,
         "Nebenkosten (€/month)": 198,
         "Heizkosten_2 (€/month)": 187},
    99: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1032,
         "Nebenkosten (€/month)": 108,
         "Heizkosten_2 (€/month)": 81},
    193: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1441,
          "Nebenkosten (€/month)": 110,
          "Heizkosten_2 (€/month)": 82.50},
    237: {"Kaltmiete zzgl. Nebenkosten (€/month)": 665,
          "Nebenkosten (€/month)": 98.92,
          "Heizkosten_2 (€/month)": 84.39},
    238: {"Kaltmiete zzgl. Nebenkosten (€/month)": 871,
          "Nebenkosten (€/month)": 149.88,
          "Heizkosten_2 (€/month)": 101.25},
    338: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1298,
          "Nebenkosten (€/month)": 197.89,
          "Heizkosten_2 (€/month)": 123.68},
    341: {"Kaltmiete zzgl. Nebenkosten (€/month)": 890,
          "Nebenkosten (€/month)": 150,
          "Heizkosten_2 (€/month)": 50},
    381: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2065.63,
          "Nebenkosten (€/month)": 136,
          "Heizkosten_2 (€/month)": 102},
    392: {"Kaltmiete zzgl. Nebenkosten (€/month)": 3979,
          "Nebenkosten (€/month)": 780,
          "Heizkosten_2 (€/month)": 204},
    415: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1539.66,
          "Nebenkosten (€/month)": 104,
          "Heizkosten_2 (€/month)": 78},
    417: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2137.96,
          "Nebenkosten (€/month)": 150,
          "Heizkosten_2 (€/month)": 112.50},
    438: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2375.63,
          "Nebenkosten (€/month)": 130,
          "Heizkosten_2 (€/month)": 97.50},
    447: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1170,
          "Nebenkosten (€/month)": 180.87,
          "Heizkosten_2 (€/month)": 105.85},
    450: {"Kaltmiete zzgl. Nebenkosten (€/month)": 752.52,
          "Nebenkosten (€/month)": 152.69,
          "Heizkosten_2 (€/month)": 67.10},
    451: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1600.63,
          "Nebenkosten (€/month)": 106,
          "Heizkosten_2 (€/month)": 79.50},
    453: {"Kaltmiete zzgl. Nebenkosten (€/month)": 939.31,
          "Nebenkosten (€/month)": 161.95,
          "Heizkosten_2 (€/month)": 95.92},
    455: {"Kaltmiete zzgl. Nebenkosten (€/month)": 427.57,
          "Nebenkosten (€/month)": 82.22,
          "Heizkosten_2 (€/month)": 70.86},
    462: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2480,
          "Nebenkosten (€/month)": 618},
    90:  {"Kaltmiete zzgl. Nebenkosten (€/month)": 471,
          "Nebenkosten (€/month)": 2552.33,
          "Heizkosten_2 (€/month)": 162,
          "Miete pro Stellplatz (€/month)": 121.50},
    472: {"Kaltmiete zzgl. Nebenkosten (€/month)": 3399,
          "Nebenkosten (€/month)": 176,
          "Heizkosten_2 (€/month)": 132},
    476: {"Kaltmiete zzgl. Nebenkosten (€/month)": 753.47,
          "Nebenkosten (€/month)": 152.83,
          "Heizkosten_2 (€/month)": 67.19},
    487: {"Kaltmiete zzgl. Nebenkosten (€/month)": 427.30,
          "Nebenkosten (€/month)": 106.82,
          "Heizkosten_2 (€/month)": 94.44},
    488: {"Kaltmiete zzgl. Nebenkosten (€/month)": 426.79,
          "Nebenkosten (€/month)": 82.08,
          "Heizkosten_2 (€/month)": 70.72},
    491: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1642.37,
          "Nebenkosten (€/month)": 244.40,
          "Heizkosten_2 (€/month)": 218.73},
    492: {"Kaltmiete zzgl. Nebenkosten (€/month)": 653.58,
          "Nebenkosten (€/month)": 99.22,
          "Heizkosten_2 (€/month)": 62.01},
    497: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1621.29,
          "Nebenkosten (€/month)": 114,
          "Heizkosten_2 (€/month)": 85.50},
    498: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1600.63,
          "Nebenkosten (€/month)": 90,
          "Heizkosten_2 (€/month)": 67.50},
    504: {"Kaltmiete zzgl. Nebenkosten (€/month)": 957.05,
          "Nebenkosten (€/month)": 176.43,
          "Heizkosten_2 (€/month)": 93.63},
    510: {"Kaltmiete zzgl. Nebenkosten (€/month)": 862.32,
          "Nebenkosten (€/month)": 148.68,
          "Heizkosten_2 (€/month)": 87.82},
    511: {"Kaltmiete zzgl. Nebenkosten (€/month)": 674.30,
          "Nebenkosten (€/month)": 147.70,
          "Heizkosten_2 (€/month)": 63.98},
    538: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1803.14,
          "Nebenkosten (€/month)": 148,
          "Heizkosten_2 (€/month)": 111},
    545: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1253.75,
          "Nebenkosten (€/month)": 201.64,
          "Heizkosten_2 (€/month)": 126.03},
    551: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1858.96,
          "Nebenkosten (€/month)": 148,
          "Heizkosten_2 (€/month)": 111},
    552: {"Kaltmiete zzgl. Nebenkosten (€/month)": 891.71,
          "Nebenkosten (€/month)": 153.85,
          "Heizkosten_2 (€/month)": 135.01},
    563: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1172.47,
          "Nebenkosten (€/month)": 210.87,
          "Heizkosten_2 (€/month)": 125.28},
    571: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2015,
          "Nebenkosten (€/month)": 160,
          "Heizkosten_2 (€/month)": 120},
    578: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1548.96,
          "Nebenkosten (€/month)": 122,
          "Heizkosten_2 (€/month)": 91.50},
    584: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2013.96,
          "Nebenkosten (€/month)": 150,
          "Heizkosten_2 (€/month)": 112.50},
    588: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1643.01,
          "Nebenkosten (€/month)": 154,
          "Heizkosten_2 (€/month)": 115.50},
    592: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2375.63,
          "Nebenkosten (€/month)": 162,
          "Heizkosten_2 (€/month)": 121.50},
    603: {"Kaltmiete zzgl. Nebenkosten (€/month)": 4028.96,
          "Nebenkosten (€/month)": 318,
          "Heizkosten_2 (€/month)": 238.50},
    605: {"Kaltmiete zzgl. Nebenkosten (€/month)": 617.13,
          "Nebenkosten (€/month)": 106.47,
          "Heizkosten_2 (€/month)": 89.05},
    625: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1701.78,
          "Nebenkosten (€/month)": 306.08,
          "Heizkosten_2 (€/month)": 182.40},
    667: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1326.55,
          "Nebenkosten (€/month)": 202.32,
          "Heizkosten_2 (€/month)": 126.45},
    721: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1930,
          "Nebenkosten (€/month)": 275,
          "Heizkosten_2 (€/month)": 145},
    744: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1454.44,
          "Nebenkosten (€/month)": 198.29,
          "Heizkosten_2 (€/month)": 123.93},
    826: {"Kaltmiete zzgl. Nebenkosten (€/month)": 3600,
          "Nebenkosten (€/month)": 432},
    980: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2400,
          "Nebenkosten (€/month)": 490,
          "Heizkosten_2 (€/month)": np.nan,
          "Miete pro Stellplatz (€/month)": 150},
    1146: {"Kaltmiete zzgl. Nebenkosten (€/month)": 657,
           "Nebenkosten (€/month)": 142.82,
           "Heizkosten_2 (€/month)": 126.84},
    1164: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1516.39,
           "Nebenkosten (€/month)": 125,
           "Heizkosten_2 (€/month)": 125},
    1169: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1286.40,
           "Nebenkosten (€/month)": 206.93,
           "Heizkosten_2 (€/month)": 129.33},
    1177: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1859.05,
           "Nebenkosten (€/month)": 283.46,
           "Heizkosten_2 (€/month)": 177.17},
    1189: {"Kaltmiete zzgl. Nebenkosten (€/month)": 674.61,
           "Nebenkosten (€/month)": 147.75,
           "Heizkosten_2 (€/month)": 64.01},
    1226: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1296.80,
           "Nebenkosten (€/month)": 197.79,
           "Heizkosten_2 (€/month)": 123.62},
    1250: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1248.32,
           "Nebenkosten (€/month)": 185.28,
           "Heizkosten_2 (€/month)": 115.80},
    1387: {"Kaltmiete zzgl. Nebenkosten (€/month)": 460},
    1392: {"Kaltmiete zzgl. Nebenkosten (€/month)": 977.50},
    1395: {"Kaltmiete zzgl. Nebenkosten (€/month)": 981.92,
           "Nebenkosten (€/month)": 124.46,
           "Heizkosten_2 (€/month)": 77.79},
    1443: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1150,
           "Nebenkosten (€/month)": 100,
           "Heizkosten_2 (€/month)": 50},
    1469: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1775,
           "Nebenkosten (€/month)": 325},
    1876: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1200,
           "Nebenkosten (€/month)": 190,
           "Heizkosten_2 (€/month)": 200},
    1893: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1290,
           "Nebenkosten (€/month)": 302,
           "Heizkosten_2 (€/month)": np.nan},
    2010: {"Kaltmiete zzgl. Nebenkosten (€/month)": 760.39,
           "Nebenkosten (€/month)": 96.39,
           "Heizkosten_2 (€/month)": 60.24},
    2014: {"Kaltmiete zzgl. Nebenkosten (€/month)": 3450,
           "Nebenkosten (€/month)": 600,
           "Heizkosten_2 (€/month)": np.nan,
           "Miete pro Stellplatz (€/month)": 150},
    2015: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1248.98,
           "Nebenkosten (€/month)": 185.40,
           "Heizkosten_2 (€/month)": 115.88},
    2016: {"Kaltmiete zzgl. Nebenkosten (€/month)": 700,
           "Nebenkosten (€/month)": np.nan,
           "Heizkosten_2 (€/month)": np.nan,
           "Miete pro Stellplatz (€/month)": 100},
    2073: {"Kaltmiete zzgl. Nebenkosten (€/month)": 6250,
           "Nebenkosten (€/month)": 914,
           "Heizkosten_2 (€/month)": np.nan,
           "Miete pro Stellplatz (€/month)": 180},
    2106: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1590,
           "Nebenkosten (€/month)": 120,
           "Heizkosten_2 (€/month)": np.nan},
    2116: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1950,
           "Nebenkosten (€/month)": 160,
           "Heizkosten_2 (€/month)": 190},
    2128: {"Kaltmiete zzgl. Nebenkosten (€/month)": 708.18,
           "Nebenkosten (€/month)": 122.10,
           "Heizkosten_2 (€/month)": 72.01},
    2151: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1000,
           "Nebenkosten (€/month)": np.nan,
           "Heizkosten_2 (€/month)": 200},
    2152: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1260},
    2215: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2800,
           "Nebenkosten (€/month)": 380,
           "Heizkosten_2 (€/month)": 180,
           "Miete pro Stellplatz (€/month)": 100},
    2222: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2300,
           "Nebenkosten (€/month)": 360},
    2251: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1124.82,
           "Nebenkosten (€/month)": 241.12},
    2268: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1300},
    2394: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1550,
           "Nebenkosten (€/month)": 340},
    2445: {"Kaltmiete zzgl. Nebenkosten (€/month)": 3454.48,
           "Nebenkosten (€/month)": 200,
           "Heizkosten_2 (€/month)": 150},
    2458: {"Kaltmiete zzgl. Nebenkosten (€/month)": 4123,
           "Nebenkosten (€/month)": 248,
           "Heizkosten_2 (€/month)": 186},
    2490: {"Kaltmiete zzgl. Nebenkosten (€/month)": 909.15,
           "Nebenkosten (€/month)": 136.09,
           "Heizkosten_2 (€/month)": 80.42},
    2499: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1400,
           "Nebenkosten (€/month)": 160},
    2519: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1813.94,
           "Nebenkosten (€/month)": 152,
           "Heizkosten_2 (€/month)": 114},
    2540: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1181.26,
           "Nebenkosten (€/month)": 124,
           "Heizkosten_2 (€/month)": 93},
    2617: {"Kaltmiete zzgl. Nebenkosten (€/month)": 350.47,
           "Nebenkosten (€/month)": 106.86,
           "Heizkosten_2 (€/month)": 94.48},
    2703: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1621.30,
           "Nebenkosten (€/month)": 118.22,
           "Heizkosten_2 (€/month)": 88.64},
    2716: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1989.16,
           "Nebenkosten (€/month)": 143.60,
           "Heizkosten_2 (€/month)": 107.70},
    2748: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2600},
    2752: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1100,
           "Nebenkosten (€/month)": 250},
    2759: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2123.50,
           "Nebenkosten (€/month)": 139.38,
           "Heizkosten_2 (€/month)": 104.54},
    2795: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1807.30,
           "Nebenkosten (€/month)": 140,
           "Heizkosten_2 (€/month)": 105},
    2908: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1819.78,
           "Nebenkosten (€/month)": 270.80,
           "Heizkosten_2 (€/month)": 234.50},
    2910: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1950,
           "Nebenkosten (€/month)": 213,
           "Heizkosten_2 (€/month)": 79},
    2921: {"Kaltmiete zzgl. Nebenkosten (€/month)": 4380.30,
           "Nebenkosten (€/month)": 398,
           "Heizkosten_2 (€/month)": 298.50},
    2922: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1290,
           "Nebenkosten (€/month)": 190,
           "Heizkosten_2 (€/month)": 160},
    2940: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1528.30,
           "Nebenkosten (€/month)": 127.96,
           "Heizkosten_2 (€/month)": 95.97},
    2960: {"Kaltmiete zzgl. Nebenkosten (€/month)": 706.70,
           "Nebenkosten (€/month)": 107.29,
           "Heizkosten_2 (€/month)": 67.05},
    2961: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1237.27,
           "Nebenkosten (€/month)": 174.27,
           "Heizkosten_2 (€/month)": 108.92},
    2974: {"Kaltmiete zzgl. Nebenkosten (€/month)": 600},
    2995: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1392.05,
           "Nebenkosten (€/month)": 218.87,
           "Heizkosten_2 (€/month)": 195.94},
    2997: {"Kaltmiete zzgl. Nebenkosten (€/month)": 649.80,
           "Nebenkosten (€/month)": 98.65,
           "Heizkosten_2 (€/month)": 61.65},
    2998: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1652.30,
           "Nebenkosten (€/month)": 174,
           "Heizkosten_2 (€/month)": 130.50},
    3000: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2141,
           "Nebenkosten (€/month)": 162,
           "Heizkosten_2 (€/month)": 121.50},
    3012: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1000,
           "Nebenkosten (€/month)": 300},
    3034: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1392.36,
           "Nebenkosten (€/month)": 218.92,
           "Heizkosten_2 (€/month)": 195.96},
    3037: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1550,
           "Nebenkosten (€/month)": 250,
           "Heizkosten_2 (€/month)": np.nan},
    3088: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1250},
    3091: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1300},
    3106: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1259},
    3117: {"Kaltmiete zzgl. Nebenkosten (€/month)": 700.59,
           "Nebenkosten (€/month)": 120.87,
           "Heizkosten_2 (€/month)": 102.10},
    3122: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1232.50,
           "Nebenkosten (€/month)": 173.59,
           "Heizkosten_2 (€/month)": 108.50},
    3176: {"Kaltmiete zzgl. Nebenkosten (€/month)": 850,
           "Nebenkosten (€/month)": 185,
           "Heizkosten_2 (€/month)": 100},
    3216: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1445.64,
           "Nebenkosten (€/month)": 134,
           "Heizkosten_2 (€/month)": 100.50},
    3223: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1358.49,
           "Nebenkosten (€/month)": 207.19,
           "Heizkosten_2 (€/month)": 129.50},
    3272: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1306.74,
           "Nebenkosten (€/month)": 237.68,
           "Heizkosten_2 (€/month)": 120.21},
    3287: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2840.63,
           "Nebenkosten (€/month)": 198,
           "Heizkosten_2 (€/month)": 148.50},
    3293: {"Kaltmiete zzgl. Nebenkosten (€/month)": 929.70,
           "Nebenkosten (€/month)": 138.22,
           "Heizkosten_2 (€/month)": 92.97},
    3298: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1311.30,
           "Nebenkosten (€/month)": 116,
           "Heizkosten_2 (€/month)": 87},
    3305: {"Kaltmiete zzgl. Nebenkosten (€/month)": 674.14,
           "Nebenkosten (€/month)": 147.68,
           "Heizkosten_2 (€/month)": 63.96},
    3330: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2406.63,
           "Nebenkosten (€/month)": 289.75,
           "Heizkosten_2 (€/month)": 121.50},
    3338: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1393.96,
           "Nebenkosten (€/month)": 104,
           "Heizkosten_2 (€/month)": 78},
    3349: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1838.29,
           "Nebenkosten (€/month)": 126,
           "Heizkosten_2 (€/month)": 94.50},
    3353: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1550,
           "Nebenkosten (€/month)": 250,
           "Heizkosten_2 (€/month)": np.nan},
    3363: {"Kaltmiete zzgl. Nebenkosten (€/month)": 674.40,
           "Nebenkosten (€/month)": 108.18,
           "Heizkosten_2 (€/month)": 67.62},
    3369: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2427.30,
           "Nebenkosten (€/month)": 180,
           "Heizkosten_2 (€/month)": 135},
    3385: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1390,
           "Nebenkosten (€/month)": 110,
           "Heizkosten_2 (€/month)": 82.50},
    3386: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1999},
    3409: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1941.63,
           "Nebenkosten (€/month)": 144,
           "Heizkosten_2 (€/month)": 108},
    3430: {"Kaltmiete zzgl. Nebenkosten (€/month)": 3067.97,
           "Nebenkosten (€/month)": 196,
           "Heizkosten_2 (€/month)": 147},
    3433: {"Kaltmiete zzgl. Nebenkosten (€/month)": 3093.80,
           "Nebenkosten (€/month)": 198,
           "Heizkosten_2 (€/month)": 148.50},
    3444: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1610.96,
           "Nebenkosten (€/month)": 106,
           "Heizkosten_2 (€/month)": 79.50},
    3447: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1600.63,
           "Nebenkosten (€/month)": 104,
           "Heizkosten_2 (€/month)": 78},
    3584: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1880.67,
           "Nebenkosten (€/month)": 110,
           "Heizkosten_2 (€/month)": 82.50},
    3598: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1424.96,
           "Nebenkosten (€/month)": 114,
           "Heizkosten_2 (€/month)": 85.50},
    3640: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1070.33,
           "Nebenkosten (€/month)": 159.27,
           "Heizkosten_2 (€/month)": 130.03},
    3643: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2148.30,
           "Nebenkosten (€/month)": 186,
           "Heizkosten_2 (€/month)": 139.50},
    3647: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1450,
           "Nebenkosten (€/month)": 205,
           "Heizkosten_2 (€/month)": 55},
    3653: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1229.66,
           "Nebenkosten (€/month)": 118,
           "Heizkosten_2 (€/month)": 88.50},
    3672: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1157.33,
           "Nebenkosten (€/month)": 138,
           "Heizkosten_2 (€/month)": 103.50},
    3688: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2100,
           "Nebenkosten (€/month)": 220},
    3697: {"Kaltmiete zzgl. Nebenkosten (€/month)": 950,
           "Nebenkosten (€/month)": 250,
           "Heizkosten_2 (€/month)": 100},
    3716: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2300,
           "Nebenkosten (€/month)": 360},
    3728: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1641.96,
           "Nebenkosten (€/month)": 120,
           "Heizkosten_2 (€/month)": 90},
    3752: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2469.66,
           "Nebenkosten (€/month)": 198,
           "Heizkosten_2 (€/month)": 148.50},
    3754: {"Kaltmiete zzgl. Nebenkosten (€/month)": 3098.97,
           "Nebenkosten (€/month)": 214,
           "Heizkosten_2 (€/month)": 160.50},
    3758: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1410.50,
           "Nebenkosten (€/month)": 140,
           "Heizkosten_2 (€/month)": 105},
    3775: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1730.83,
           "Nebenkosten (€/month)": 134,
           "Heizkosten_2 (€/month)": 100.50},
    3776: {"Kaltmiete zzgl. Nebenkosten (€/month)": 3305.63,
           "Nebenkosten (€/month)": 214,
           "Heizkosten_2 (€/month)": 160.50},
    3779: {"Kaltmiete zzgl. Nebenkosten (€/month)": 4028.96,
           "Nebenkosten (€/month)": 214,
           "Heizkosten_2 (€/month)": 160.50},
    3783: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2013.96,
           "Nebenkosten (€/month)": 140,
           "Heizkosten_2 (€/month)": 105},
    3790: {"Kaltmiete zzgl. Nebenkosten (€/month)": 1746.33,
           "Nebenkosten (€/month)": 106,
           "Heizkosten_2 (€/month)": 79.50},
    471: {"Kaltmiete zzgl. Nebenkosten (€/month)": 2552.33,
           "Nebenkosten (€/month)": 162,
           "Heizkosten_2 (€/month)": 121.50},
    3794: {"Kaltmiete zzgl. Nebenkosten (€/month)": 4132.30,
           "Nebenkosten (€/month)": 280,
           "Heizkosten_2 (€/month)": 210}
}

# Loop over each update and set only the specified columns.
for idx, col_updates in updates.items():
    for col, new_val in col_updates.items():
        df.loc[idx, col] = new_val

# Merging "Deposit 1" and "Deposit 2" into "Deposit"

I combined the information from the 'Deposit 1' and 'Deposit 2' columns into a single 'Deposit' column to simplify the dataset.

In [ ]:
df["Deposit"] = df["Deposit 1"].fillna(df["Deposit 2"])
df.drop(columns=["Deposit 1", "Deposit 2"], inplace=True)

<ipython-input-72-f6ecdc6cca76>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Deposit"] = df["Deposit 1"].fillna(df["Deposit 2"])


I removed 10 listings that were located outside Berlin, as this analysis focuses exclusively on the rental housing market within the city.

In [ ]:
df[['Bezirk', 'Zip Code']]

,Bezirk,Zip Code
0,Treptow-Köpenick,12557
1,Mitte,13359
2,Friedrichshain-Kreuzberg,10178
3,Reinickendorf,13503
4,Pankow,10407
...,...,...
3797,Friedrichshain-Kreuzberg,10317
3799,Mitte,10557
3801,Mitte,10553
3805,Charlottenburg-Wilmersdorf,10715


In [ ]:
df.drop(df[df["Zip Code"].isin([14480, 14513, 16341, 16356, 16548])].index, inplace=True)

In [ ]:
df["Bezirk"].value_counts(dropna=False)

,count
Bezirk,
Friedrichshain-Kreuzberg,696
Mitte,679
Charlottenburg-Wilmersdorf,446
Pankow,363
Treptow-Köpenick,184
Tempelhof-Schöneberg,156
Neukölln,148
Steglitz-Zehlendorf,91
Lichtenberg-Hohenschönhausen,74


# Cleaning columns related to real estate agencies

Here I standardized and consolidated real estate-related data by:

- Replacing values in multiple real estate columns with specific company names or "Privater Anbieter."
- Removing redundant columns to keep only "Real Estate 3".
- Grouping rare values in "Real Estate 3" under "other" if they appear fewer than 5 times.

In [ ]:
real_estate_cols = ["Real Estate 1", "Real Estate 2", "Real Estate 3", "Real Estate 4","Real Estate 5 Subtitle", "Real Estate 5 Title"]
df.loc[df["Real Estate 4"] == "Privater Anbieter", real_estate_cols] = "Privater Anbieter"
df.loc[df["Real Estate 3"] == "Ambius Immobilien GmbH", real_estate_cols] = "Ambius Immobilien GmbH"
df.loc[df["Real Estate 3"] == "Blueground Germany GmbH", real_estate_cols] = "Blueground Germany GmbH"
df.loc[df["Real Estate 3"] == "HOMELIKE INTERNET GmbH", real_estate_cols] = "HOMELIKE INTERNET GmbH"
df.loc[df["Real Estate 3"] == "talyo. Property Services GmbH", real_estate_cols] = "talyo. Property Services GmbH"
df.loc[df["Real Estate 3"] == "Ukio Germany", real_estate_cols] = "Ukio Germany"
df.loc[df["Real Estate 4"] == "VON POLL IMMOBILIEN Shop Berlin City-West", real_estate_cols] = "VON POLL IMMOBILIEN Shop Berlin City-West"
df.loc[df["Real Estate 3"] == "Müller Merkle Immobilien GmbH", real_estate_cols] = "Müller Merkle Immobilien GmbH"
df.loc[df["Real Estate 3"] == "ohne-makler.net - Immobilien selbst vermarkten", real_estate_cols] = "ohne-makler.net - Immobilien selbst vermarkten"
df.loc[df["Real Estate 3"] == "CASTILLO management GmbH&Co KG", real_estate_cols] = "CASTILLO management GmbH&Co KG"
df.loc[df["Real Estate 3"] == "Urban Ground GmbH", real_estate_cols] = "Urban Ground GmbH"
df.loc[df["Real Estate 3"] == "EW Immobilien Real Estate GmbH", real_estate_cols] = "EW Immobilien Real Estate GmbH"
df.loc[df["Real Estate 3"] == "SCHNORR & PARTNER IMMOBILIEN", real_estate_cols] = "SCHNORR & PARTNER IMMOBILIEN"
df.loc[df["Real Estate 3"] == "ZBVV - Zentral Boden Vermietung und Verwaltung GmbH", real_estate_cols] = "ZBVV - Zentral Boden Vermietung und Verwaltung GmbH"
df.loc[df["Real Estate 3"] == "BUWOG Immobilien Treuhand GmbH­", real_estate_cols] = "BUWOG Immobilien Treuhand GmbH­"
df.loc[df["Real Estate 3"] == "Grand City Property Ltd. Zweigniederlassung Deutschland", real_estate_cols] = "Grand City Property Ltd. Zweigniederlassung Deutschland­"
df.loc[df["Real Estate 3"] == "HAMBURG TEAM Property Management GmbH", real_estate_cols] = "HAMBURG TEAM Property Management GmbH­"
df.loc[df["Real Estate 3"] == "HIHC Horvat Real Estate GmbH", real_estate_cols] = "HIHC Horvat Real Estate GmbH­"
df.loc[df["Real Estate 3"] == "KENSINGTON Finest Properties International", real_estate_cols] = "KENSINGTON Finest Properties International­"
df.loc[df["Real Estate 3"] == "Lanz & Melzer ImmobilienService GmbH", real_estate_cols] = "Lanz & Melzer ImmobilienService GmbH­"
df.loc[df["Real Estate 3"] == "Lienz Liegenschaften GmbH & Co. Engelhardt Immobilien KG", real_estate_cols] = "Lienz Liegenschaften GmbH & Co. Engelhardt Immobilien KG­"
df.loc[df["Real Estate 3"] == "MGI Immobilien Inh. Marco Güttler", real_estate_cols] = "MGI Immobilien Inh. Marco Güttler"
df.loc[df["Real Estate 3"] == "Meine Makler", real_estate_cols] = "Meine Makler - Frau Birgit Heinemann"
df.loc[df["Real Estate 3"] == "Mietz GmbH", real_estate_cols] = "Mietz GmbH"
df.loc[df["Real Estate 3"] == "AAAA-Immobilien", real_estate_cols] = "AAAA-Immobilien"
df.loc[df["Real Estate 3"] == "ADEN Immo GmbH", real_estate_cols] = "ADEN Immo GmbH"
df.loc[df["Real Estate 3"] == "AGROMEX Invest GmbH", real_estate_cols] = "AGROMEX Invest GmbH"
df.loc[df["Real Estate 3"] == "ARRIVA Relocation & Immobilien", real_estate_cols] = "ARRIVA Relocation & Immobilien"
df.loc[df["Real Estate 3"] == "Acopio Facility GmbH & Co. KG", real_estate_cols] = "Acopio Facility GmbH & Co. KG"
df.loc[df["Real Estate 3"] == "Adler Group", real_estate_cols] = "Adler Group"
df.loc[df["Real Estate 3"] == "Agas Immobilien GmbH", real_estate_cols] = "Agas Immobilien GmbH"
df.loc[df["Real Estate 3"] == "Alexandra Hausverwaltung und Service GmbH", real_estate_cols] = "Alexandra Hausverwaltung und Service GmbH"
df.loc[df["Real Estate 3"] == "Alma Via Pflegegruppe GmbH Der Sofienhof", real_estate_cols] = "Alma Via Pflegegruppe GmbH Der Sofienhof"
df.loc[df["Real Estate 3"] == "Alpin Real Estate Projektentwicklungs- und Vertriebs GmbH & Co KG", real_estate_cols] = "Alpin Real Estate Projektentwicklungs- und Vertriebs GmbH & Co KG"
df.loc[df["Real Estate 3"] == "Andrea Corsica Goymann", real_estate_cols] = "Andrea Corsica Goymann"
df.loc[df["Real Estate 3"] == "Architekturbüro-Hausverwaltung Greinert Inh. Eva Greinert", real_estate_cols] = "Architekturbüro-Hausverwaltung Greinert Inh. Eva Greinert"
df.loc[df["Real Estate 3"] == "Areal Service GmbH", real_estate_cols] = "Areal Service GmbH"
df.loc[df["Real Estate 3"] == "Arges Immobilien", real_estate_cols] = "Arges Immobilien"
df.loc[df["Real Estate 3"] == "Argos Real Estate GmbH", real_estate_cols] = "Argos Real Estate GmbH"
df.loc[df["Real Estate 3"] == "B.I.S. Berliner ImmobilienService GmbH", real_estate_cols] = "B.I.S. Berliner ImmobilienService GmbH"
df.loc[df["Real Estate 3"] == "BB Estates GmbH", real_estate_cols] = "BB Estates GmbH"
df.loc[df["Real Estate 3"] == "BETTERHOMES Deutschland GmbH", real_estate_cols] = "BETTERHOMES Deutschland GmbH"
df.loc[df["Real Estate 3"] == "BGP Immobilien", real_estate_cols] = "BGP Immobilien"
df.loc[df["Real Estate 3"] == "BWK-Campus GmbH", real_estate_cols] = "BWK-Campus GmbH"
df.loc[df["Real Estate 3"] == "Bauwerk Capital GmbH & Co. KG", real_estate_cols] = "Bauwerk Capital GmbH & Co. KG"
df.loc[df["Real Estate 3"] == "Bellatris Real Estate GmbH & Co. KG", real_estate_cols] = "Bellatris Real Estate GmbH & Co. KG"
df.loc[df["Real Estate 3"] == "Berlin City Properties", real_estate_cols] = "Berlin City Properties"
df.loc[df["Real Estate 3"] == "Berlin Immobilien Spezialist Neumann GmbH", real_estate_cols] = "Berlin Immobilien Spezialist Neumann GmbH"
df.loc[df["Real Estate 3"] == "Berliner Bau- und Wohnungsgenossenschaft von 1892 eG", real_estate_cols] = "Berliner Bau- und Wohnungsgenossenschaft von 1892 eG"
df.loc[df["Real Estate 3"] == "Berlinhaus Verwaltung GmbH", real_estate_cols] = "Berlinhaus Verwaltung GmbH"
df.loc[df["Real Estate 2"] == "Firma DAHLER & COMPANY Berlin GmbH & Co. KG", real_estate_cols] = "Firma DAHLER & COMPANY Berlin GmbH & Co. KG"
df.loc[df["Real Estate 3"] == "Engel & Völkers Immobilien Deutschland GmbH", real_estate_cols] = "Engel & Völkers Berlin Mitte GmbH"
df.loc[df["Real Estate 5 Subtitle"] == "Privater Anbieter", real_estate_cols] = "Privater Anbieter"
df.loc[df["Real Estate 5 Title"] == "Privater Anbieter", real_estate_cols] = "Privater Anbieter"
df.loc[df["Real Estate 3"] == "Inseriert auf 123provisionsfrei.de", real_estate_cols] = "Privater Anbieter"
df.loc[df["Real Estate 3"] == "VON POLL IMMOBILIEN Berlin - Nikolassee/Wannsee - R-Quadrat Immobilien GmbH", real_estate_cols] = "VON POLL IMMOBILIEN Berlin"
df.loc[df["Real Estate 3"] == "VON POLL IMMOBILIEN Berlin - Pankow - Ulf Sobeck", real_estate_cols] = "VON POLL IMMOBILIEN Berlin"
df.loc[df["Real Estate 3"] == "VON POLL IMMOBILIEN Berlin - Treptow-Köpenick - Immobilienträume Berlin Süd-Ost GmbH", real_estate_cols] = "VON POLL IMMOBILIEN Berlin"
df.loc[df["Real Estate 3"] == "VON POLL IMMOBILIEN Berlin-Mitte - Quartier Sieben Real Estate GmbH", real_estate_cols] = "VON POLL IMMOBILIEN Berlin"
df.loc[df["Real Estate 3"] == "VON POLL IMMOBILIEN Berlin-Prenzlauer Berg - Ulf Sobeck", real_estate_cols] = "VON POLL IMMOBILIEN Berlin"
df.loc[df["Real Estate 3"] == "VON POLL IMMOBILIEN Shop Berlin City-West", real_estate_cols] = "VON POLL IMMOBILIEN Berlin"

df.drop(columns=["Real Estate 1", "Real Estate 2", "Real Estate 4","Real Estate 5 Subtitle", "Real Estate 5 Title"], inplace=True)
counts = df["Real Estate 3"].value_counts() # Find value counts
to_replace = counts[counts < 5].index # Identify values that appear less than 5 times
df["Real Estate 3"] = df["Real Estate 3"].replace(to_replace, "other") # Replace these values with "other"

# Features cleaning

## Feature 1


I cleaned and reassigned values from the 'Feature_1' column as follows:

- Transferring Availability Information:
  - Moved specific 'Feature_1' values to the 'availability' column for certain rows.
  - For those rows, set 'Feature_1' to NaN to avoid duplication.
- Handling Specific Keywords:
  - Identified rows where 'Feature_1' contained the word "ab" (e.g., "ab 01.2025") and set 'Feature_1' to NaN.
  - Applied the same approach for:
    - "ofor" (to capture both "Sofort" and "sofort").
    - "frag" (to target "auf Anfrage" and "auf anfrage").
    - "nach Absprache" (on request).
- Manual Assignments:
  - After reviewing the dataset and URLs for specific rows, I manually assigned "sofort" to the 'availability' column where appropriate.
- Reassigning Values & Handling Unusable Data:
  - Reassigned additional values to their corresponding columns when possible.
  - Replaced unusable values with np.nan where necessary.
- Avoiding Duplicates:
  - If a value was already present in its corresponding column, I simply removed it from 'Feature_1' instead of duplicating it.


In [ ]:
#####Feature 1######
indices = [462, 835, 2099, 2473, 3099, 3470]
df.loc[indices, "availability"] = df.loc[indices, "Feature_1"]
df.loc[indices, "Feature_1"] = np.nan

indices = df[df["Feature_1"].str.contains("ab", na=False)].index.tolist()
df.loc[indices, "Feature_1"] = np.nan

indices = df[df["Feature_1"].str.contains("ofor", na=False)].index.tolist()
df.loc[indices, "availability"] = "sofort"
df.loc[indices, "Feature_1"] = np.nan

indices = df[df["Feature_1"].str.contains("frag", na=False)].index.tolist()
df.loc[indices, "availability"] = np.nan
df.loc[indices, "Feature_1"] = np.nan

indices = df[df["Feature_1"].str.contains("nach Absprache", na=False)].index.tolist()
df.loc[indices, "availability"] = np.nan
df.loc[indices, "Feature_1"] = np.nan

indices = [2035, 2074, 3542]
df.loc[indices, "availability"] = "sofort"

indices = df[df["Feature_1"].str.contains("nach Vereinbarung", na=False)].index.tolist()
df.loc[indices, "Feature_1"] = np.nan

indices = df[df["Feature_1"].str.contains("voraussichtlich 04.2025", na=False)].index.tolist()
df.loc[indices, "availability"] = "04.2025"
df.loc[indices, "Feature_1"] = np.nan

indices = df[df["Feature_1"].str.contains("ca. Nov. 22", na=False)].index.tolist()
df.loc[indices, "availability"] = np.nan
df.loc[indices, "Feature_1"] = np.nan

indices = df[df["Feature_1"].str.contains("eschoss", na=False)].index.tolist()
df.loc[indices, "Feature_1"] = np.nan

## Feature 2

I cleaned and transferred relevant availability-related values from 'Feature_2' to 'availability', while also handling floor-related information when applicable.

Steps Taken:
- Handling Availability Keywords:
  - Identified rows where 'Feature_2' contained "Frei ab" or "frei ab", then set those values to NaN.
  - Found rows where 'Feature_2' contained "sofort", transferred "sofort" to 'availability', and then cleared 'Feature_2' after processing.
- Exception Handling:
  - Row 1100 was excluded from the process because the 'availability' column already contained more specific information. After reviewing the URL of the listing, I determined that this original information was more relevant than simply replacing it with "sofort".
- Manual Assignments:
  - Using printed values from 'Feature_2' and 'availability', I manually moved relevant values from 'Feature_2' to 'availability' for specific indices (2152, 807, 2106) and cleared 'Feature_2' afterward.
- Handling Unusable or Redundant Data:
  - Set unusable or redundant values in 'Feature_2' to np.nan.
- Floor Information Handling:
  - Found occurrences of "eschoss" (a part of "Geschoss", meaning "floor" in German).
  - Removed it from 'Feature_2' and manually reassigned the extracted values to the 'Floor' column.


In [ ]:
#####Feature 2######
indices = df[df["Feature_2"].str.contains("Frei ab", na=False)].index.tolist()
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("frei ab", na=False)].index.tolist()
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("sofort", na=False)].index.tolist()
if 1100 in indices:
    indices.remove(1100)
df.loc[indices, "availability"] = df.loc[indices, "Feature_2"]
indices = df[df["Feature_2"].str.contains("sofort", na=False)].index.tolist()
df.loc[indices, "Feature_2"] = np.nan

df.loc[2152, "availability"] = df.loc[2152, "Feature_2"]
df.loc[2152, "Feature_2"] = np.nan

df.loc[807, "availability"] = df.loc[807, "Feature_2"]
df.loc[807, "Feature_2"] = np.nan

df.loc[2106, "availability"] = df.loc[2106, "Feature_2"]
df.loc[2106, "Feature_2"] = np.nan

df.loc[1030, "Feature_2"] = np.nan
df.loc[1469, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("ca. März 23", na=False)].index.tolist()
df.loc[indices, "availability"] = df.loc[indices, "Feature_2"]
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("Nach Vereinbarung", na=False)].index.tolist()
df.loc[indices, "availability"] = np.nan
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("nach Vereinbarung", na=False)].index.tolist()
if 1977 in indices:
    indices.remove(1977)
df.loc[indices, "availability"] = np.nan
indices = df[df["Feature_2"].str.contains("nach Vereinbarung", na=False)].index.tolist()
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("SOFORT", na=False)].index.tolist()
df.loc[indices, "availability"] = "sofort"
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("nach Absprache", na=False)].index.tolist()
df.loc[indices, "availability"] = np.nan
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("voraussichtlich 15.12.2024", na=False)].index.tolist()
df.loc[indices, "availability"] = df.loc[indices, "Feature_2"]
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("2027", na=False)].index.tolist()
df.loc[indices, "availability"] = df.loc[indices, "Feature_2"]
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("ab 01.02.2025", na=False)].index.tolist()
df.loc[indices, "availability"] = df.loc[indices, "Feature_2"]
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("Nach Absprache", na=False)].index.tolist()
df.loc[indices, "availability"] = np.nan
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("01-2025", na=False)].index.tolist()
df.loc[indices, "availability"] = df.loc[indices, "Feature_2"]
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("1960", na=False)].index.tolist()
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("2022", na=False)].index.tolist()
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("voraussichtlich 01.02.2025", na=False)].index.tolist()
df.loc[indices, "availability"] = df.loc[indices, "Feature_2"]
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("eschoss", na=False)].index.tolist()
df.loc[indices, "Feature_2"] = np.nan
df.loc[961, "Floor"] = 0
df.loc[2344, "Floor"] = 1
df.loc[3024, "Floor"] = 1
df.loc[3047, "Floor"] = 1

## Feature 3

The cleaning process for 'Feature_3' followed a similar approach to the one used for 'Feature_1' and 'Feature_2', with some key differences:

- Handling Outdated Availability Dates:
  - Some availability-related values in 'Feature_3' mentioned dates older than 12.02.25 (the date of data scraping).
  - To standardize the data, I replaced any date equal to or older than 12.02.25 with "sofort" (immediately available).
  - Listings that only mentioned availability from 2025 were interpreted as 01.01.25. Since this date is before 12.02.25, I also replaced those values with "sofort".
- Creating a Lease Duration Column ('time_lim'):
  - Unlike 'Feature_1' and 'Feature_2', some values in 'Feature_3' provided information about lease duration rather than availability.
  - Since there was no existing column to store this information, I created a new column called 'time_lim' and populated it with the relevant values instead of discarding them.

In [ ]:
#####Feature 3######
indices = df[df["Feature_3"].str.contains("sofort", na=False)].index.tolist()
df.loc[indices, "availability"] = df.loc[indices, "Feature_3"]
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("frei ab", na=False)].index.tolist()
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("nach Absprache", na=False)].index.tolist()
df.loc[indices, "availability"] = np.nan
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("nach Vereinbarung", na=False)].index.tolist()
df.loc[826, "availability"] = "sofort"
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("ab Januar 2025", na=False)].index.tolist()
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("Sofort", na=False)].index.tolist()
df.loc[indices, "availability"] = "sofort"
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("2025", na=False)].index.tolist()
df.loc[[55, 1054], "availability"] = "sofort"
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("ab 01.11.2024", na=False)].index.tolist()
df.loc[indices, "availability"] = "sofort"
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("ab 01.04.2019", na=False)].index.tolist()
df.loc[indices, "availability"] = np.nan
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("Nach Vereinbarung", na=False)].index.tolist()
df.loc[indices, "availability"] = np.nan
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("auf Anfrage", na=False)].index.tolist()
df.loc[indices, "availability"] = np.nan
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("10/2024", na=False)].index.tolist()
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("2007", na=False)].index.tolist()
df.loc[indices, "availability"] = np.nan
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("03/2023", na=False)].index.tolist()
df.loc[indices, "availability"] = np.nan
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("01.02.205", na=False)].index.tolist()
df.loc[indices, "Feature_3"] = np.nan

df["time_lim"] = np.nan
indices = df[df["Feature_3"].str.contains("24 Monate", na=False)].index.tolist()
df.loc[indices, "time_lim"] = "24 Monate"
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("eschoss", na=False)].index.tolist()
df.loc[indices, "Feature_3"] = np.nan
df.loc[168, "Floor"] = 4
df.loc[608, "Floor"] = 2
df.loc[1041, "Floor"] = 4
df.loc[2005, "Floor"] = 6
df.loc[2075, "Floor"] = 2
df.loc[2191, "Floor"] = 3
df.loc[2218, "Floor"] = 2
df.loc[2488, "Floor"] = 1
df.loc[3604, "Floor"] = 4

<ipython-input-79-de3fc4c2bde0>:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '24 Monate' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[indices, "time_lim"] = "24 Monate"


## Feature 4

The cleaning process for this column followed the same approach as for 'Feature_1', 'Feature_2', and 'Feature_3'. Additionally, I continued populating the 'time_lim' column where relevant to retain lease duration information.

In [ ]:
#####Feature 4######
indices = df[df["Feature_4"].str.contains("sofort", na=False)].index.tolist()
df.loc[indices, "Feature_4"] = np.nan

indices = df[df["Feature_4"].str.contains("frei ab", na=False)].index.tolist()
df.loc[indices, "Feature_4"] = np.nan

indices = df[df["Feature_4"].str.contains("nach Vereinbarung", na=False)].index.tolist()
df.loc[indices, "availability"] = np.nan
df.loc[indices, "Feature_4"] = np.nan

indices = df[df["Feature_4"].str.contains("ab", na=False)].index.tolist()
df.loc[indices, "Feature_4"] = np.nan

indices = df[df["Feature_4"].str.contains("2025", na=False)].index.tolist()
df.loc[indices, "Feature_4"] = np.nan
df.loc[[1017], "availability"] = "01.04.2025"

indices = df[df["Feature_4"].str.contains("1994", na=False)].index.tolist()
df.loc[indices, "Feature_4"] = np.nan

indices = df[df["Feature_4"].str.contains("Erdgeschoss", na=False)].index.tolist()
df.loc[indices, "Floor"] = 0
df.loc[indices, "Feature_4"] = np.nan

indices = df[df["Feature_4"].str.contains("eschoss", na=False)].index.tolist()
df.loc[indices, "Floor"] = [int(re.search(r'\d+', s).group())
     for s in df.loc[df["Feature_4"].str.contains("eschoss", na=False), "Feature_4"]]
df.loc[indices, "Feature_4"] = np.nan

## Feature 5

In this step, I extracted and reassigned relevant information from 'Feature_5' to appropriate columns while removing redundant or unusable data.

The following transformations were applied:
- Extracted lease duration information → stored in 'time_lim'.
- Extracted floor level information → stored in 'Floor'.
- Identified multiple-floor properties → updated the 'Multiple floors' column accordingly.
- Removed redundant or unusable values from 'Feature_5'.
- Deleted extracted values from 'Feature_5' to keep the column clean and focused.


In [ ]:
#####Feature 5######
indices = df[df["Feature_5"].str.contains("Monat", na=False)].index.tolist()
df.loc[indices, "time_lim"] = df.loc[indices, "Feature_5"]
df.loc[indices, "Feature_5"] = np.nan

df.loc[2149, "Floor"] = 1
df.loc[2149, "Multiple floors"] = 1
df.loc[2149, "Feature_5"] = np.nan

indices = df[df["Feature_5"].str.contains("Erdgeschoss", na=False)].index.tolist()
df.loc[indices, "Floor"] = 0
df.loc[indices, "Feature_5"] = np.nan

df.loc[[0, 2349, 2429, 2456, 2487, 2535, 2563, 2618, 2938, 3515], "Multiple floors"] = 1
df.loc[[0, 2349, 2429, 2456, 2487, 2535, 2563, 2618, 2938, 3515], "Feature_5"] = np.nan

indices = df[df["Feature_5"].str.contains("eschoss", na=False)].index.tolist()
df.loc[indices, "Floor"] = [int(re.search(r'\d+', s).group())
     for s in df.loc[df["Feature_5"].str.contains("eschoss", na=False), "Feature_5"]]
df.loc[indices, "Feature_5"] = np.nan

## Feature 6

In this step, I extracted and reassigned relevant information from 'Feature_6' to appropriate columns while ensuring the data remains structured and clean.

The following transformations were applied:
- Extracted lease duration information → stored in 'time_lim'.
- Extracted floor level information → stored in 'Floor'.
- Identified multiple-floor properties → updated the 'Multiple floors' column accordingly.
- Removed redundant or unusable values from 'Feature_6'.
- Deleted extracted values from 'Feature_6' to keep the column focused and clean.

In [ ]:
#####Feature 6######
indices = df[df["Feature_6"].str.contains("Monat", na=False)].index.tolist()
df.loc[indices, "time_lim"] = df.loc[indices, "Feature_6"]
df.loc[indices, "Feature_6"] = np.nan

df.loc[3, "Floor"] = 0
indices = df[df["Feature_6"].str.contains("Erdgeschoss", na=False)].index.tolist()
df.loc[indices, "Feature_6"] = np.nan

df.loc[[19, 83, 2389, 2408, 2436, 2558, 2565 ], "Multiple floors"] = 1
df.loc[[19, 83, 98, 2334, 2372, 2389, 2397, 2408, 2436, 2493, 2558, 2565], "Feature_6"] = np.nan

indices = df[df["Feature_6"].str.contains("eschoss", na=False)].index.tolist()
df.loc[indices, "Floor"] = [int(re.search(r'\d+', s).group())
     for s in df.loc[df["Feature_6"].str.contains("eschoss", na=False), "Feature_6"]]
df.loc[indices, "Feature_6"] = np.nan

## Feature 7

In this step, I extracted and reassigned relevant information from 'Feature_7' to appropriate columns while ensuring data consistency.

The following transformations were applied:
- Extracted lease duration information → stored in 'time_lim'.
- Extracted floor level information → stored in 'Floor'.
- Removed redundant or unusable values from 'Feature_7'.
- Deleted extracted values from 'Feature_7' to keep the column clean and structured.

In [ ]:
#####Feature 7######
indices = df[df["Feature_7"].str.contains("onat", na=False)].index.tolist()
df.loc[indices, "time_lim"] = df.loc[indices, "Feature_7"]
df.loc[indices, "Feature_7"] = np.nan

indices = df[df["Feature_7"].str.contains("Erdgeschoss", na=False)].index.tolist()
df.loc[indices, "Floor"] = 0
df.loc[indices, "Feature_7"] = np.nan

df.loc[2325, "Feature_7"] = np.nan

indices = df[df["Feature_7"].str.contains("eschoss", na=False)].index.tolist()
df.loc[indices, "Floor"] = [int(re.search(r'\d+', s).group())
     for s in df.loc[df["Feature_7"].str.contains("eschoss", na=False), "Feature_7"]]
df.loc[indices, "Feature_7"] = np.nan

## Feature 8

In this step, I extracted and reassigned relevant information from 'Feature_8' to appropriate columns while ensuring data consistency.

The following transformations were applied:
- Extracted lease duration information → stored in 'time_lim'.
- Extracted floor level information → stored in 'Floor'.
- Identified multiple-floor properties → updated the 'Multiple floors' column accordingly.
- Removed redundant or unusable values from 'Feature_8'.
- Deleted extracted values from 'Feature_8' to keep the column clean and structured.

In [ ]:
#####Feature 8######
indices = df[df["Feature_8"].str.contains("onat", na=False)].index.tolist()
df.loc[indices, "time_lim"] = df.loc[indices, "Feature_8"]
df.loc[indices, "Feature_8"] = np.nan

indices = df[df["Feature_8"].str.contains("Erdgeschoss", na=False)].index.tolist()
df.loc[indices, "Feature_8"] = np.nan

df.loc[[2166, 2351, 2373, 2402], "Multiple floors"] = 1
df.loc[[2166, 2351, 2373, 2402], "Feature_8"] = np.nan

indices = df[df["Feature_8"].str.contains("eschoss", na=False)].index.tolist()
df.loc[indices, "Floor"] = [int(re.search(r'\d+', s).group())
     for s in df.loc[df["Feature_8"].str.contains("eschoss", na=False), "Feature_8"]]
df.loc[indices, "Feature_8"] = np.nan

## New column "lim_inhabitants"

After inspecting the distinct values remaining in 'Feature_1' to 'Feature_8', I decided to create a new column to store information about inhabitant limitations instead of simply deleting those values.
- A value of 1 in this new column indicates that the listing has inhabitant restrictions.

Methodology:
- I searched for rows where any of the 'Feature_4' to 'Feature_8' columns contained "erson", capturing variations like:
  - "Person"
  - "person"
  - "Personen"
  - "personen"
- I did not include 'Feature_1' to 'Feature_3' in the filtering process, as these columns did not contain relevant values matching "erson".

In [ ]:
df['Feature_1'].value_counts(dropna=False)

,count
Feature_1,
"Einbauküche, Kochnische",1219
Einbauküche,846
"Einbauküche, Offene Küche",405
Personenaufzug,234
NaN,134
neuwertig,48
Offene Küche,26
Kein Personenaufzug,21
vermietet,19


In [ ]:
df['Feature_4'].value_counts(dropna=False)

,count
Feature_4,
NaN,1726
möbliert,350
Balkon,321
Kelleranteil,188
Terrasse,71
Haustiere erlaubt,47
Garten,41
Außen-Stellplatz,39
Bad mit Dusche,39


In [ ]:
df["lim_inhabitants"] = np.nan

indices = df[df["Feature_4"].str.contains("erson", na=False)].index.tolist()
df.loc[indices, "lim_inhabitants"] = 1
df.loc[indices, "Feature_4"] = np.nan

indices = df[df["Feature_5"].str.contains("erson", na=False)].index.tolist()
df.loc[indices, "lim_inhabitants"] = 1
df.loc[indices, "Feature_5"] = np.nan

indices = df[df["Feature_6"].str.contains("erson", na=False)].index.tolist()
df.loc[indices, "lim_inhabitants"] = 1
df.loc[indices, "Feature_6"] = np.nan

indices = df[df["Feature_7"].str.contains("erson", na=False)].index.tolist()
df.loc[indices, "lim_inhabitants"] = 1
df.loc[indices, "Feature_7"] = np.nan

indices = df[df["Feature_8"].str.contains("erson", na=False)].index.tolist()
df.loc[indices, "lim_inhabitants"] = 1
df.loc[indices, "Feature_8"] = np.nan

In [ ]:
df['Feature_4'].value_counts(dropna=False)

,count
Feature_4,
NaN,1732
möbliert,350
Balkon,321
Kelleranteil,188
Terrasse,71
Haustiere erlaubt,47
Garten,41
Außen-Stellplatz,39
Bad mit Dusche,39


## New column "balcony"


After reviewing the distinct values remaining in 'Feature_1' to 'Feature_8', I decided to create a new column to store information about whether a flat or house has a balcony.
- A value of 1 in this new column indicates that the listing includes a balcony.

Methodology:
- I searched for rows where any of the 'Feature_1' to 'Feature_5' columns contained "alkon", capturing variations like:
  - "balkon"
  - "Balkon"
- I excluded 'Feature_6' to 'Feature_8', as they did not contain relevant values with "alkon".

In [ ]:
df['Feature_1'].value_counts(dropna=False)

,count
Feature_1,
"Einbauküche, Kochnische",1219
Einbauküche,846
"Einbauküche, Offene Küche",405
Personenaufzug,234
NaN,134
neuwertig,48
Offene Küche,26
Kein Personenaufzug,21
vermietet,19


In [ ]:
df["balcony"] = np.nan

indices = df[df["Feature_1"].str.contains("alkon", na=False)].index.tolist()
df.loc[indices, "balcony"] = 1
df.loc[indices, "Feature_1"] = np.nan

indices = df[df["Feature_2"].str.contains("alkon", na=False)].index.tolist()
df.loc[indices, "balcony"] = 1
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_3"].str.contains("alkon", na=False)].index.tolist()
df.loc[indices, "balcony"] = 1
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_4"].str.contains("alkon", na=False)].index.tolist()
df.loc[indices, "balcony"] = 1
df.loc[indices, "Feature_4"] = np.nan

indices = df[df["Feature_5"].str.contains("alkon", na=False)].index.tolist()
df.loc[indices, "balcony"] = 1
df.loc[indices, "Feature_5"] = np.nan

In [ ]:
df['Feature_1'].value_counts(dropna=False)

,count
Feature_1,
"Einbauküche, Kochnische",1219
Einbauküche,846
"Einbauküche, Offene Küche",405
Personenaufzug,234
NaN,143
neuwertig,48
Offene Küche,26
Kein Personenaufzug,21
vermietet,19


## New column "furnished"

After reviewing the distinct values remaining in 'Feature_1' to 'Feature_8', I created a new column to store information about whether a flat or house is furnished.
- A value of 1 in this new column indicates that the listing is furnished.

Methodology:
- I searched for rows where any of the 'Feature_1' to 'Feature_8' columns contained "blier", capturing variations like:
  - "möbliert"
  - "Möbliert"


In [ ]:
df['Feature_2'].value_counts(dropna=False)

,count
Feature_2,
NaN,2094
Personenaufzug,578
"Personenaufzug, Lastenaufzug",83
DSL-Anschluss,49
neuwertig,37
möbliert,29
Terrasse,25
Haustiere erlaubt,21
Kelleranteil,19


In [ ]:
df["furnished"] = np.nan

indices = df[df["Feature_2"].str.contains("blier", na=False)].index.tolist()
df.loc[indices, "furnished"] = 1
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_3"].str.contains("blier", na=False)].index.tolist()
df.loc[indices, "furnished"] = 1
df.loc[indices, "Feature_3"] = np.nan

df.loc[[0, 2149, 2563, 2938, 3515], "Feature_4"] = np.nan
indices = df[df["Feature_4"].str.contains("blier", na=False)].index.tolist()
df.loc[indices, "furnished"] = 1
df.loc[indices, "Feature_4"] = np.nan

df.loc[[5, 83, 98, 2334, 2397, 2408, 2493, 2558], "Feature_5"] = np.nan
indices = df[df["Feature_5"].str.contains("blier", na=False)].index.tolist()
df.loc[indices, "furnished"] = 1
df.loc[indices, "Feature_5"] = np.nan

indices = df[df["Feature_6"].str.contains("blier", na=False)].index.tolist()
df.loc[indices, "furnished"] = 1
df.loc[indices, "Feature_6"] = np.nan

indices = df[df["Feature_7"].str.contains("blier", na=False)].index.tolist()
df.loc[indices, "furnished"] = 1
df.loc[indices, "Feature_7"] = np.nan

indices = df[df["Feature_8"].str.contains("blier", na=False)].index.tolist()
df.loc[indices, "furnished"] = 1
df.loc[indices, "Feature_8"] = np.nan

In [ ]:
df['Feature_2'].value_counts(dropna=False)

,count
Feature_2,
NaN,2125
Personenaufzug,578
"Personenaufzug, Lastenaufzug",83
DSL-Anschluss,49
neuwertig,37
Terrasse,25
Haustiere erlaubt,21
Kelleranteil,19
Garten,11


## New column "garden"

I followed the same process and steps used for populating the 'lim_inhabitants', 'balcony', and 'furnished' columns to identify listings with a garden.

Handling 'Wintergarten' Cases:
- In some instances, the filtering captured "Wintergarten" instead of just "Garten".
- Since "Wintergarten" refers to a sunroom/winter garden, I reassigned those values to the 'balcony' column instead of 'garden'.

In [ ]:
df['Feature_1'].value_counts(dropna=False)

,count
Feature_1,
"Einbauküche, Kochnische",1219
Einbauküche,846
"Einbauküche, Offene Küche",405
Personenaufzug,234
NaN,143
neuwertig,48
Offene Küche,26
Kein Personenaufzug,21
vermietet,19


In [ ]:
df["garden"] = np.nan

indices = df[df["Feature_1"].str.contains("rten", na=False)].index.tolist()
df.loc[indices, "garden"] = 1
df.loc[indices, "Feature_1"] = np.nan

indices = df[df["Feature_2"].str.contains("rten", na=False)].index.tolist()
df.loc[indices, "garden"] = 1
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_3"].str.contains("rten", na=False)].index.tolist()
df.loc[indices, "garden"] = 1
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_4"].str.contains("rten", na=False)].index.tolist()
df.loc[indices, "garden"] = 1
df.loc[indices, "Feature_4"] = np.nan

indices = df[df["Feature_5"].str.contains("rten", na=False)].index.tolist()
df.loc[indices, "garden"] = 1
df.loc[indices, "Feature_5"] = np.nan

indices = df[df["Feature_6"].str.contains("rten", na=False)].index.tolist()
df.loc[1968, "garden"] = 1
df.loc[3017, "balcony"] = 1
df.loc[indices, "Feature_6"] = np.nan

indices = df[df["Feature_7"].str.contains("rten", na=False)].index.tolist()
df.loc[indices, "balcony"] = 1
df.loc[indices, "Feature_7"] = np.nan

indices = df[df["Feature_8"].str.contains("rten", na=False)].index.tolist()
df.loc[indices, "balcony"] = 1
df.loc[indices, "Feature_8"] = np.nan

## New column "Terrasse"

I followed the same process and steps used for populating the 'lim_inhabitants', 'balcony', 'furnished', and 'garden' columns to extract relevant information for this new column.

In [ ]:
df['Feature_2'].value_counts(dropna=False)

,count
Feature_2,
NaN,2143
Personenaufzug,578
"Personenaufzug, Lastenaufzug",83
DSL-Anschluss,49
neuwertig,37
Terrasse,25
Haustiere erlaubt,21
Kelleranteil,19
Einbauküche,9


In [ ]:
df["Terrasse"] = np.nan

indices = df[df["Feature_2"].str.contains("errass", na=False)].index.tolist()
df.loc[indices, "Terrasse"] = 1
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_3"].str.contains("errass", na=False)].index.tolist()
df.loc[indices, "Terrasse"] = 1
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_4"].str.contains("errass", na=False)].index.tolist()
df.loc[indices, "Terrasse"] = 1
df.loc[indices, "Feature_4"] = np.nan

indices = df[df["Feature_5"].str.contains("errass", na=False)].index.tolist()
df.loc[indices, "Terrasse"] = 1
df.loc[indices, "Feature_5"] = np.nan

indices = df[df["Feature_6"].str.contains("errass", na=False)].index.tolist()
df.loc[indices, "Terrasse"] = 1
df.loc[indices, "Feature_6"] = np.nan

## New column "pets_allowed"

I followed the same process and steps used for populating the 'lim_inhabitants', 'balcony', 'furnished', 'garden', and 'Terrasse' columns to extract relevant information for this new column.

In [ ]:
df['Feature_2'].value_counts(dropna=False)

,count
Feature_2,
NaN,2172
Personenaufzug,578
"Personenaufzug, Lastenaufzug",83
DSL-Anschluss,49
neuwertig,37
Haustiere erlaubt,21
Kelleranteil,19
Einbauküche,9
Bad mit Fenster,6


In [ ]:
df["pets_allowed"] = np.nan

indices = df[df["Feature_2"].str.contains("tier", na=False)].index.tolist()
df.loc[indices, "pets_allowed"] = 1
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_3"].str.contains("tier", na=False)].index.tolist()
df.loc[indices, "pets_allowed"] = 1
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_4"].str.contains("tier", na=False)].index.tolist()
df.loc[indices, "pets_allowed"] = 1
df.loc[indices, "Feature_4"] = np.nan

indices = df[df["Feature_5"].str.contains("tier", na=False)].index.tolist()
df.loc[indices, "pets_allowed"] = 1
df.loc[indices, "Feature_5"] = np.nan

indices = df[df["Feature_6"].str.contains("tier", na=False)].index.tolist()
df.loc[indices, "pets_allowed"] = 1
df.loc[indices, "Feature_6"] = np.nan

## New column "Garage"

This column indicates whether a listing includes a garage or parking space. The identification was based on the presence of the following substrings:
- 'arage' (to capture 'Garage')
-  'ellplat' (to capture 'Stellplatz')
-  'Parkhaus'
-  'tellpl' (alternative form of 'Stellplatz')
-  'Parkb'

In [ ]:
df['Feature_2'].value_counts(dropna=False)

,count
Feature_2,
NaN,2193
Personenaufzug,578
"Personenaufzug, Lastenaufzug",83
DSL-Anschluss,49
neuwertig,37
Kelleranteil,19
Einbauküche,9
Bad mit Fenster,6
Bad mit Dusche,5


In [ ]:
df["Garage"] = np.nan

indices = df[df["Feature_2"].str.contains("arage", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_2"].str.contains("ellplat", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_3"].str.contains("arage", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("ellplat", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("ellplät", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_3"].str.contains("Parkhaus", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_4"].str.contains("arage", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_4"] = np.nan

indices = df[df["Feature_4"].str.contains("ellplat", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_4"] = np.nan

indices = df[df["Feature_4"].str.contains("Parkh", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_4"] = np.nan

indices = df[df["Feature_4"].str.contains("Parkb", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_4"] = np.nan

indices = df[df["Feature_5"].str.contains("tellpl", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_5"] = np.nan

indices = df[df["Feature_5"].str.contains("arag", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_5"] = np.nan

indices = df[df["Feature_5"].str.contains("Parkb", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_5"] = np.nan

indices = df[df["Feature_5"].str.contains("Parkh", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_5"] = np.nan

indices = df[df["Feature_6"].str.contains("arag", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_6"] = np.nan

indices = df[df["Feature_6"].str.contains("tellpl", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_6"] = np.nan

indices = df[df["Feature_6"].str.contains("Parkhaus", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_6"] = np.nan

indices = df[df["Feature_7"].str.contains("tellpl", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_7"] = np.nan

indices = df[df["Feature_7"].str.contains("arag", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_7"] = np.nan

indices = df[df["Feature_8"].str.contains("arag", na=False)].index.tolist()
df.loc[indices, "Garage"] = 1
df.loc[indices, "Feature_8"] = np.nan

## New columns "floor covering", "shower", "bath", "Einbaukueche", "kitchenette", "lift", "open kitchen", "basement" and "window in bathroom"

I created these new columns based on the remaining unprocessed distinct values in 'Feature_1' to 'Feature_8', identified using .value_counts(dropna=False).

The steps taken here were similar to those used for creating and populating the columns:
- 'lim_inhabitants'
- 'balcony'
- 'furnished'
- 'garden'
- 'Terrasse'
- 'pets_allowed'
- 'garage'
This ensured that all relevant information was properly categorized and stored in dedicated columns.

In [ ]:
df['Feature_1'].value_counts(dropna=False)

,count
Feature_1,
"Einbauküche, Kochnische",1219
Einbauküche,846
"Einbauküche, Offene Küche",405
Personenaufzug,234
NaN,144
neuwertig,48
Offene Küche,26
Kein Personenaufzug,21
vermietet,19


In [ ]:
df["floor covering"] = np.nan
df["shower"] = np.nan
df["bath"] = np.nan
df["Einbaukueche"] = np.nan
df["kitchenette"] = np.nan
df["lift"] = np.nan
df["open kitchen"] = np.nan
df["basement"] = np.nan
df["window in bathroom"] = np.nan

indices = df[df["Feature_1"].str.contains("Einbauküche, Kochnische, Offene Küche, Speisekammer", na=False)].index.tolist()
df.loc[indices, ['Einbaukueche', 'kitchenette', 'open kitchen', "Feature_1"]] = [1, 1, 1, np.nan]

indices = df[df["Feature_1"].str.contains("Einbauküche, Kochnische, Speisekammer", na=False)].index.tolist()
df.loc[indices, ['Einbaukueche', 'kitchenette', "Feature_1"]] = [1, 1, np.nan]

indices = df[df["Feature_1"].str.contains("Einbauküche, Offene Küche, Speisekammer", na=False)].index.tolist()
df.loc[indices, ['Einbaukueche', 'open kitchen', "Feature_1"]] = [1, 1, np.nan]

indices = df[df["Feature_1"].str.contains("Einbauküche, Offene Küche", na=False)].index.tolist()
df.loc[indices, ['Einbaukueche', 'open kitchen', "Feature_1"]] = [1, 1, np.nan]

indices = df[df["Feature_1"].str.contains("Einbauküche, Kochnische", na=False)].index.tolist()
df.loc[indices, ['Einbaukueche', 'kitchenette', "Feature_1"]] = [1, 1, np.nan]

indices = df[df["Feature_1"].str.contains("Kochnische, Speisekammer", na=False)].index.tolist()
df.loc[indices, ['kitchenette', "Feature_1"]] = [1, np.nan]

indices = df[df["Feature_1"].str.contains("Kochnische", na=False)].index.tolist()
df.loc[indices, ['kitchenette', "Feature_1"]] = [1, np.nan]

indices = df[df["Feature_1"].str.contains("Offene Küche", na=False)].index.tolist()
df.loc[indices, ['open kitchen', "Feature_1"]] = [1, np.nan]

indices = df[df["Feature_1"].str.contains("Einbauküche", na=False)].index.tolist()
df.loc[indices, ['Einbaukueche', "Feature_1"]] = [1, np.nan]

indices = df[df["Feature_2"].str.contains("Einbauküche, Offene Küche", na=False)].index.tolist()
df.loc[indices, ['Einbaukueche', 'open kitchen', "Feature_2"]] = [1, 1, np.nan]

indices = df[df["Feature_2"].str.contains("Einbauküche, Speisekammer", na=False)].index.tolist()
df.loc[indices, ['Einbaukueche', "Feature_2"]] = [1, np.nan]

indices = df[df["Feature_2"].str.contains("Einbauküche", na=False)].index.tolist()
df.loc[indices, ['Einbaukueche', "Feature_2"]] = [1, np.nan]

indices = df[df["Feature_2"].str.contains("Offene Küche", na=False)].index.tolist()
df.loc[indices, ['open kitchen', "Feature_2"]] = [1, np.nan]

indices = df[df["Feature_2"].str.contains("Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ['window in bathroom', "Feature_2"]] = [1, np.nan]

indices = df[df["Feature_2"].str.contains("Bad mit Dusche", na=False)].index.tolist()
df.loc[indices, ['shower', "Feature_2"]] = [1, np.nan]

indices = df[df["Feature_3"].str.contains("Badezimmer: Badewanne, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["bath", 'window in bathroom', "Feature_3"]] = [1, 1, np.nan]

indices = df[df["Feature_3"].str.contains("Badezimmer: Badewanne, Bad mit Dusche, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["bath", "shower", 'window in bathroom', "Feature_3"]] = [1, 1, 1, np.nan]

indices = df[df["Feature_3"].str.contains("Badezimmer: Bad mit Dusche, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["shower", 'window in bathroom', "Feature_3"]] = [1, 1, np.nan]

indices = df[df["Feature_3"].str.contains("Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ['window in bathroom', "Feature_3"]] = [1, np.nan]

indices = df[df["Feature_3"].str.contains("Bad mit Dusche", na=False)].index.tolist()
df.loc[indices, ['shower', "Feature_3"]] = [1, np.nan]

indices = df[df["Feature_3"].str.contains("Badewanne", na=False)].index.tolist()
df.loc[indices, ['bath', "Feature_3"]] = [1, np.nan]

indices = df[df["Feature_4"].str.contains("Badezimmer: Badewanne, Bad mit Dusche, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["bath", "shower", 'window in bathroom', "Feature_4"]] = [1, 1, 1, np.nan]

indices = df[df["Feature_4"].str.contains("Badezimmer: Bad mit Dusche, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["shower", 'window in bathroom', "Feature_4"]] = [1, 1, np.nan]

indices = df[df["Feature_4"].str.contains("Badezimmer: Badewanne, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["bath", 'window in bathroom', "Feature_4"]] = [1, 1, np.nan]

indices = df[df["Feature_4"].str.contains("Badezimmer: Badewanne, Bad mit Dusche", na=False)].index.tolist()
df.loc[indices, ["bath", 'shower', "Feature_4"]] = [1, 1, np.nan]

indices = df[df["Feature_4"].str.contains("Badewanne", na=False)].index.tolist()
df.loc[indices, ['bath', "Feature_4"]] = [1, np.nan]

indices = df[df["Feature_4"].str.contains("Bad mit Dusche", na=False)].index.tolist()
df.loc[indices, ['shower', "Feature_4"]] = [1, np.nan]

indices = df[df["Feature_5"].str.contains("Badezimmer: Badewanne, Bad mit Dusche, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["bath", "shower", 'window in bathroom', "Feature_5"]] = [1, 1, 1, np.nan]

indices = df[df["Feature_5"].str.contains("Badezimmer: Badewanne, Bad mit Dusche", na=False)].index.tolist()
df.loc[indices, ["bath", "shower", "Feature_5"]] = [1, 1, np.nan]

indices = df[df["Feature_5"].str.contains("Badezimmer: Bad mit Dusche, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["shower", 'window in bathroom', "Feature_5"]] = [1, 1, np.nan]

indices = df[df["Feature_5"].str.contains("Badezimmer: Badewanne, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["bath", 'window in bathroom', "Feature_5"]] = [1, 1, np.nan]

indices = df[df["Feature_5"].str.contains("Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ['window in bathroom', "Feature_5"]] = [1, np.nan]

indices = df[df["Feature_5"].str.contains("Badewanne", na=False)].index.tolist()
df.loc[indices, ['bath', "Feature_5"]] = [1, np.nan]

indices = df[df["Feature_5"].str.contains("Bad mit Dusche", na=False)].index.tolist()
df.loc[indices, ['shower', "Feature_5"]] = [1, np.nan]

indices = df[df["Feature_6"].str.contains("Badezimmer: Badewanne, Bad mit Dusche, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["bath", "shower", 'window in bathroom', "Feature_6"]] = [1, 1, 1, np.nan]

indices = df[df["Feature_6"].str.contains("Badezimmer: Badewanne, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["bath", 'window in bathroom', "Feature_6"]] = [1, 1, np.nan]

indices = df[df["Feature_6"].str.contains("Badezimmer: Bad mit Dusche, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["shower", 'window in bathroom', "Feature_6"]] = [1, 1, np.nan]

indices = df[df["Feature_6"].str.contains("Badezimmer: Badewanne, Bad mit Dusche", na=False)].index.tolist()
df.loc[indices, ["bath", 'shower', "Feature_6"]] = [1, 1, np.nan]

indices = df[df["Feature_6"].str.contains("Badewanne", na=False)].index.tolist()
df.loc[indices, ["bath", "Feature_6"]] = [1, np.nan]

indices = df[df["Feature_6"].str.contains("Bad mit Dusche", na=False)].index.tolist()
df.loc[indices, ["shower", "Feature_6"]] = [1, np.nan]

indices = df[df["Feature_7"].str.contains("Badezimmer: Badewanne, Bad mit Dusche, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["bath", "shower", 'window in bathroom', "Feature_7"]] = [1, 1, 1, np.nan]

indices = df[df["Feature_7"].str.contains("Badezimmer: Bad mit Dusche, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["shower", 'window in bathroom', "Feature_7"]] = [1, 1, np.nan]

indices = df[df["Feature_7"].str.contains("Badezimmer: Badewanne, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["bath", 'window in bathroom', "Feature_7"]] = [1, 1, np.nan]

indices = df[df["Feature_7"].str.contains("Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ['window in bathroom', "Feature_7"]] = [1, np.nan]

indices = df[df["Feature_7"].str.contains("Badewanne, Bad mit Dusche", na=False)].index.tolist()
df.loc[indices, ["bath", "shower", "Feature_7"]] = [1, 1, np.nan]

indices = df[df["Feature_7"].str.contains("Badewanne", na=False)].index.tolist()
df.loc[indices, ['bath', "Feature_7"]] = [1, np.nan]

indices = df[df["Feature_7"].str.contains("Bad mit Dusche", na=False)].index.tolist()
df.loc[indices, ['shower', "Feature_7"]] = [1, np.nan]

indices = df[df["Feature_8"].str.contains("Badezimmer: Badewanne, Bad mit Dusche, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["bath", "shower", 'window in bathroom', "Feature_8"]] = [1, 1, 1, np.nan]

indices = df[df["Feature_8"].str.contains("Badezimmer: Badewanne, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["bath", 'window in bathroom', "Feature_8"]] = [1, 1, np.nan]

indices = df[df["Feature_8"].str.contains("Badezimmer: Bad mit Dusche, Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ["shower", 'window in bathroom', "Feature_8"]] = [1, 1, np.nan]

indices = df[df["Feature_8"].str.contains("Bad mit Fenster", na=False)].index.tolist()
df.loc[indices, ['window in bathroom', "Feature_8"]] = [1, np.nan]

indices = df[df["Feature_8"].str.contains("Badezimmer: Badewanne, Bad mit Dusche", na=False)].index.tolist()
df.loc[indices, ["bath", 'shower', "Feature_8"]] = [1, 1, np.nan]

indices = df[df["Feature_8"].str.contains("Badewanne", na=False)].index.tolist()
df.loc[indices, ['bath', "Feature_8"]] = [1, np.nan]

indices = df[df["Feature_8"].str.contains("Bad mit Dusche", na=False)].index.tolist()
df.loc[indices, ['shower', "Feature_8"]] = [1, np.nan]

indices = df[df["Feature_2"].str.contains("odenbel", na=False)].index.tolist()
df.loc[indices, "floor covering"] = df.loc[indices, "Feature_2"]
df.loc[indices, "Feature_2"] = np.nan

indices = df[df["Feature_3"].str.contains("odenbel", na=False)].index.tolist()
df.loc[indices, "floor covering"] = df.loc[indices, "Feature_3"]
df.loc[indices, "Feature_3"] = np.nan

indices = df[df["Feature_4"].str.contains("odenbel", na=False)].index.tolist()
df.loc[indices, "floor covering"] = df.loc[indices, "Feature_4"]
df.loc[indices, "Feature_4"] = np.nan

indices = df[df["Feature_5"].str.contains("odenbel", na=False)].index.tolist()
df.loc[indices, "floor covering"] = df.loc[indices, "Feature_5"]
df.loc[indices, "Feature_5"] = np.nan

indices = df[df["Feature_6"].str.contains("odenbel", na=False)].index.tolist()
df.loc[indices, "floor covering"] = df.loc[indices, "Feature_6"]
df.loc[indices, "Feature_6"] = np.nan

indices = df[df["Feature_7"].str.contains("odenbel", na=False)].index.tolist()
df.loc[indices, "floor covering"] = df.loc[indices, "Feature_7"]
df.loc[indices, "Feature_7"] = np.nan

indices = df[df["Feature_8"].str.contains("odenbel", na=False)].index.tolist()
df.loc[indices, "floor covering"] = df.loc[indices, "Feature_8"]
df.loc[indices, "Feature_8"] = np.nan

indices = df[df["Feature_1"].str.contains("ellerantei", na=False)].index.tolist()
df.loc[indices, ['basement', "Feature_1"]] = [1, np.nan]

indices = df[df["Feature_2"].str.contains("ellerantei", na=False)].index.tolist()
df.loc[indices, ['basement', "Feature_2"]] = [1, np.nan]

indices = df[df["Feature_3"].str.contains("ellerantei", na=False)].index.tolist()
df.loc[indices, ['basement', "Feature_3"]] = [1, np.nan]

indices = df[df["Feature_4"].str.contains("ellerantei", na=False)].index.tolist()
df.loc[indices, ['basement', "Feature_4"]] = [1, np.nan]

indices = df[df["Feature_5"].str.contains("ellerantei", na=False)].index.tolist()
df.loc[indices, ['basement', "Feature_5"]] = [1, np.nan]

indices = df[df["Feature_6"].str.contains("ellerantei", na=False)].index.tolist()
df.loc[indices, ['basement', "Feature_6"]] = [1, np.nan]

indices = df[df["Feature_7"].str.contains("ellerantei", na=False)].index.tolist()
df.loc[indices, ['basement', "Feature_7"]] = [1, np.nan]

indices = df[df["Feature_8"].str.contains("ellerantei", na=False)].index.tolist()
df.loc[indices, ['basement', "Feature_8"]] = [1, np.nan]

indices = df[df["Feature_1"].str.contains("Kein Personenaufzug", na=False)].index.tolist()
df.loc[indices, "Feature_1"] = np.nan

indices = df[df["Feature_1"].str.contains("Personenaufzug", na=False)].index.tolist()
df.loc[indices, ['lift', "Feature_1"]] = [1, np.nan]

indices = df[df["Feature_2"].str.contains("Personenaufzug", na=False)].index.tolist()
df.loc[indices, ['lift', "Feature_2"]] = [1, np.nan]

indices = df[df["Feature_3"].str.contains("Personenaufzug", na=False)].index.tolist()
df.loc[indices, ['lift', "Feature_3"]] = [1, np.nan]

indices = df[df["Feature_2"].str.contains("oll unterkeller", na=False)].index.tolist()
df.loc[indices, ['basement', "Feature_2"]] = [1, np.nan]

indices = df[df["Feature_3"].str.contains("oll unterkeller", na=False)].index.tolist()
df.loc[indices, ['basement', "Feature_3"]] = [1, np.nan]

indices = df[df["Feature_4"].str.contains("oll unterkeller", na=False)].index.tolist()
df.loc[indices, ['basement', "Feature_4"]] = [1, np.nan]

indices = df[df["Feature_5"].str.contains("oll unterkeller", na=False)].index.tolist()
df.loc[indices, ['basement', "Feature_5"]] = [1, np.nan]

indices = df[df["Feature_6"].str.contains("oll unterkeller", na=False)].index.tolist()
df.loc[indices, ['basement', "Feature_6"]] = [1, np.nan]

indices = df[df["Feature_7"].str.contains("oll unterkeller", na=False)].index.tolist()
df.loc[indices, ['basement', "Feature_7"]] = [1, np.nan]

<ipython-input-104-1e580521c599>:174: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Bodenbelag: Holzdielen' 'Bodenbelag: Parkett']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[indices, "floor covering"] = df.loc[indices, "Feature_2"]


In [ ]:
df['Feature_1'].value_counts(dropna=False)

,count
Feature_1,
NaN,2925
neuwertig,48
vermietet,19
DSL-Anschluss,7
Erstbezug,5
0,1
Speisekammer,1


# "Condition" column simplification and cleaning (1)

The 'Condition' column contained several values that actually belonged in the 'Year of Construction' column.

In [ ]:
indices = df[df["Condition"].str.contains("Altbau", na=False)].index.tolist()
df.loc[indices, ['Year of Construction', 'Condition']]

,Year of Construction,Condition
41,1907.0,"Altbau (bis 1945), renoviert / saniert"
43,1901.0,Altbau (bis 1945)
227,NaN,Altbau (bis 1945)
484,1880.0,"Altbau (bis 1945), renoviert / saniert"
500,1905.0,"Altbau (bis 1945), renoviert / saniert"
...,...,...
3671,1900.0,"Altbau (bis 1945), renoviert / saniert"
3680,1912.0,"Altbau (bis 1945), renoviert / saniert"
3688,1905.0,"Altbau (bis 1945), renoviert / saniert"
3690,1912.0,"Altbau (bis 1945), Erstbezug"


In [ ]:
indices = df[df["Condition"].str.contains("Altbau", na=False)].index.tolist()
df.loc[indices, ['Year of Construction']] = "Altbau"

indices = df[df["Condition"].str.contains("Neubau", na=False)].index.tolist()
df.loc[indices, ['Year of Construction']] = "Neubau"

df.loc[967, ['Year of Construction']] = np.nan

<ipython-input-107-c3fec925785c>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Altbau' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[indices, ['Year of Construction']] = "Altbau"


Since the 'Year of Construction' column contained imprecise and broad-ranging values, I decided to simplify it using the following criteria:
- Assigning 'Altbau' (old building):
  - If the 'Condition' column mentioned 'Altbau'.
  - If 'Year of Construction' was ≤ 1945.
- Assigning 'Neubau' (new building):
  - If the 'Condition' column mentioned 'Neubau'.
  - If 'Year of Construction' was > 1945.

This approach ensures a clearer classification between older and newer buildings.

In [ ]:
# Convert only numeric values while keeping existing "Altbau", "Neubau", and NaN untouched
def categorize_year(value):
    if isinstance(value, (int, float)):  # Ensure we're working with numbers
        if value <= 1945:
            return "Altbau"
        elif value > 1945:
            return "Neubau"
    return value  # Keep existing "Altbau", "Neubau", NaN, or other non-numeric values unchanged

# Apply function to the column
df['Year of Construction'] = df['Year of Construction'].apply(categorize_year)

I cleaned and standardized property condition descriptions by mapping complex condition labels to simpler, more uniform terms.

In [ ]:
df['Condition'].value_counts(dropna=False)

,count
Condition,
NaN,2097
neuwertig,271
renoviert / saniert,178
Erstbezug,129
Gepflegt,96
"Altbau (bis 1945), renoviert / saniert",81
Neubau,47
"Neubau, Erstbezug",29
"Neubau, neuwertig",23


In [ ]:
# Define the mapping dictionary
condition_mapping = {
    "Altbau (bis 1945), renoviert / saniert": "renoviert / saniert",
    "Neubau, Erstbezug": "Erstbezug",
    "Neubau, neuwertig": "neuwertig",
    "Altbau (bis 1945)": np.nan,
    "Neubau": np.nan,
    "renovierungsbedürftig / sanierungsbedürftig": np.nan,
    "Neubau, renoviert / saniert": "renoviert / saniert",
    "Massivhaus, Erstbezug": "Erstbezug",
    "Altbau (bis 1945), Erstbezug": "Erstbezug",
    "Neubau, Gepflegt": "Gepflegt",
    "Massivhaus, neuwertig": "neuwertig",
    "Massivhaus, Gepflegt": "Gepflegt",
    "Altbau (bis 1945), neuwertig": "neuwertig",
    "Neubau, Massivhaus, Erstbezug": "Erstbezug",
    "Altbau (bis 1945), Gepflegt": "Gepflegt",
    "Massivhaus, renoviert / saniert": "renoviert / saniert"
}

# Apply the mapping to the column
df['Condition'] = df['Condition'].replace(condition_mapping)

I also extracted remaining condition-related information from the 'Feature_*' columns and moved it to the 'Condition' column.

Finally, after extracting all relevant information, I cleaned up the 'Feature_*' columns to remove any redundant or unnecessary data.

In [ ]:
df[df["Feature_1"].str.contains("neuwertig", na=False)].index.tolist()
df.loc[indices, ["Feature_1", 'Condition']]

,Feature_1,Condition
2,NaN,renoviert / saniert
3,NaN,NaN
11,NaN,neuwertig
13,NaN,Erstbezug
17,NaN,neuwertig
...,...,...
3557,NaN,Erstbezug
3558,DSL-Anschluss,neuwertig
3560,NaN,Erstbezug
3759,NaN,renoviert / saniert


In [ ]:
indices = [216, 749, 869, 884, 915, 940, 957, 1120, 1992, 2020, 2049, 2053, 2063, 2104, 3485, 3500, 3555]
df.loc[indices, ['Condition', "Feature_1"]] = ["neuwertig", np.nan]

indices = df[df["Feature_1"].str.contains("neuwertig", na=False)].index.tolist()
df.loc[indices, ["Feature_1"]] = [np.nan]

indices = df[df["Feature_1"].str.contains("Erstbezug", na=False)].index.tolist()
df.loc[indices, ['Condition', "Feature_1"]] = ["Erstbezug", np.nan]

indices = df[df["Feature_2"].str.contains("Erstbezug", na=False)].index.tolist()
df.loc[indices, ['Condition', "Feature_2"]] = ["Erstbezug", np.nan]

indices = [134, 852, 865, 889, 893, 906, 921, 935, 1073, 1087, 1121, 1128, 1426, 1988, 2002, 2011, 2013, 2065, 2081, 3474, 3567]
df.loc[indices, ['Condition', "Feature_2"]] = ["neuwertig", np.nan]

indices = df[df["Feature_2"].str.contains("neuwertig", na=False)].index.tolist()
df.loc[indices, ["Feature_2"]] = [np.nan]

indices = df[df["Feature_3"].str.contains("neuwertig", na=False)].index.tolist()
if 2 in indices:
    indices.remove(2)
if 3458 in indices:
    indices.remove(3458)
df.loc[indices, ['Condition', "Feature_3"]] = ["neuwertig", np.nan]

indices = df[df["Feature_3"].str.contains("neuwertig", na=False)].index.tolist()
df.loc[indices, ["Feature_3"]] = [np.nan]

indices = df[df["Feature_3"].str.contains("Erstbezug", na=False)].index.tolist()
df.loc[indices, ["Feature_3"]] = [np.nan]

indices = df[df["Feature_3"].str.contains("renoviert", na=False)].index.tolist()
df.loc[indices, ['Condition', "Feature_3"]] = ["renoviert / saniert", np.nan]

# Features cleaning and "time_lim" and "Garage" population

I extracted information about short-term rentals ('kurzfristig') and carports ('arpor') from various 'Feature_*' columns and moved them to their respective dedicated columns:
- 'time_lim' for short-term rentals.
- 'garage' for parking facility.

This ensured that relevant details were properly categorized and stored in structured columns.


In [ ]:
indices = df[df["Feature_2"].str.contains("kurzfristig", na=False)].index.tolist()
df.loc[indices, ["time_lim", "Feature_2"]] = ["yes", np.nan]

indices = df[df["Feature_2"].str.contains("KURZFRISTIG", na=False)].index.tolist()
df.loc[indices, ["time_lim", "Feature_2"]] = ["yes", np.nan]

indices = df[df["Feature_3"].str.contains("kurzfristig", na=False)].index.tolist()
df.loc[indices, ["time_lim", "Feature_3"]] = ["yes", np.nan]

indices = df[df["Feature_4"].str.contains("arpor", na=False)].index.tolist()
df.loc[indices, ["Garage", "Feature_4"]] = [1, np.nan]

indices = df[df["Feature_5"].str.contains("arpor", na=False)].index.tolist()
df.loc[indices, ["Garage", "Feature_5"]] = [1, np.nan]

# New column "WG-geeignet"

I extracted information about whether an apartment is suitable for shared living ('WG-geeignet') from various 'Feature_*' columns and moved it to a new dedicated column called 'WG-geeignet'.

In [ ]:
df['Feature_4'].value_counts(dropna=False)

,count
Feature_4,
NaN,2990
Fernblick,3
Barrierefrei,2
Seeblick,2
Fenster: Kunststoff,2
Nicht barrierefrei,2
WG-geeignet,1
"Barrierefrei, Rollstuhlgerecht",1
Waschraum,1


In [ ]:
df["WG-geeignet"] = np.nan

indices = df[df["Feature_4"].str.contains("eeigne", na=False)].index.tolist()
df.loc[indices, ["WG-geeignet", "Feature_4"]] = [1, np.nan]

indices = df[df["Feature_5"].str.contains("eeigne", na=False)].index.tolist()
df.loc[indices, ["WG-geeignet", "Feature_5"]] = [1, np.nan]

indices = df[df["Feature_6"].str.contains("Nicht für WG-geeignet", na=False)].index.tolist()
df.loc[indices, ["Feature_6"]] = [np.nan]

indices = df[df["Feature_6"].str.contains("WG-geeignet", na=False)].index.tolist()
df.loc[indices, ["WG-geeignet", "Feature_6"]] = [1, np.nan]

indices = df[df["Feature_7"].str.contains("Nicht für WG-geeignet", na=False)].index.tolist()
df.loc[indices, ["Feature_7"]] = [np.nan]

indices = df[df["Feature_7"].str.contains("WG-geeignet", na=False)].index.tolist()
df.loc[indices, ["WG-geeignet", "Feature_7"]] = [1, np.nan]

indices = df[df["Feature_8"].str.contains("Nicht für WG-geeignet", na=False)].index.tolist()
df.loc[indices, ["Feature_8"]] = [np.nan]

indices = df[df["Feature_8"].str.contains("WG-geeignet", na=False)].index.tolist()
df.loc[indices, ["WG-geeignet", "Feature_8"]] = [1, np.nan]

In [ ]:
df['Feature_4'].value_counts(dropna=False)

,count
Feature_4,
NaN,2991
Fernblick,3
Barrierefrei,2
Seeblick,2
Fenster: Kunststoff,2
Nicht barrierefrei,2
"Barrierefrei, Rollstuhlgerecht",1
Waschraum,1
Badezimmer,1


# Cleaning the "Title" column

After visually inspecting the 'Title' column, I decided to simplify and group its values into fewer categories by creating additional columns using the one-hot encoding method.

## New column "Wohnen auf Zeit in title"

I noticed that certain listing titles contained the phrase 'Wohnen auf Zeit', such as:
- Wohnung zur Miete · Wohnen auf Zeit
- Studio zur Miete · Wohnen auf Zeit
- WG-Zimmer zur Miete · Wohnen auf Zeit


Instead of keeping this phrase in the title, I extracted it and created a new column called 'Wohnen auf Zeit in title', where a value of 1 indicates that the listing falls under this category.

This approach allows for better categorization and analysis of rental types.



In [ ]:
df["Title"].value_counts(dropna=False)

,count
Title,
Wohnung zur Miete · Wohnen auf Zeit,1713
Wohnung zur Miete,674
Studio zur Miete,462
Maisonette zur Miete,37
Studio zur Miete · Wohnen auf Zeit,28
Penthouse zur Miete,25
WG-Zimmer zur Miete,24
Terrassenwohnung zur Miete,18
Wohnung zur Miete · nur mit Wohnberechtigungsschein,12


In [ ]:
df["Wohnen auf Zeit in title"] = np.nan

indices = df[df["Title"].str.contains("Wohnen auf Zeit", na=False)].index.tolist()
df.loc[indices, ["Wohnen auf Zeit in title"]] = 1

# Define the mapping dictionary
title_mapping = {
    "Wohnung zur Miete\xa0·\xa0Wohnen auf Zeit": "Wohnung zur Miete",
    "Studio zur Miete\xa0·\xa0Wohnen auf Zeit": "Studio zur Miete",
    "WG-Zimmer zur Miete\xa0·\xa0Wohnen auf Zeit": "WG-Zimmer zur Miete"
}

# Apply the mapping to the column
df['Title'] = df['Title'].replace(title_mapping)

In [ ]:
df["Title"].value_counts(dropna=False)

,count
Title,
Wohnung zur Miete,2387
Studio zur Miete,490
Maisonette zur Miete,37
WG-Zimmer zur Miete,30
Penthouse zur Miete,25
Terrassenwohnung zur Miete,18
Wohnung zur Miete · nur mit Wohnberechtigungsschein,12
Loft zur Miete,6
Studio zur Miete · nur mit Wohnberechtigungsschein,1


## New column "nur mit Wohnberechtigungsschein in title"

In this step, I:
- Flagged listings that require a 'Wohnberechtigungsschein' (WBS) (a public housing eligibility certificate) by creating a new indicator column.
- Standardized the 'Title' column by cleaning and simplifying its values for consistency and better categorization.

This ensures that listings requiring special eligibility are easily identifiable while improving the clarity of the title data.



In [ ]:
df["Title"].value_counts(dropna=False)

,count
Title,
Wohnung zur Miete,2387
Studio zur Miete,490
Maisonette zur Miete,37
WG-Zimmer zur Miete,30
Penthouse zur Miete,25
Terrassenwohnung zur Miete,18
Wohnung zur Miete · nur mit Wohnberechtigungsschein,12
Loft zur Miete,6
Studio zur Miete · nur mit Wohnberechtigungsschein,1


In [ ]:
df["nur mit Wohnberechtigungsschein in title"] = np.nan

indices = df[df["Title"].str.contains("nur mit Wohnberechtigungsschein", na=False)].index.tolist()
df.loc[indices, ["nur mit Wohnberechtigungsschein in title"]] = 1

# Define the mapping dictionary
title_mapping = {
    "Wohnung zur Miete\xa0·\xa0nur mit Wohnberechtigungsschein": "Wohnung zur Miete",
    "Studio zur Miete\xa0·\xa0nur mit Wohnberechtigungsschein": "Studio zur Miete"
}

# Apply the mapping to the column
df['Title'] = df['Title'].replace(title_mapping)

In [ ]:
df["Title"].value_counts(dropna=False)

,count
Title,
Wohnung zur Miete,2399
Studio zur Miete,491
Maisonette zur Miete,37
WG-Zimmer zur Miete,30
Penthouse zur Miete,25
Terrassenwohnung zur Miete,18
Loft zur Miete,6


## Retrieving WG information from listings titles

I flagged listings that mention 'WG' (Wohngemeinschaft, meaning shared apartments) in the 'Title' column and assigned a value of 1 to the 'WG-geeignet' column to indicate that the property is suitable for shared living.


In [ ]:
indices = df[df["Title"].str.contains("WG", na=False)].index.tolist()
df.loc[indices, ['WG-geeignet']] = 1

# Cleaning the "availability" column

I cleaned and standardized the 'availability' column by:
- Correcting incorrectly formatted availability dates.
- Removing irrelevant or incorrect values.
- Standardizing the format of specific availability dates for consistency.

This ensures that the availability data is clean, accurate, and uniformly formatted for better analysis.


In [ ]:
indices = df[df["availability"].str.contains("ofor", na=False)].index.tolist()
df.loc[indices, ["availability"]] = "ab sofort"

indices = df[df["availability"].str.contains("Baujahr", na=False)].index.tolist()
df.loc[indices, ["availability"]] = np.nan

indices = df[df["availability"].str.contains("Zustand der Immobilie", na=False)].index.tolist()
df.loc[indices, ["availability"]] = np.nan

indices = df[df["availability"].str.contains("Fahrtzeitberechnung", na=False)].index.tolist()
df.loc[indices, ["availability"]] = np.nan

indices = df[df["availability"].str.contains("Wesentliche Energieträger", na=False)].index.tolist()
df.loc[indices, ["availability"]] = np.nan

indices = df[df["availability"].str.contains("Heizungsart", na=False)].index.tolist()
df.loc[indices, ["availability"]] = np.nan

indices = df[df["availability"].str.contains("ca. März 23", na=False)].index.tolist()
df.loc[indices, ["availability"]] = "01.03.2025"

indices = df[df["availability"].str.contains("voraussichtlich 15.12.2024", na=False)].index.tolist()
df.loc[indices, ["availability"]] = "ab sofort"

indices = df[df["availability"].str.contains("voraussichtlich 01.02.2025", na=False)].index.tolist()
df.loc[indices, ["availability"]] = "ab sofort"

indices = df[df["availability"].str.contains("01.03.25", na=False)].index.tolist()
df.loc[indices, ["availability"]] = "01.03.2025"

indices = df[df["availability"].str.contains("Ab 01.04.2025", na=False)].index.tolist()
df.loc[indices, ["availability"]] = "01.04.2025"

indices = df[df["availability"].str.contains('ab 01.02.2025', na=False)].index.tolist()
df.loc[indices, ["availability"]] = "ab sofort"

df.loc[2186, ["availability"]] = "01.04.2025"

indices = df[df["availability"].str.contains("2027", na=False)].index.tolist()
df.loc[indices, ["availability"]] = "01.01.2027"

indices = df[df["availability"].str.contains("GmbH & Co", na=False)].index.tolist()
df.loc[indices, ["availability"]] = np.nan

indices = df[df["availability"].str.contains("€", na=False)].index.tolist()
df.loc[indices, ["availability"]] = np.nan

## Transforming the "availability" column values to a date datatype

I converted the 'availability' column into a standardized datetime format and calculated the number of days until availability from a reference date (12.02.2025, the web scraping date).

Steps Taken:
- Converted 'availability' into a clean datetime format ('availability_date').
- Handled different date formats:
  - 'DD.MM.YYYY' → Directly converted.
  - 'MM-YYYY' → Set to the 1st day of the corresponding month.
  - 'ab sofort' (available immediately) → Set to 12.02.2025.
- Ensured that no availability date was before 12.02.2025 to maintain data consistency.
- Calculated 'days_until_available', which represents the difference between the availability date and 12.02.2025.
- Printed unrecognized date formats for manual inspection and debugging, ensuring all values were correctly processed.


In [ ]:
# Reference date (e.g., today or a specific date like "12.02.2025")
reference_date_str = "12.02.2025"
reference_date = datetime.strptime(reference_date_str, "%d.%m.%Y")

# List to store errors
error_values = []

# Function to transform dates
def transform_availability(value):
    if pd.isna(value):  # Ignore NaN values
        return value  # Leave NaN values untouched

    value = str(value).strip()  # Convert to string and remove whitespace

    if value == "ab sofort":
        return reference_date  # Treat "ab sofort" as the reference date

    # Try parsing standard date format "DD.MM.YYYY"
    try:
        parsed_date = datetime.strptime(value, "%d.%m.%Y")
        return max(parsed_date, reference_date)  # Ensure it's not before reference date
    except ValueError:
        pass

    # Handle cases like "01-2025" (assuming first day of the month)
    try:
        parsed_date = datetime.strptime(value, "%m-%Y")
        parsed_date = parsed_date.replace(day=1)  # Set to first day of the month
        return max(parsed_date, reference_date)  # Ensure it's not before reference date
    except ValueError:
        pass

    # If value is unrecognized, store it in the error list
    error_values.append(value)
    return None  # Return None for unrecognized formats

# Apply the transformation
df["availability_date"] = df["availability"].apply(transform_availability)

# Convert the new column to proper datetime format
df["availability_date"] = pd.to_datetime(df["availability_date"])

# Calculate the number of days difference from the reference date
df["days_until_available"] = (df["availability_date"] - reference_date).dt.days

# Show errors if any
if error_values:
    print("Unrecognized date formats detected:", error_values)

# Cleaning the "Condition" column (2)

I modified the 'Condition' column by replacing 'renoviert / saniert' (renovated/modernized) with 'neuwertig' (like-new condition).

This was done to:
- Reduce the number of distinct values for better data consistency.
- Standardize similar meanings, as 'renoviert / saniert' and 'neuwertig' convey nearly the same condition of the property.



In [ ]:
df['Condition'].value_counts(dropna=False)

,count
Condition,
NaN,2104
neuwertig,354
renoviert / saniert,270
Erstbezug,174
Gepflegt,104


In [ ]:
df[df["Condition"].str.contains("renoviert / saniert", na=False)].index.tolist()
df.loc[indices, "Condition"]

,Condition
2000,NaN
2014,NaN
2087,NaN
2090,NaN
2140,NaN
2148,NaN
2161,NaN
2467,NaN
2541,NaN
2559,NaN


In [ ]:
indices = df[df["Condition"].str.contains("renoviert / saniert", na=False)].index.tolist()
df.loc[indices, "Condition"] = "neuwertig"

In [ ]:
df['Condition'].value_counts(dropna=False)

,count
Condition,
NaN,2104
neuwertig,624
Erstbezug,174
Gepflegt,104


# One-hot-encoding transformations (1)

## "Heating type" column transformation

I transformed and categorized the 'Heating type' column using one-hot encoding. Each heating type was converted into a separate binary column, where a value of 1 indicates the presence of that specific heating type.

In [ ]:
df["Heating type"].value_counts(dropna=False)

,count
Heating type,
NaN,2091
Fernwärme,493
Gas,174
Elektro,53
Öl,34
"Fernwärme, Luft-/Wasser-Wärmepumpe",28
"Fernwärme, Gas",27
Luft-/Wasser-Wärmepumpe,16
"Elektro, Gas",14


In [ ]:
df["Heating_Elektro"] = np.nan
df["Heating_Fernwaerme"] = np.nan
df["Heating_Gas"] = np.nan
df["Heating_Oel"] = np.nan
df["Heating_Luft-/Wasser-Waermepumpe"] = np.nan
df["Heating_Blockheizkraftwerk"] = np.nan
df["Heating_Solar"] = np.nan
df["Heating_Erdwaerme"] = np.nan

indices = df[df["Heating type"].str.contains("Elektro, Gas, Luft-/Wasser-Wärmepumpe", na=False)].index.tolist()
df.loc[indices, ["Heating_Elektro", "Heating_Gas", "Heating_Luft-/Wasser-Waermepumpe", "Heating_Blockheizkraftwerk", "Heating_Solar", "Heating_Fernwaerme", "Heating_Oel"]] = [1, 1, 1,  np.nan, np.nan, np.nan, np.nan]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Erdwärme, Solar, Luft-/Wasser-Wärmepumpe", na=False)].index.tolist()
df.loc[indices, ["Heating_Erdwaerme", "Heating_Solar", "Heating_Luft-/Wasser-Waermepumpe"]] = [1, 1, 1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Erdwärme, Luft-/Wasser-Wärmepumpe", na=False)].index.tolist()
df.loc[indices, ["Heating_Erdwaerme", "Heating_Luft-/Wasser-Waermepumpe"]] = [1, 1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Elektro, Luft-/Wasser-Wärmepumpe", na=False)].index.tolist()
df.loc[indices, ["Heating_Elektro", "Heating_Luft-/Wasser-Waermepumpe"]] = [1, 1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Gas, Luft-/Wasser-Wärmepumpe", na=False)].index.tolist()
df.loc[indices, ["Heating_Gas", "Heating_Luft-/Wasser-Waermepumpe"]] = [1, 1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Fernwärme, Luft-/Wasser-Wärmepumpe", na=False)].index.tolist()
df.loc[indices, ["Heating_Fernwaerme", "Heating_Luft-/Wasser-Waermepumpe"]] = [1, 1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Luft-/Wasser-Wärmepumpe", na=False)].index.tolist()
df.loc[indices, ["Heating_Luft-/Wasser-Waermepumpe"]] = [1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Fernwärme, Gas, Blockheizkraftwerk", na=False)].index.tolist()
df.loc[indices, ["Heating_Fernwaerme", "Heating_Gas", "Heating_Blockheizkraftwerk"]] = [1, 1, 1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Gas, Blockheizkraftwerk", na=False)].index.tolist()
df.loc[indices, ["Heating_Gas", "Heating_Blockheizkraftwerk"]] = [1, 1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Fernwärme, Blockheizkraftwerk", na=False)].index.tolist()
df.loc[indices, ["Heating_Fernwaerme", "Heating_Blockheizkraftwerk"]] = [1, 1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Blockheizkraftwerk", na=False)].index.tolist()
df.loc[indices, ["Heating_Blockheizkraftwerk"]] = [1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Fernwärme, Erdwärme", na=False)].index.tolist()
df.loc[indices, ["Heating_Fernwaerme", "Heating_Erdwaerme"]] = [1, 1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Fernwärme, Elektro", na=False)].index.tolist()
df.loc[indices, ["Heating_Fernwaerme", "Heating_Elektro"]] = [1, 1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Fernwärme, Gas", na=False)].index.tolist()
df.loc[indices, ["Heating_Fernwaerme", "Heating_Gas"]] = [1, 1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Erdwärme, Solar", na=False)].index.tolist()
df.loc[indices, ["Heating_Erdwaerme", "Heating_Solar"]] = [1, 1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Erdwärme", na=False)].index.tolist()
df.loc[indices, ["Heating_Erdwaerme"]] = 1
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Elektro, Gas", na=False)].index.tolist()
df.loc[indices, ["Heating_Elektro", "Heating_Gas"]] = [1, 1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Gas, Solar", na=False)].index.tolist()
df.loc[indices, ["Heating_Solar", "Heating_Gas"]] = [1, 1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Gas, Öl", na=False)].index.tolist()
df.loc[indices, ["Heating_Oel", "Heating_Gas"]] = [1, 1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Gas", na=False)].index.tolist()
df.loc[indices, ["Heating_Gas"]] = 1
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Elektro, Öl", na=False)].index.tolist()
df.loc[indices, ["Heating_Oel", "Heating_Elektro"]] = [1, 1]
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Öl", na=False)].index.tolist()
df.loc[indices, ["Heating_Oel"]] = 1
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Elektro", na=False)].index.tolist()
df.loc[indices, ["Heating_Elektro"]] = 1
df.loc[indices, ["Heating type"]] = np.nan

indices = df[df["Heating type"].str.contains("Fernwärme", na=False)].index.tolist()
df.loc[indices, ["Heating_Fernwaerme"]] = 1
df.loc[indices, ["Heating type"]] = np.nan

df.drop(columns=['Heating type', 'availability'], inplace=True)

## Heating system" column transformation

I transformed and categorized the 'Heating system' column using one-hot encoding. Each heating system type was converted into a separate binary column, where a value of 1 indicates the presence of that specific heating system.

In [ ]:
df["Heating system"].value_counts(dropna=False)

,count
Heating system,
NaN,2084
Zentralheizung,368
Fußbodenheizung,322
Zentralheizung: Fußbodenheizung,117
Etagenheizung,113
Etagenheizung: Fußbodenheizung,2


In [ ]:
df["Heatingsys_Zentralheizung"] = np.nan
df["Heatingsys_Fussbodenheizung"] = np.nan
df["Heatingsys_Etagenheizung"] = np.nan

indices = df[df["Heating system"].str.contains("Etagenheizung: Fußbodenheizung", na=False)].index.tolist()
df.loc[indices, ['Heatingsys_Fussbodenheizung', 'Heatingsys_Etagenheizung']] = [1, 1]
df.loc[indices, ['Heating system']] = np.nan

indices = df[df["Heating system"].str.contains("Zentralheizung: Fußbodenheizung", na=False)].index.tolist()
df.loc[indices, ['Heatingsys_Fussbodenheizung', 'Heatingsys_Zentralheizung']] = [1, 1]
df.loc[indices, ['Heating system']] = np.nan

indices = df[df["Heating system"].str.contains("Fußbodenheizung", na=False)].index.tolist()
df.loc[indices, ['Heatingsys_Fussbodenheizung', 'Heating system']] = [1, np.nan]

indices = df[df["Heating system"].str.contains("Zentralheizung", na=False)].index.tolist()
df.loc[indices, ['Heatingsys_Zentralheizung', 'Heating system']] = [1, np.nan]

indices = df[df["Heating system"].str.contains("Etagenheizung", na=False)].index.tolist()
df.loc[indices, ['Heatingsys_Etagenheizung', 'Heating system']] = [1, np.nan]

df.drop(columns=['Heating system'], inplace=True)

# New column "time_lim (in months)"

I extracted the rental duration from the 'time_lim' column and moved it to a new column called 'time_lim (in months)' to explicitly include the time unit in the column name. After this transformation, I removed the original 'time_lim' column to maintain clarity and consistency in the dataset.

In [ ]:
df["time_lim (in months)"] = np.nan

indices = df[df["time_lim"].str.contains("6 Monate", na=False)].index.tolist()
df.loc[indices, ['time_lim (in months)', "time_lim"]] = [6, np.nan]

indices = df[df["time_lim"].str.contains("12 Monate", na=False)].index.tolist()
df.loc[indices, ['time_lim (in months)', "time_lim"]] = [12, np.nan]

indices = df[df["time_lim"].str.contains("2 Monate", na=False)].index.tolist()
df.loc[indices, ['time_lim (in months)', "time_lim"]] = [2, np.nan]

indices = df[df["time_lim"].str.contains("1 Monat", na=False)].index.tolist()
df.loc[indices, ['time_lim (in months)', "time_lim"]] = [1, np.nan]

indices = df[df["time_lim"].str.contains("3 Monate", na=False)].index.tolist()
df.loc[indices, ['time_lim (in months)', "time_lim"]] = [3, np.nan]

indices = df[df["time_lim"].str.contains("24 Monate", na=False)].index.tolist()
df.loc[indices, ['time_lim (in months)', "time_lim"]] = [24, np.nan]

indices = df[df["time_lim"].str.contains("18 Monate", na=False)].index.tolist()
df.loc[indices, ['time_lim (in months)', "time_lim"]] = [18, np.nan]

indices = df[df["time_lim"].str.contains("yes", na=False)].index.tolist()
df.loc[indices, ["time_lim"]] = np.nan

df.drop(columns=['time_lim'], inplace=True)

## New column "accessible"

This code creates a new column called "accessible" in a DataFrame (df) to flag listings that are wheelchair accessible or barrier-free based on specific keywords found in the 'Feature_*' columns.

Steps taken:
- Creates a binary indicator for accessibility by initializing the 'accessible' column with NaN.
- Checks for accessibility-related keywords ('Rollstuhlgerecht' for wheelchair accessibility and 'Barrierefrei' for barrier-free access) in selected 'Feature_*' columns.
- Assigns a value of 1 in the 'accessible' column if a listing contains any of these keywords.
- Listings without these keywords remain as NaN, indicating that no accessibility information was found.

In [ ]:
df["accessible"] = np.nan

In [ ]:
indices = df[df["Feature_3"].str.contains("Rollstuhlgerecht", na=False)].index.tolist()
df.loc[indices, ["accessible", "Feature_3"]]

,accessible,Feature_3
245,NaN,Rollstuhlgerecht
3024,NaN,"Barrierefrei, Rollstuhlgerecht"


In [ ]:
indices = df[df["Feature_3"].str.contains("Rollstuhlgerecht", na=False)].index.tolist()
df.loc[indices, ["accessible"]] = 1

indices = df[df["Feature_4"].str.contains("Barrierefrei", na=False)].index.tolist()
df.loc[indices, ["accessible"]] = 1

indices = df[df["Feature_5"].str.contains("Barrierefrei", na=False)].index.tolist()
df.loc[indices, ["accessible"]] = 1

indices = df[df["Feature_5"].str.contains("Rollstuhlgerecht", na=False)].index.tolist()
df.loc[indices, ["accessible"]] = 1

indices = df[df["Feature_6"].str.contains("Barrierefrei", na=False)].index.tolist()
df.loc[indices, ["accessible"]] = 1

indices = df[df["Feature_7"].str.contains("Barrierefrei", na=False)].index.tolist()
df.loc[indices, ["accessible"]] = 1

indices = df[df["Feature_8"].str.contains("Barrierefrei", na=False)].index.tolist()
df.loc[indices, ["accessible"]] = 1

# One-hot-encoding tranformations (2)

## "floor covering" column transformation

This code transforms the "floor covering" column into multiple binary (one-hot encoded) columns, assigning a 1 to indicate the presence of a specific floor covering type in each listing. It also removes the original "floor covering" column after transformation.

Steps taken:
- Creates one-hot encoded columns for each floor covering type (Floor_cov_*).
- Detects multiple floor covering types and assigns 1 to the relevant columns.
- Processes listings with single floor coverings separately.
- Removes the original "floor covering" column after transformation.

In [ ]:
df["Floor_cov_Parkett"] = np.nan
df["Floor_cov_Fliesen"] = np.nan
df["Floor_cov_Laminat"] = np.nan
df["Floor_cov_Holzdielen"] = np.nan
df["Floor_cov_Kunststoff"] = np.nan
df["Floor_cov_Linoleum"] = np.nan
df["Floor_cov_Stein"] = np.nan

indices = df[df['floor covering'].str.contains("Bodenbelag: Fliesen, Holzdielen, Kunststoff, Linoleum", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Fliesen', 'Floor_cov_Holzdielen', 'Floor_cov_Kunststoff', 'Floor_cov_Linoleum']] = [np.nan, 1, 1, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Fliesen, Parkett, Kunststoff", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Fliesen', 'Floor_cov_Parkett', 'Floor_cov_Kunststoff']] = [np.nan, 1, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Fliesen, Parkett, Laminat, Kunststoff", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Fliesen', 'Floor_cov_Parkett', 'Floor_cov_Kunststoff', 'Floor_cov_Laminat']] = [np.nan, 1, 1, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Fliesen, Holzdielen, Laminat", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Fliesen', 'Floor_cov_Holzdielen', 'Floor_cov_Laminat']] = [np.nan, 1, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Fliesen, Holzdielen, Parkett, Stein", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Fliesen', 'Floor_cov_Holzdielen', 'Floor_cov_Parkett', 'Floor_cov_Stein']] = [np.nan, 1, 1, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Fliesen, Parkett, Linoleum", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Fliesen', 'Floor_cov_Parkett', 'Floor_cov_Linoleum']] = [np.nan, 1, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Fliesen, Laminat, Kunststoff", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Fliesen', 'Floor_cov_Laminat', 'Floor_cov_Kunststoff']] = [np.nan, 1, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Fliesen, Holzdielen, Parkett", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Fliesen', 'Floor_cov_Holzdielen', 'Floor_cov_Parkett']] = [np.nan, 1, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Fliesen, Parkett, Laminat", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Fliesen', 'Floor_cov_Parkett', 'Floor_cov_Laminat']] = [np.nan, 1, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Holzdielen, Parkett", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Holzdielen', 'Floor_cov_Parkett']] = [np.nan, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Parkett, Stein", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Stein', 'Floor_cov_Parkett']] = [np.nan, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Kunststoff, Linoleum", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Kunststoff', 'Floor_cov_Linoleum']] = [np.nan, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Fliesen, Linoleum", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Fliesen', 'Floor_cov_Linoleum']] = [np.nan, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Laminat, Linoleum", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Laminat', 'Floor_cov_Linoleum']] = [np.nan, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Fliesen, Parkett", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Fliesen', 'Floor_cov_Parkett']] = [np.nan, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Fliesen, Laminat", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Fliesen', 'Floor_cov_Laminat']] = [np.nan, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Fliesen, Holzdielen", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Fliesen', 'Floor_cov_Holzdielen']] = [np.nan, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Fliesen, Kunststoff", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Fliesen', 'Floor_cov_Kunststoff']] = [np.nan, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Fliesen, Stein", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Fliesen', 'Floor_cov_Stein']] = [np.nan, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Laminat, Kunststoff", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Kunststoff', 'Floor_cov_Laminat']] = [np.nan, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Parkett, Laminat", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Parkett', 'Floor_cov_Laminat']] = [np.nan, 1, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Holzdielen, Linoleum", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Holzdielen', 'Floor_cov_Linoleum']] = [np.nan, 1, 1]

indices = df[df['floor covering'].str.contains("Parkett", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Parkett']] = [np.nan, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Fliesen", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Fliesen']] = [np.nan, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Holzdielen", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Holzdielen']] = [np.nan, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Laminat", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Laminat']] = [np.nan, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Kunststoff", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Kunststoff']] = [np.nan, 1]

indices = df[df['floor covering'].str.contains("Bodenbelag: Linoleum", na=False)].index.tolist()
df.loc[indices, ['floor covering', 'Floor_cov_Linoleum']] = [np.nan, 1]

df.drop(columns=['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7', 'Feature_8', 'floor covering'], inplace=True)

# Fill some NaN values

## Fill NaN values for "Warmmiete (€/month)" column



I calculated missing "Warmmiete (€/month)" values by summing different rental cost components and updated specific rows with manual values.

Steps
- Calculating "Warmmiete (€/month)" using available rental costs.
- Only updating missing (NaN) values to avoid overwriting existing data.
- Manually correcting "Warmmiete" for specific rows where necessary.

In [ ]:
# Calculate 'Warmmiete (€/month)' using temporary NaN replacement
calculated_warmmiete = (
    df['Kaltmiete zzgl. Nebenkosten (€/month)'].fillna(0) +
    df['Nebenkosten (€/month)'].fillna(0) +
    df['Heizkosten_2 (€/month)'].fillna(0) +
    df['Miete pro Stellplatz (€/month)'].fillna(0)
)

# Update only NaN values in 'Warmmiete (€/month)' column while preserving NaNs in other columns
df.loc[pd.isna(df['Warmmiete (€/month)']), 'Warmmiete (€/month)'] = calculated_warmmiete[pd.isna(df['Warmmiete (€/month)'])]

df.loc[1848, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)', 'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [2700, 2950, np.nan, np.nan]
df.loc[3501, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)', 'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [1299, 1699, np.nan, np.nan]
df.loc[1898, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)', 'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [4830 , 5477.87, 376.20, 271.67]
df.loc[3205, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)', 'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [2070 , 2570, 250, 250]
df.loc[439, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)', 'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [2054.40 , 1699.47, 187.76, 169.17]
df.loc[1329, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)', 'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [1974 , 2416, 442, 164]
df.loc[3420, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)', 'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [3590 , 3996.32, 243.79, 162.53]

<ipython-input-142-04ab25ac4999>:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Kaltmiete zzgl. Nebenkosten (€/month)'].fillna(0) +
<ipython-input-142-04ab25ac4999>:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Nebenkosten (€/month)'].fillna(0) +


## Fill NaN values for the column "Surface Area (m^2)"

There were only a few missing values in the 'Surface Area (m²)' column that were not captured by the web scraper. I manually updated these missing values by opening the corresponding listing URLs and retrieving the correct information.

In [ ]:
df.loc[2191, ["Surface Area (m^2)"]] = 21
df.loc[2196, ["Surface Area (m^2)"]] = 40
df.loc[3361, ["Surface Area (m^2)"]] = 20
df.loc[3578, ["Surface Area (m^2)"]] = 22
df.loc[3596, ["Surface Area (m^2)"]] = 22
df.loc[3604, ["Surface Area (m^2)"]] = 17
df.loc[3642, ["Surface Area (m^2)"]] = 15
df.loc[1040, ["Surface Area (m^2)"]] = 19
df.loc[1063, ["Surface Area (m^2)"]] = 21
df.loc[1897, ["Surface Area (m^2)"]] = 22
df.loc[1912, ["Surface Area (m^2)"]] = 16
df.loc[2098, ["Surface Area (m^2)"]] = 10
df.loc[2110, ["Surface Area (m^2)"]] = 12
df.loc[2160, ["Surface Area (m^2)"]] = 12
df.loc[2169, ["Surface Area (m^2)"]] = 9
df.loc[2177, ["Surface Area (m^2)"]] = 62
df.loc[2185, ["Surface Area (m^2)"]] = 21
df.loc[2190, ["Surface Area (m^2)"]] = 12

df.loc[227, ["Surface Area (m^2)"]] = 19
df.loc[912, ["Surface Area (m^2)"]] = 20
df.loc[1037, ["Surface Area (m^2)"]] = 16
df.loc[1041, ["Surface Area (m^2)"]] = 185
df.loc[1079, ["Surface Area (m^2)"]] = 26
df.loc[1081, ["Surface Area (m^2)"]] = 21
df.loc[1083, ["Surface Area (m^2)"]] = 31
df.loc[1086, ["Surface Area (m^2)"]] = 38
df.loc[1097, ["Surface Area (m^2)"]] = 21

df.loc[1101, ["Surface Area (m^2)"]] = 55
df.loc[1111, ["Surface Area (m^2)"]] = 26
df.loc[1989, ["Surface Area (m^2)"]] = 42
df.loc[2842, ["Surface Area (m^2)"]] = 30
df.loc[3024, ["Surface Area (m^2)"]] = 20
df.loc[3227, ["Surface Area (m^2)"]] = 10
df.loc[3471, ["Surface Area (m^2)"]] = 23.2
df.loc[3481, ["Surface Area (m^2)"]] = 31

df.loc[3592, ["Surface Area (m^2)"]] = 25.1
df.loc[1831, ["Surface Area (m^2)"]] = 6
df.loc[313, ["Surface Area (m^2)"]] = 6
df.loc[328, ["Surface Area (m^2)"]] = 7
df.loc[387, ["Surface Area (m^2)"]] = 7
df.loc[960, ["Surface Area (m^2)"]] = 8.5
df.loc[2103, ["Surface Area (m^2)"]] = 7

## Fill NaN values for "Kaltmiete zzgl. Nebenkosten (€/month)" column

I filled in missing values for 'Kaltmiete zzgl. Nebenkosten (€/month)' by estimating them using 'Price per sqm' and 'Surface Area (m²)', and manually corrected certain rows based on verified listings.

Steps taken:
- Automatically Calculated Missing Rent Values:
  - If 'Kaltmiete zzgl. Nebenkosten (€/month)' was missing (NaN), and both 'Price per sqm' and 'Surface Area (m²)' were available:
Rent was estimated as:
  ```
'Kaltmiete zzgl. Nebenkosten (€/month)' = 'Price per sqm' * 'Surface Area (m²)'
  ```
  - This ensured only missing values were updated while preserving existing data.
- Manually Corrected Specific Rows:
  - For listings with known rent values (verified via their URLs), I directly updated 'Kaltmiete', 'Warmmiete', 'Nebenkosten', and other relevant columns.
  - The manually updated values include row-specific corrections for 'Deposit', 'Miete pro Stellplatz (€/month)', and 'Heizkosten_2 (€/month)'.
- Ensured 'Nebenkosten' Was Set for Row 666:
  - Specifically, row 666 was updated to ensure it had the correct Nebenkosten (service charges) = 300€.

In [ ]:
 # Create a mask for rows where 'Kaltmiete zzgl. Nebenkosten (€/month)' is NaN
mask = pd.isna(df['Kaltmiete zzgl. Nebenkosten (€/month)'])

# Additional condition to ensure 'Price per sqm' and 'Surface Area (m^2)' are NOT NaN
valid_rows = mask & pd.notna(df['Price per sqm']) & pd.notna(df['Surface Area (m^2)'])

# Apply calculation only to valid rows
df.loc[valid_rows, 'Kaltmiete zzgl. Nebenkosten (€/month)'] = df.loc[valid_rows, 'Price per sqm'] * df.loc[valid_rows, 'Surface Area (m^2)']


df.loc[57, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)',
            'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)',
            'Price per sqm', "Deposit"]] = [520, 635, 70, np.nan, 43.70, 1560]

df.loc[213, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)',
            'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)',
            'Price per sqm', 'Miete pro Stellplatz (€/month)']] = [900, 1180, 280, 150, 58.52, 100]

df.loc[439, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)',
            'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)',
            'Price per sqm', 'Miete pro Stellplatz (€/month)', "Deposit"]] = [1699, 2054, 185, 169, 21.50, 237, 5098]

df.loc[704, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)',
            'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)',
            'Price per sqm']] = [1990, 2230, 120, 120, 28.07]

df.loc[709, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)',
            'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)',
            'Price per sqm']] = [2560, 2910, 175, 175, 24.94]

df.loc[726, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)',
            'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)',
            'Price per sqm']] = [2550, 2935, 192.50, 192.50, 23.04]

df.loc[748, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)',
            'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)',
            'Price per sqm']] = [1715, 3145, 130, 1300, 23.06]

df.loc[777, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)',
            'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)',
            'Price per sqm']] = [4740, 5310, 285, 285, 28.01]

df.loc[793, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)',
            'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)',
            'Price per sqm']] = [1945, 2210, 132.50, 132.50, 25.06]

df.loc[828, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)',
            'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)',
            'Price per sqm']] = [2925, 3325, 200, 200, 25.05]

df.loc[837, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)',
            'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)',
            'Price per sqm']] = [2630, 2960, 165, 165, 27.06]

df.loc[838, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)',
            'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)',
            'Price per sqm']] = [2120, 2385, 132.50, 132.50, 27.05]

df.loc[2048, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)',
            'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)',
            'Price per sqm']] = [2270, 2615, 172.50, 172.50, 23.06]

df.loc[2695, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)',
            'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)',
            'Price per sqm']] = [1860, 2235, 375, np.nan, 22.35]

df.loc[3248, ['Kaltmiete zzgl. Nebenkosten (€/month)', 'Warmmiete (€/month)',
            'Nebenkosten (€/month)', 'Heizkosten_2 (€/month)',
            'Price per sqm']] = [600, 715, 70, np.nan, 50.42]

df.loc[666, ["Nebenkosten (€/month)"]] = 300

# Check for erreneous values (1)

I performed outlier detection, error correction, and missing value imputation.

Steps taken:
- Detect "Nebenkosten" outliers and manually corrects some values.
- Remove unrealistic "Surface Area" values (e.g., 1m², 2m²).
- Compute "Price per sqm" where missing using Kaltmiete or Warmmiete.
- Calculate missing "Kaltmiete" from "Warmmiete" where possible.
- Remove extreme "Price per sqm" outliers by dropping rows.

In [ ]:
#Check outliers for different columns, here Nebenkosten
column_name = 'Nebenkosten (€/month)'

# Calculate mean and standard deviation
mean = df[column_name].mean()
std_dev = df[column_name].std()

# Identify outliers (values more than 2 standard deviations from the mean)
outlier_indices = df[(df[column_name] < mean - 2 * std_dev) | (df[column_name] > mean + 2 * std_dev)].index.tolist()

In [ ]:
df.loc[outlier_indices, ['Nebenkosten (€/month)', 'Heizkosten_2 (€/month)', 'URL']]

,Nebenkosten (€/month),Heizkosten_2 (€/month),URL
28,1583.0,NaN,https://www.immowelt.de/expose/73679151-0828-4...
48,714.0,NaN,https://www.immowelt.de/expose/2e21f5a6-b0e5-4...
58,900.0,NaN,https://www.immowelt.de/expose/64d80f7e-bccb-4...
90,2552.33,162.0,https://www.immowelt.de/expose/e4e952f5-aade-4...
291,810.0,NaN,https://www.immowelt.de/expose/51e7bd2e-a661-4...
392,780,204.0,https://www.immowelt.de/expose/1fba70d7-02c4-4...
532,705.0,NaN,https://www.immowelt.de/expose/0679beef-e786-4...
606,705.0,NaN,https://www.immowelt.de/expose/b1fa2ca4-231b-4...
612,1037.1,NaN,https://www.immowelt.de/expose/6fd664af-56ba-4...
796,1317.0,NaN,https://www.immowelt.de/expose/0cdf5dc2-493f-4...


In [ ]:
df.loc[90, ['Nebenkosten (€/month)']] = np.nan
df.loc[113, ['Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [1465, np.nan]
df.loc[462, ['Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [618, np.nan]
df.loc[987, ['Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [534, np.nan]
df.loc[1963, ['Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [550, np.nan]
df.loc[1982, ['Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [1072, np.nan]
df.loc[1996, ['Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [675, np.nan]
df.loc[2231, ['Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [646, np.nan]
df.loc[2699, ['Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [550, np.nan]
df.loc[3101, ['Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [800, np.nan]
df.loc[3254, ['Nebenkosten (€/month)']] = np.nan
df.loc[3367, ['Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [880, np.nan]
df.loc[3478, ['Nebenkosten (€/month)', 'Heizkosten_2 (€/month)']] = [550.0, np.nan]

df.loc[918, ['Surface Area (m^2)']] = 156


#Strange sqm values (1² and 2²):
df.loc[[2218, 2344, 2488, 393], 'Surface Area (m^2)'] = np.nan

condition = (
    df['Price per sqm'].isna() &
    df['Kaltmiete zzgl. Nebenkosten (€/month)'].notna() &
    df['Surface Area (m^2)'].notna()
)

# Get row indices where conditions are met
indices_list = df[condition].index.tolist()

# compute price/sqm values
df.loc[indices_list, 'Price per sqm'] = df.loc[indices_list, 'Kaltmiete zzgl. Nebenkosten (€/month)'] / df.loc[indices_list, 'Surface Area (m^2)']

#compute some possible Katlmiete prices:
df.loc[3540, 'Kaltmiete zzgl. Nebenkosten (€/month)'] = df.loc[3540, 'Warmmiete (€/month)'] - df.loc[3540, 'Nebenkosten (€/month)'] - df.loc[3540, 'Heizkosten_2 (€/month)']
df.loc[3047, 'Kaltmiete zzgl. Nebenkosten (€/month)'] = df.loc[3047, 'Warmmiete (€/month)'] - df.loc[3047, 'Nebenkosten (€/month)']

condition = (
    df['Price per sqm'].isna() &
    df['Kaltmiete zzgl. Nebenkosten (€/month)'].notna() &
    df['Surface Area (m^2)'].notna()
)

# Get row indices where conditions are met
indices_list = df[condition].index.tolist()

# compute price/sqm values
df.loc[indices_list, 'Price per sqm'] = df.loc[indices_list, 'Kaltmiete zzgl. Nebenkosten (€/month)'] / df.loc[indices_list, 'Surface Area (m^2)']



condition = (
    df['Price per sqm'].isna() &
    df['Warmmiete (€/month)'].notna() &
    df['Surface Area (m^2)'].notna()
)

# Get row indices where conditions are met
indices_list = df[condition].index.tolist()

#compute price/sqm
df.loc[indices_list, 'Price per sqm'] = df.loc[indices_list, 'Warmmiete (€/month)'] / df.loc[indices_list, 'Surface Area (m^2)']

# Correcting strange price/sqm vallues (more than 2sd away from the mean)
df.loc[216, "Price per sqm"] = 1600/73
df = df.drop([1529, 1826, 1772, 1500, 3446, 27, 313, 1390, 3297, 1574, 304, 1498, 3173, 3198, 1432, 1194,  2542, 2861, 3322, 1012, 1831, 2679, 2615, 2586], axis=0)

<ipython-input-149-6c0d03fb2ed5>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[39.25 32.5]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[indices_list, 'Price per sqm'] = df.loc[indices_list, 'Kaltmiete zzgl. Nebenkosten (€/month)'] / df.loc[indices_list, 'Surface Area (m^2)']


## Filtering out WG listings

During data analysis, I noticed that several unusual values (i.e. high rent for small surface areas) were caused by listings offering shared-flat options. These listings were distorting the rent/surface area ratio by inflating the price per square meter.

To ensure the dataset accurately represents full rental properties, I decided to filter out these shared-flat listings.

In [ ]:
#removing all private room of WG announces:
df = df[df['Surface Area (m^2)'] >= 15]

In addition to removing shared-flat listings, I also dropped all listings with a surface area of 22m² or smaller, even if they were not explicitly marked as 'WG-geeignet'.

Reasoning:
- After manually reviewing several listings via their URLs, I found that flats of 22m² or smaller were highly likely to be private rooms in larger shared apartments, rather than full rental units.

In [ ]:
# Define the condition to filter the rows
condition = df['WG-geeignet'].notna()

# Get row indices where the condition is met
indices_list = df[condition].index.tolist()

# Drop rows within indices_list where 'Surface Area (m^2)' is ≤ 22
df = df.drop(df.loc[indices_list][df['Surface Area (m^2)'] <= 22].index).reset_index(drop=True)

<ipython-input-151-bade5993441b>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df.drop(df.loc[indices_list][df['Surface Area (m^2)'] <= 22].index).reset_index(drop=True)


# New column "district_name"

Initially, I wanted to include not only the district ('Bezirk') information but also the subdistrict ('Ortsteil') for each listing.

However, I later realized that 'Bezirk' already refers to the district level in Berlin. What I actually intended with the new column was to capture the subdistrict (Ortsteil) instead.

In [ ]:
#Create a column with district names
# Function to extract district names
def extract_district(address):
    # Remove street names and numbers
    address = address.split(",")[-2].strip() if "," in address else address.split("(")[0].strip()

    # Remove "Berlin" if it appears at the end
    address = address.replace(" Berlin", "").strip()

    return address

# Apply the function
df["district_name"] = df["Address"].apply(extract_district)

# Checking for errenous values (2)

I performed outlier detection and correction for the "Deposit" column.

Steps taken:
- Identify "Deposit" outliers using mean ± 2 standard deviations.
- Remove incorrect "Deposit" values by setting them to NaN.
- Display "Deposit" outliers for further manual verification.
- Manually correct "Deposit" for specific listings.


In [ ]:
#Check outliers for deposit values
column_name = 'Deposit'

# Calculate mean and standard deviation
mean = df[column_name].mean()
std_dev = df[column_name].std()

# Identify outliers (values more than 2 standard deviations from the mean)
outlier_indices = df[(df[column_name] < mean - 2 * std_dev) | (df[column_name] > mean + 2 * std_dev)].index.tolist()

In [ ]:
df.loc[outlier_indices, ["Deposit", "URL"]]

,Deposit,URL
231,6.206970e+15,https://www.immowelt.de/expose/3ec2addc-1fea-4...
1649,1.002912e+16,https://www.immowelt.de/expose/7ce17caa-7103-4...
1692,3.429450e+16,https://www.immowelt.de/expose/3a3dd7d6-e225-4...
1769,6.164970e+15,https://www.immowelt.de/expose/ecd9f934-dec5-4...
2012,6.986970e+15,https://www.immowelt.de/expose/ad94eecf-82f2-4...
2236,3.569970e+16,https://www.immowelt.de/expose/9fdf7e14-7392-4...


In [ ]:
#Correcting outlier deposit values:
df.loc[[882, 880], ["Deposit"]]= [np.nan, np.nan]
df.loc[[231, 1649, 1692, 1769, 2012, 2236], ["Deposit"]]= [6207, 9525, 3297, 6165, 6987, 3570]
df.loc[[882, 880], ["Deposit"]]= [np.nan]
df.loc[[1701, 1481, 318], ["Deposit"]]= [np.nan]

In [ ]:
df.loc[outlier_indices, ["Deposit", "URL"]]

,Deposit,URL
231,6207.0,https://www.immowelt.de/expose/3ec2addc-1fea-4...
1649,9525.0,https://www.immowelt.de/expose/7ce17caa-7103-4...
1692,3297.0,https://www.immowelt.de/expose/3a3dd7d6-e225-4...
1769,6165.0,https://www.immowelt.de/expose/ecd9f934-dec5-4...
2012,6987.0,https://www.immowelt.de/expose/ad94eecf-82f2-4...
2236,3570.0,https://www.immowelt.de/expose/9fdf7e14-7392-4...


# Doing some manipulations to a smoother import in Power BI

I cleaned and optimized the dataset for Power BI by:

- Handling missing values (NaN replacements).
- Dropping irrelevant rows (removing WG listings).
- Ensuring proper data types (e.g., converting text, numerical, boolean, and datetime columns).
- Handling conversion issues (logging failed numerical conversions).


In [ ]:
#Lighter csv file with NaN instead of 0 values
df["Multiple floors"].replace(0, np.nan, inplace=True)


# Find indices where 'Title' contains "WG"
indices = df[df['Title'].str.contains("WG", na=False)].index.tolist()
# Drop the rows permanently
df.drop(index=indices, inplace=True)





#COnverting datatypes to allow a smooth use of the df in Power BI:
# Convert text columns explicitly to object (ensures no unintended changes)
text_columns = [
    "Title", "Heizkosten_1 (€/month)", "Address", "Year of Construction",
    "Condition", "Real Estate 3", "URL", "Bezirk", "district_name"
]
df[text_columns] = df[text_columns].astype(str)

# Convert monetary and numerical columns from object to float64
float_columns = [
    "Kaltmiete zzgl. Nebenkosten (€/month)", "Warmmiete (€/month)", "Nebenkosten (€/month)",
    "Price per sqm"
]

# Dictionary to store failed conversions
conversion_issues = {}

for col in float_columns:
    # Create a mask to identify rows where conversion fails
    mask = pd.to_numeric(df[col], errors='coerce').isna() & df[col].notna()

    # If there are conversion issues, store row indices
    if mask.any():
        conversion_issues[col] = df.loc[mask, col].index.tolist()

    # Proceed with conversion
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Display conversion issues (if any)
if conversion_issues:
    print("⚠️ Conversion issues found in the following columns:")
    for col, rows in conversion_issues.items():
        print(f"❌ Column: {col}, Affected Rows: {rows[:10]}{'...' if len(rows) > 10 else ''}")  # Show first 10 rows for preview
else:
    print("✅ All numerical conversions were successful!")

# Convert columns with 1.0 as True and NaN as False to boolean
bool_columns = [
    "Multiple floors", "lim_inhabitants", "balcony", "furnished", "garden",
    "Terrasse", "pets_allowed", "Garage", "shower", "bath", "Einbaukueche",
    "kitchenette", "lift", "open kitchen", "basement", "window in bathroom",
    "WG-geeignet", "Wohnen auf Zeit in title", "nur mit Wohnberechtigungsschein in title",
    "Heating_Elektro", "Heating_Fernwaerme", "Heating_Gas", "Heating_Oel",
    "Heating_Luft-/Wasser-Waermepumpe", "Heating_Blockheizkraftwerk",
    "Heating_Solar", "Heating_Erdwaerme", "Heatingsys_Zentralheizung",
    "Heatingsys_Fussbodenheizung", "Heatingsys_Etagenheizung",
    "accessible", "Floor_cov_Parkett", "Floor_cov_Fliesen",
    "Floor_cov_Laminat", "Floor_cov_Holzdielen", "Floor_cov_Kunststoff",
    "Floor_cov_Linoleum", "Floor_cov_Stein"
]
for col in bool_columns:
    df[col] = df[col].apply(lambda x: True if x == 1 else False)

# List of problematic numeric columns in Power BI
numeric_columns = [
    "Kaltmiete zzgl. Nebenkosten (€/month)", "Warmmiete (€/month)", "Nebenkosten (€/month)",
    "Heizkosten_2 (€/month)", "Miete pro Stellplatz (€/month)", "Rooms_number",
    "Surface Area (m^2)", "pics_number", "energy_efficiency", "Price per sqm",
    "Deposit", "days_until_available", "time_lim (in months)"
]

# Function to clean numeric columns (remove spaces, special characters)
def clean_numeric_column(col):
    return pd.to_numeric(df[col].astype(str).str.replace(r"[^\d.-]", "", regex=True), errors="coerce")

# Apply cleaning function to all numeric columns
for col in numeric_columns:
    df[col] = clean_numeric_column(col)

# Convert datetime columns explicitly
df["availability_date"] = pd.to_datetime(df["availability_date"], errors="coerce")

✅ All numerical conversions were successful!


<ipython-input-158-752d707a8a81>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Multiple floors"].replace(0, np.nan, inplace=True)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2294 entries, 0 to 2300
Data columns (total 63 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   Title                                     2294 non-null   object        
 1   Kaltmiete zzgl. Nebenkosten (€/month)     1262 non-null   float64       
 2   Warmmiete (€/month)                       2294 non-null   float64       
 3   Nebenkosten (€/month)                     1043 non-null   float64       
 4   Heizkosten_1 (€/month)                    2294 non-null   object        
 5   Heizkosten_2 (€/month)                    570 non-null    float64       
 6   Miete pro Stellplatz (€/month)            189 non-null    float64       
 7   Rooms_number                              2286 non-null   float64       
 8   Surface Area (m^2)                        2294 non-null   float64       
 9   Floor                              

# Exporting cleaned dataframe to a CSV file

In [ ]:
df.to_csv('updated_listings.csv', index=False)
files.download('updated_listings.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# General information after data cleaning

This rental property dataset contains 2294 rental property listings (rows) from Berlin with 63 columns providing details on rent, property characteristics, location, and amenities.

- **Title:**
  - Title of the property listing from the real estate website (string).    
- **Kaltmiete zzgl. Nebenkosten (€/month):**
  - Cold rent in euros per month (continuous numeric).
- **Warmmiete (€/month):**
  - Warm rent (base rent + heating and service costs) in euros per month (continuous numeric).     
- **Nebenkosten (€/month):**
  - Additional service costs (e.g., maintenance, water) in euros per month (continuous numeric).     
- **Heizkosten_1 (€/month):**
  - Indicates whether heating costs are included in warm rent or additional costs (string).     
- **Heizkosten_2 (€/month):**
  - Heating costs in euros per month (continuous numeric).     
- **Miete pro Stellplatz (€/month):**
  - Monthly rent for a parking space in euros (continuous numeric).      
- **Rooms_number:**   
  - Number of rooms (discrete numeric). Includes whole numbers (e.g., 1, 2, 3) and half numbers (e.g., 1.5, 2.5) indicating half-rooms.   
- **Surface Area (m^2):**
  - Total surface area of the property in square meters (continuous numeric).     
- **Floor:**
  - The floor level the property is located on (discrete numeric).
  - Half numbers (e.g., 1.5) most often indicate properties spanning multiple floors, where an average value is assigned.
    - Example: A property spanning from the 1st to the 2nd floor → (1+2)/2 = 1.5.
  - Whole numbers do not necessarily mean the property is on a single floor.
    - Example: A property spanning from the 1st to the 3rd floor → (1+2+3)/3 = 2 results in a whole number, but the property still spans multiple floors.
  - 0 represents the ground floor (Erdgeschoss).  
- **Address:**
  - Full address of the property (string).       
- **Zip Code:**
  - Postal code of the property location (discrete numeric).       
- **pics_number:**     
  - Number of images available in the listing (discrete numeric).
- **energy_efficiency:**  
  - Energy efficiency rating of the property (discrete numeric).
    - Coded from 0 (A+) to 8 (H).    
- **Year of Construction:**   
  -  Binary category:
    - "Neubau" (New construction, built after 1945).
    - "Altbau" (Old construction, built in 1945 or earlier).   
- **Condition:**    
  - 3 possible values:
    - "neuwertig" (like new).
    - "Erstbezug" (first occupancy).
    - "Gepflegt" (well maintained).    
- **Price per sqm:**    
  - Rent per square meter (continuous numeric).
  - Most values were web-scraped.
  - If missing, it was estimated using the following logic:
    - Calculated as Kaltmiete / Surface Area when possible.
    - Otherwise estimated from Warmmiete / Surface Area.
- **Real Estate 3:**    
  - The real estate agency managing the listing (string).
    - Agencies with fewer than 5 listings are grouped under "other".  
- **URL:**     
  - The web link to the listing (string).   
- **Multiple floors:**
  - Property spans multiple floors.
    - 1 = Yes, 0 = No or not specified.       
- **Bezirk:**
  - The district in Berlin where the property is located (string).       
- **Deposit:**
  - Security deposit amount (continuous numeric).     
- **lim_inhabitants:**
  - Indicates whether the listing has a restriction on the number of tenants.
    - 1 = Yes, 0 = No or not specified.        
- **balcony:**
  - Includes a balcony and/or a winter garden
    - 1 = Yes, 0 = No or not specified.      
- **furnished:**
  - The property is furnished.
    - 1 = Yes, 0 = No or not specified.     
- **garden:**
  - Has a garden.
    - 1 = Yes, 0 = No or not specified.         
- **Terrasse:**
  - Has a terrace.
    - 1 = Yes, 0 = No or not specified.        
- **pets_allowed:**
  - Pets are allowed.
    - 1 = Yes, 0 = No or not specified.
- **Garage:**
  - Includes any parking facility (garage, carport, or parking space).
    - 1 = Yes, 0 = No or not specified.     
- **shower:**
  - Has a shower.
    - 1 = Yes, 0 = No or not specified.        
- **bath:**
  - Has a bathtub.
    - 1 = Yes, 0 = No or not specified.         
- **Einbaukueche:**
  - Includes a built-in kitchen.
    - 1 = Yes, 0 = No or not specified.         
- **kitchenette:**
  - Includes a kitchenette.  
    - 1 = Yes, 0 = No or not specified.         
- **lift:**
  - Has an elevator.
    - 1 = Yes, 0 = No or not specified.          
- **open kitchen:**
  - Has an open kitchen
    - 1 = Yes, 0 = No or not specified.       
- **basement:**
  - Has a basement.
    - 1 = Yes, 0 = No or not specified.         
- **window in bathroom:**
  - Bathroom has a window.
    - 1 = Yes, 0 = No or not specified.        
- **WG-geeignet:**
  - Suitable for shared living (WG = Wohngemeinschaft).
    - 1 = Yes, 0 = No or not specified.        
- **Wohnen auf Zeit in title:**
  - Indicates temporary housing in the title.
    - 1 = Yes, 0 = No.        
- **nur mit Wohnberechtigungsschein in title:**
  - Requires a WBS (public housing eligibility certificate).
    - 1 = Yes, 0 = No.         
- **availability_date:**
  - The earliest move-in date (datetime format).
- **days_until_available:**
  - Number of days between the web scraping date (12.02.2025) and the move-in date.     
- **Heating_Elektro:**
  - Electric heating.  
    - 1 = Yes, 0 = No or not specified.        
- **Heating_Fernwaerme:**
  - District heating.  
    - 1 = Yes, 0 = No or not specified.        
- **Heating_Gas:**
  - Gas heating.  
    - 1 = Yes, 0 = No or not specified.        
- **Heating_Oel:**
  - Oil heating.
    - 1 = Yes, 0 = No or not specified.           
- **Heating_Luft-/Wasser-Waermepumpe:**
  - Air/water heat pump.
    - 1 = Yes, 0 = No or not specified.        
- **Heating_Blockheizkraftwerk:**
  - Block heating plant.  
    - 1 = Yes, 0 = No or not specified.         
- **Heating_Solar:**
  - Solar heating.  
    - 1 = Yes, 0 = No or not specified.        
- **Heating_Erdwaerme:**
  - Geothermal heating.  
    - 1 = Yes, 0 = No or not specified.         
- **Heatingsys_Zentralheizung:**
  - Central heating system.   
    - 1 = Yes, 0 = No or not specified.        
- **Heatingsys_Fussbodenheizung:**
  - Underfloor heating system.
    - 1 = Yes, 0 = No or not specified.          
- **Heatingsys_Etagenheizung:**
  - Floor heating system.
    - 1 = Yes, 0 = No or not specified.          
- **accessible:**
  - Indicates barrier-free and/or wheelchair-accessible properties.
    - 1 = Yes, 0 = No or not specified.        
- **time_lim (in months):**
  - If applicable, the maximum rental duration in months (continuous numeric).    
- **Floor_cov_Parkett:**
  - Parquet flooring.
    - 1 = Yes, 0 = No or not specified.          
- **Floor_cov_Fliesen:**
  - Tiled flooring.  
    - 1 = Yes, 0 = No or not specified.         
- **Floor_cov_Laminat:**
  - Laminate flooring.  
    - 1 = Yes, 0 = No or not specified.         
- **Floor_cov_Holzdielen:**
  - Wooden plank flooring.
    - 1 = Yes, 0 = No or not specified.          
- **Floor_cov_Kunststoff:**
  - Plastic flooring.  
    - 1 = Yes, 0 = No or not specified.         
- **Floor_cov_Linoleum:**
  - Linoleum flooring.   
    - 1 = Yes, 0 = No or not specified.        
- **Floor_cov_Stein:**
  - Stone flooring.  
    - 1 = Yes, 0 = No or not specified.         
- **district_name:**
  - The subdistrict (Ortsteil) where the property is located (string).    

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2294 entries, 0 to 2300
Data columns (total 63 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   Title                                     2294 non-null   object        
 1   Kaltmiete zzgl. Nebenkosten (€/month)     1262 non-null   float64       
 2   Warmmiete (€/month)                       2294 non-null   float64       
 3   Nebenkosten (€/month)                     1043 non-null   float64       
 4   Heizkosten_1 (€/month)                    2294 non-null   object        
 5   Heizkosten_2 (€/month)                    570 non-null    float64       
 6   Miete pro Stellplatz (€/month)            189 non-null    float64       
 7   Rooms_number                              2286 non-null   float64       
 8   Surface Area (m^2)                        2294 non-null   float64       
 9   Floor                              

In [ ]:
df.shape

(2294, 63)